In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

In [ ]:
tf.__version__

## Configuration of Neural Network

In [ ]:
class eAISimple:
    def __init__(self, wieght_func=None, bias_func=None):

        # Input
        self.img_size = 64
        self.input_num_channels = 3
        self.img_size_flat = self.img_size * \
                             self.img_size * \
                             self.input_num_channels
        x = tf.placeholder(tf.float32, shape=[None, self.img_size_flat], name='x')
        self.input = tf.reshape(x, [-1, self.img_size, self.img_size, self.input_num_channels])
        

        # Convolutional Layer 1.
        self.filter_size1 = 3          # Convolution filters are 3 x 3 pixels.
        self.num_filters1 = 8         # There are 8 of these filters.

        # Convolutional Layer 2.
        self.filter_size2 = 3          # Convolution filters are 3 x 3 pixels.
        self.num_filters2 = 16         # There are 16 of these filters.

        # Convolutional Layer 3.
        self.filter_size3 = 3          # Convolution filters are 3 x 3 pixels.
        self.num_filters3 = 16         # There are 16 of these filters.

        # Fully-connected layer.
        self.fc_size = 3             # Number of neurons in fully-connected layer.
        
        self.wfunc = wieght_func
        self.bfunc = bias_func
        
    def print_object(self, name):
        print(tf.__version__)
        print(name)

    def new_weights(self, shape ):
        
        return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
        
        
    def new_biases(self,length,layer=0,baises_name=None ):
        
        if (baises_name != None):
            baises_name=baises_name+"_Biases"
        else:
            baises_name = "Biases_"+str(layer)
            
        return tf.Variable(tf.constant(0.05, shape=[length]))   
    
    def new_conv_layer(self,
                       input,              # The previous layer.
                       num_input_channels, # Num. channels in prev. layer.
                       filter_size,        # Width and height of each filter.
                       num_filters,        # Number of filters.
                       layer_num=0,
                       weight_func=None,
                       layer_name=None,
                       use_pooling=True):  # Use 2x2 max-pooling.

        
        # Padding Shape
        pad_top = 0
        pad_bottom = 0
        pad_left = 0
        pad_right = 0

        #Create Layer Name
        if ( layer_name != None ):
            conv_name = layer_name+"_conv"
            pool_name = layer_name+"_pool"
            relu_name = layer_name+"_relu"
        else:
            conv_name = layer_name
            pool_name = layer_name
            relu_name = layer_name
            
        # Shape of the filter-weights for the convolution.
        # This format is determined by the TensorFlow API.
        weights_shape = [filter_size, filter_size, num_input_channels, num_filters]
        
        # Create new weights aka. filters with the given shape.
        # Create new weights and biases.
        if ( self.wfunc != None or self.bfunc != None ):
            weights =self.wfunc(layer_num ,weights_name=layer_name+str("_weights"))
            biases = self.bfunc(layer_num, bias_name=layer_name+str("_bias"))
            
        else:    
                        
            weights = self.new_weights(weights_shape)
            # Create new biases, one for each filter.
            biases = self.new_biases(length=num_filters,
                                 layer=layer_num)
        print(weights)
        
        # Create the TensorFlow operation for convolution.
        # Note the strides are set to 1 in all dimensions.
        # The first and last stride must always be 1,
        # because the first is for the image-number and
        # the last is for the input-channel.
        # But e.g. strides=[1, 2, 2, 1] would mean that the filter
        # is moved 2 pixels across the x- and y-axis of the image.
        # The padding is set to 'SAME' which means the input image
        # is padded with zeroes so the size of the output is the same.
        if ( tf.__version__  == '1.8.0' ):
            layer = tf.nn.conv2d(input=input,
                                 filter=weights,
                                 name=conv_name,
                                 strides=[1, 1, 1, 1],
                                 padding="SAME")
        else:
            layer = tf.nn.conv2d(input=input,
                             filter=weights,
                             name=conv_name,
                             strides=[1, 1, 1, 1],
                             padding=[[0, 0], 
                                      [pad_top, pad_bottom], 
                                      [pad_left, pad_right], 
                                      [0, 0]])
        print(layer)
        # Add the biases to the results of the convolution.
        # A bias-value is added to each filter-channel.
        #layer += biases
            
        # Use pooling to down-sample the image resolution?
        if use_pooling:
            # This is 2x2 max-pooling, which means that we
            # consider 2x2 windows and select the largest value
            # in each window. Then we move 2 pixels to the next window.
            layer = tf.nn.max_pool(value=layer,
                                   name=pool_name,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding="SAME")

        # Rectified Linear Unit (ReLU).
        # It calculates max(x, 0) for each input pixel x.
        # This adds some non-linearity to the formula and allows us
        # to learn more complicated functions.
        layer = tf.nn.relu(layer,
                           name=relu_name)

        # Note that ReLU is normally executed before the pooling,
        # but since relu(max_pool(x)) == max_pool(relu(x)) we can
        # save 75% of the relu-operations by max-pooling first.

        # We return both the resulting layer and the filter-weights
        # because we will plot the weights later.
        # print('Weigths : '+ str(weights))
        
        print(layer)

        return layer, weights

    def flatten_layer(self, layer):
        # Get the shape of the input layer.
        layer_shape = layer.get_shape()

        # The shape of the input layer is assumed to be:
        # layer_shape == [num_images, img_height, img_width, num_channels]

        # The number of features is: img_height * img_width * num_channels
        # We can use a function from TensorFlow to calculate this.
        num_features = layer_shape[1:4].num_elements()

        # Reshape the layer to [num_images, num_features].
        # Note that we just set the size of the second dimension
        # to num_features and the size of the first dimension to -1
        # which means the size in that dimension is calculated
        # so the total size of the tensor is unchanged from the reshaping.
        layer_flat = tf.reshape(layer, [-1, num_features])

        # The shape of the flattened layer is now:
        # [num_images, img_height * img_width * num_channels]

        # Return both the flattened layer and the number of features.
        print (num_features)
        print (layer_flat)

        return layer_flat, num_features

    def new_fc_layer(self, input,          # The previous layer.
                     num_inputs,     # Num. inputs from prev. layer.
                     num_outputs,    # Num. outputs.
                     layer_num=4,
                     layer_name="FC",
                     use_relu=True): # Use Rectified Linear Unit (ReLU)?

        # Create new weights and biases.
        if ( self.wfunc != None or self.bfunc != None ):
            weights = self.wfunc(layer_num, layer_name+"weights")
            biases = self.bfunc(layer_num, layer_name+"bias")
        else:    
            weights = self.new_weights(shape=[num_inputs, num_outputs])
            biases = self.new_biases(length=num_outputs)
        
        print(weights)            
        # Calculate the layer as the matrix multiplication of
        # the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + 5

        # Use ReLU?
        if use_relu:
            layer = tf.nn.relu(layer,
                              name=layer_name)
        print (layer)

        return layer

    def network(self):
        print(self.input)
        # Layer 1
        print("Layer 1 :")
        self.layer, W1 = self.new_conv_layer(self.input, 
                       self.input_num_channels, 
                       self.filter_size1, 
                       self.num_filters1,
                       layer_name="Layer_1", 
                       layer_num=1)
        # Layer 2
        print("Layer 2 :")
        self.layer, W2 = self.new_conv_layer(self.layer, 
                       self.num_filters1, 
                       self.filter_size2, 
                       self.num_filters2,
                       layer_num=2,
                       layer_name="Layer_2",
                       use_pooling = False)
        # Layer 3
        print("Layer 3 :")
        self.layer, W3 = self.new_conv_layer(self.layer, 
                       self.num_filters2, 
                       self.filter_size3,
                       self.num_filters3,
                       layer_name="Layer_3",
                       layer_num=3)
        # Fully Connected
        print("Full Connected 1 :")
        self.layer, F = self.flatten_layer( self.layer )
        self.layer = self.new_fc_layer(self.layer, 
                                F,
                                self.fc_size)
        # Output
        self.y_pred = tf.nn.softmax(self.layer)
        self.y_pred_cls = tf.argmax(self.y_pred, axis=1)
        

In [ ]:
def Renesas_eAI_Weights (layer, weights_name ):
        
        r_shapes = {
            1:[8, 3, 3, 3],
            2:[16,8, 3, 3],
            3:[16,16,3, 3],
            4:[3136,3]
        }
        
        r_weights ={
            1:[0.12916739284992218,-0.09634407609701157,-0.002463021781295538,-0.14984747767448425,-0.19196732342243195,-0.15791405737400055,-0.020562658086419106,-0.13582563400268555,-0.13704067468643188,-0.022995563223958015,0.16236382722854614,0.250929057598114,-0.06617768853902817,0.03846248611807823,-0.19337204098701477,0.25768253207206726,-0.02520260587334633,-0.036114729940891266,-0.30627042055130005,-0.18681925535202026,-0.25855833292007446,-0.01983066461980343,0.05242019146680832,0.1902996301651001,0.12275553494691849,0.00868998933583498,-0.08308348804712296,-0.11687520146369934,0.1801159381866455,-0.08150005340576172,-0.22438083589076996,0.15172439813613892,0.027342069894075394,-0.3190034031867981,-0.2856859564781189,0.20134465396404266,-0.05594947934150696,0.27256330847740173,0.04644308611750603,-0.07668503373861313,0.15963898599147797,0.31631624698638916,-0.15175388753414154,-0.008040132001042366,-0.009392067790031433,0.07271559536457062,0.009544513188302517,0.14473918080329895,-0.3207956552505493,0.05121538043022156,0.23552045226097107,0.016968877986073494,-0.29631245136260986,0.18307429552078247,-0.12433957308530807,-0.183709055185318,0.022682644426822662,0.05226953700184822,0.1819179505109787,-0.28971925377845764,-0.17540739476680756,0.09576351195573807,-0.14996692538261414,-0.1065043956041336,-0.23381255567073822,0.08949266374111176,0.10680986195802689,-0.1420513391494751,-0.2520335614681244,0.1550697386264801,0.21001334488391876,0.05240846425294876,0.24865639209747314,-0.29781100153923035,0.22848887741565704,-0.024642709642648697,0.027335327118635178,-0.028720306232571602,0.19919303059577942,-0.05623586103320122,0.14217382669448853,0.058192573487758636,0.2236187905073166,-0.0973401740193367,-0.14917109906673431,0.16950540244579315,-0.01747984066605568,0.007735408376902342,0.2744283676147461,0.25228163599967957,-0.005012313835322857,-0.009880119003355503,-0.0988060012459755,-0.25196653604507446,-0.3644234836101532,-0.26978129148483276,0.04394698888063431,-0.31511712074279785,0.12170761823654175,0.263021320104599,0.06633343547582626,-0.19174496829509735,0.1924309879541397,-0.18587087094783783,-0.2001902461051941,0.07894428074359894,-0.04275978356599808,0.129738911986351,0.010494627989828587,0.191063791513443,-0.19326657056808472,-0.16608963906764984,-0.043261293321847916,0.049053989350795746,-0.1650196611881256,-0.08724338561296463,0.03952891379594803,-0.14486242830753326,0.006821164395660162,0.10919181257486343,0.11483392864465714,0.0738450214266777,0.11039386689662933,0.01315175648778677,0.12745121121406555,0.1107558086514473,0.14398033916950226,0.1122254729270935,-0.12586040794849396,0.03410201892256737,0.28660809993743896,0.20296350121498108,0.14430810511112213,0.09915462136268616,-0.11525025963783264,0.07454598695039749,-0.24582554399967194,0.06097223609685898,-0.09367962181568146,0.17235375940799713,-0.10590633004903793,-0.14055775105953217,0.13528607785701752,0.16191968321800232,-0.08581112325191498,-0.24988143146038055,0.24371279776096344,-0.27744024991989136,0.01733221486210823,0.08944098651409149,-0.09697993844747543,0.1064770445227623,0.11736112087965012,0.17897488176822662,-0.13813826441764832,-0.13116496801376343,-0.23336860537528992,0.04617488384246826,0.037710390985012054,-0.14105485379695892,0.10324551910161972,0.36170634627342224,0.10587954521179199,0.22600938379764557,-0.09064044803380966,-0.13076883554458618,0.09711268544197083,-0.23766471445560455,-0.18273690342903137,-0.04509435221552849,-0.24826043844223022,-0.11574036628007889,0.02853832207620144,-0.08265937864780426,0.0015687895938754082,-0.09033185988664627,0.26701104640960693,-0.0974513366818428,-0.24932822585105896,0.21448777616024017,-0.11382254958152771,-0.21012017130851746,-0.03180229291319847,-0.04041774570941925,-0.280539870262146,-0.193033367395401,0.0013168910518288612,0.1355787217617035,-0.20195090770721436,0.12993693351745605,-0.15037226676940918,0.23642608523368835,-0.0385139025747776,0.19961585104465485,0.19932474195957184,0.03774723783135414,-0.2591824531555176,0.21003076434135437,-0.13732142746448517,0.05846131592988968,0.2670944035053253,0.02321762964129448,-0.2501084804534912,-0.08183976262807846,-0.09215716272592545,-0.024853136390447617,0.1658424437046051,0.15762203931808472,-0.019831035286188126,0.33957424759864807,-0.10289052128791809,-0.2311757206916809,-0.2205057591199875,-0.2003358006477356,-0.2289041131734848,-0.06201670691370964],
            2:[-0.4047820270061493,-0.600922703742981,-0.35152173042297363,-0.1543342024087906,-0.4182316064834595,-0.385189950466156,-0.16742044687271118,-0.02253442443907261,-0.35333728790283203,-0.3029363751411438,-0.23771804571151733,-0.08867769688367844,-0.09452437609434128,-0.005823491606861353,0.09995637834072113,0.2784889042377472,0.21790482103824615,-0.038675758987665176,0.1656470000743866,-0.1929018646478653,-0.06332099437713623,-0.06377313286066055,0.07094787061214447,0.02756160870194435,0.02851182408630848,-0.06903468817472458,0.0960785374045372,0.22321255505084991,0.11821355670690536,-0.12621527910232544,0.038733597844839096,0.045423392206430435,0.33888524770736694,0.05374855548143387,0.31323957443237305,0.21433018147945404,-0.08472976088523865,-0.017874490469694138,-0.24095813930034637,0.1273360699415207,0.11616358906030655,-0.1365908533334732,-0.0008549442864023149,0.10500593483448029,0.03460658714175224,0.13623091578483582,-0.0549330897629261,-0.16526290774345398,0.21035674214363098,0.19450613856315613,0.17734026908874512,0.07146772742271423,0.006992978043854237,0.21826684474945068,-0.0838049128651619,0.06587385386228561,0.11190719902515411,0.052088186144828796,-0.18033543229103088,-0.07025482505559921,0.18051575124263763,-0.11844202876091003,-0.24942386150360107,-0.1153920516371727,-0.12924981117248535,-0.05578906089067459,-0.15917234122753143,0.0010886919917538762,0.04494273290038109,0.13450279831886292,0.24981553852558136,0.23183408379554749,0.1404723823070526,0.3222048878669739,0.3024350106716156,-0.0375438816845417,0.11786815524101257,0.042301908135414124,-0.07859779894351959,0.1843901425600052,0.1529553234577179,0.19033417105674744,-0.10177866369485855,0.07031247764825821,0.2718456983566284,0.2139607071876526,0.058438077569007874,-0.047741733491420746,0.08538611978292465,0.1709733009338379,0.17313089966773987,0.058920733630657196,0.1500106304883957,0.22407177090644836,0.15457741916179657,0.02057729661464691,0.1980287730693817,0.032271213829517365,0.07272892445325851,0.11354294419288635,0.01717454567551613,0.07278479635715485,-0.16631940007209778,-0.11018455028533936,0.11175289750099182,-0.1871928721666336,-0.09670121222734451,0.14680072665214539,0.013477299362421036,0.1429160088300705,0.03454951196908951,0.09116089344024658,0.09996906667947769,-0.16942964494228363,-0.06450578570365906,0.020582042634487152,-0.11451402306556702,0.29747340083122253,-0.04239378497004509,0.06881743669509888,0.12351745367050171,-0.058462515473365784,-0.043006956577301025,0.12157653272151947,-0.07600749284029007,0.009354635141789913,0.06623129546642303,-0.019395433366298676,0.3172769844532013,0.1899895817041397,-0.01900092139840126,0.08916646242141724,0.01616184040904045,0.14600728452205658,0.20792125165462494,-0.11544156819581985,-0.02529255673289299,0.04156604781746864,-0.14026309549808502,0.08251583576202393,-0.2145402878522873,-0.08404918760061264,-0.1392287164926529,-0.00864805281162262,-0.18040290474891663,0.05325430631637573,-0.09873735159635544,-0.10172083973884583,0.1028996929526329,-0.0047544026747345924,-0.050666920840740204,-0.030094420537352562,0.04810674488544464,0.15484946966171265,-0.11495649069547653,0.05332644283771515,0.27420130372047424,-0.009423952549695969,0.2013951689004898,0.04956618323922157,0.2161298543214798,0.24135836958885193,-0.22280876338481903,0.019802512601017952,0.13189856708049774,-0.06294792890548706,0.04848232492804527,0.09557080268859863,-0.0031873120460659266,0.041811034083366394,0.11291561275720596,-0.2093048095703125,-0.0780978575348854,-0.1438114047050476,0.023028869181871414,-0.20110324025154114,-0.026050375774502754,-0.011136992834508419,-0.0738511011004448,0.10153603553771973,-0.08788168430328369,-0.19440525770187378,-0.11696166545152664,-0.0074653225019574165,0.007793363183736801,-0.11999227851629257,0.034986648708581924,-0.071367047727108,0.1277504712343216,-0.1189684122800827,-0.3641199767589569,0.07296352833509445,0.07673387229442596,-0.21479874849319458,-0.15287774801254272,-0.021112516522407532,-0.012731136754155159,0.10792884975671768,-0.16314630210399628,0.052406761795282364,-0.031218793243169785,0.03932557627558708,-0.22479429841041565,-0.1832589954137802,-0.133122980594635,-0.17104844748973846,-0.17377541959285736,-0.042749471962451935,0.12513138353824615,0.17645719647407532,-0.10520348697900772,0.059186626225709915,0.07555229216814041,0.13068704307079315,-0.07985696941614151,0.021449366584420204,0.36232948303222656,0.34469136595726013,0.17975765466690063,0.15277397632598877,0.10316368192434311,0.2930019497871399,0.28813233971595764,0.019360335543751717,0.08576413244009018,-0.07871995866298676,0.11343991756439209,-0.07497087866067886,0.19866345822811127,0.028495198115706444,0.014371108263731003,-0.012064087204635143,0.155739888548851,-0.09293685108423233,0.2504010498523712,0.18168780207633972,0.079070083796978,0.14748762547969818,0.15935418009757996,0.08193819224834442,0.254437118768692,0.14238327741622925,-0.03369443863630295,0.3067469596862793,0.1418953686952591,0.30133992433547974,-0.00039390719030052423,0.09353236854076385,0.2929007411003113,-0.061285439878702164,-0.08383344113826752,0.2274733930826187,-0.22733598947525024,-0.030055085197091103,-0.1895008534193039,0.07104779779911041,-0.19898569583892822,-0.08717439323663712,0.04948034882545471,-0.04197223111987114,0.0025974391028285027,0.16820484399795532,-0.04296419769525528,-0.09589266031980515,-0.0088659031316638,0.031186729669570923,-0.12498894333839417,0.1241106390953064,0.16971935331821442,-0.1408349573612213,0.32244062423706055,0.09808823466300964,0.2743335962295532,0.11840862035751343,0.26669299602508545,0.3576982319355011,0.2978551983833313,0.10267230868339539,0.04251113906502724,-0.07649115473031998,0.1397218555212021,0.07604547590017319,-0.07472488284111023,-0.010759206488728523,-0.033044543117284775,0.14345036447048187,0.07634204626083374,-0.11112447828054428,-0.08568128943443298,-0.04755907878279686,-0.22622312605381012,0.17353521287441254,-0.034291647374629974,-0.04499232769012451,0.222502663731575,0.004662434104830027,0.15806637704372406,0.35348325967788696,0.19761204719543457,0.12262366712093353,0.05859372019767761,0.25574374198913574,0.1373005509376526,-0.2827928364276886,-0.24442625045776367,-0.06025635451078415,0.07536435127258301,-0.013043657876551151,-0.0789000391960144,-0.02595401741564274,-0.08766961097717285,-0.17452389001846313,-0.02079610526561737,0.10636557638645172,0.02667178027331829,-0.07220614701509476,-0.05803343653678894,-0.09331134706735611,-0.05588015541434288,0.18548361957073212,0.14190715551376343,0.029841342940926552,0.15545746684074402,0.14566411077976227,0.03088059276342392,0.09616324305534363,0.3335557281970978,-0.11480952799320221,-0.11257383227348328,-0.007660810835659504,-0.12389929592609406,-0.18830130994319916,-0.19869187474250793,-0.0815201923251152,0.06534280627965927,0.05082036182284355,-0.1700141578912735,-0.1050017848610878,0.008505171164870262,-0.1211908832192421,-0.17555342614650726,-0.14924268424510956,-0.05847107246518135,-0.06060079485177994,0.06011876091361046,0.25585824251174927,0.250749796628952,0.23293836414813995,-0.0021050323266535997,0.11987471580505371,0.18276268243789673,0.0012318516382947564,0.09505545347929001,-0.11738605052232742,0.07638487964868546,0.12153462320566177,0.20424269139766693,-0.2448405772447586,-0.015462223440408707,-0.11403106898069382,0.12387324124574661,0.2052852362394333,-0.0025572853628546,0.07423348724842072,-0.06424040347337723,0.15155597031116486,0.09003277122974396,0.058467622846364975,-0.08985709398984909,0.3087237775325775,0.3303040564060211,0.251663476228714,0.3112781047821045,0.2896786332130432,0.09996329247951508,0.17486754059791565,0.13089022040367126,0.18322795629501343,0.11064547300338745,-0.016195867210626602,-0.0691438689827919,0.08519361168146133,-0.030412033200263977,0.06203022971749306,0.07948204129934311,0.044181037694215775,-0.03674791753292084,-0.0014884232077747583,-0.16522549092769623,0.015456287190318108,-0.09355467557907104,0.28201431035995483,0.02746313065290451,-0.05058557540178299,-0.19008074700832367,-0.19660347700119019,0.03669970855116844,-0.08269721269607544,0.11245030164718628,-0.06288957595825195,-0.08503949642181396,0.17705515027046204,-0.04069170355796814,0.0904705673456192,-0.059405431151390076,0.3772645890712738,0.2046663910150528,0.22145861387252808,0.19347406923770905,0.11391406506299973,0.11346005648374557,0.04161899909377098,-0.0664220005273819,-0.09277605265378952,0.11400318145751953,0.06284678727388382,0.012304588221013546,-0.03932013362646103,0.07472497224807739,-0.09925822913646698,0.29891788959503174,0.28719115257263184,0.16413982212543488,0.01593909040093422,-0.11835717409849167,-0.10997471213340759,0.18708157539367676,0.1761426329612732,-0.04488643631339073,0.08809956163167953,-0.0586489774286747,0.04567262902855873,0.09292708337306976,0.04366692528128624,-0.07009514421224594,-0.03481626138091087,-0.12235584855079651,-0.18265846371650696,-0.12041191756725311,0.07834091037511826,-0.09971370548009872,0.1621970236301422,0.2158319354057312,0.17984943091869354,0.1060371920466423,0.206930473446846,0.09229584038257599,0.11775904893875122,0.07959465682506561,0.06401444971561432,-0.034432634711265564,0.12524737417697906,0.011548774316906929,-0.11517707258462906,-0.051305029541254044,-0.17684130370616913,-0.021487541496753693,0.0021110207308083773,-0.014816293492913246,-0.20726066827774048,-0.2688405215740204,-0.1386953741312027,-0.06419891119003296,-0.06048696115612984,-0.09340012818574905,-0.08555584400892258,-0.16253137588500977,-0.03844136744737625,-0.1634199321269989,0.11885188519954681,-0.004466531332582235,-0.0026428382843732834,0.04259606450796127,-0.13538981974124908,-0.13870956003665924,0.11831386387348175,-0.014800714328885078,-0.1066366508603096,0.030472852289676666,0.003983037080615759,-0.13377749919891357,0.12801356613636017,0.1300666779279709,-0.01395302452147007,0.1799575835466385,0.0820525586605072,-0.1833195835351944,-0.1499280482530594,-0.08096212148666382,-0.13947974145412445,-0.05227520689368248,-0.04142073169350624,-0.04051182046532631,-0.23856475949287415,-0.27532950043678284,0.051262930035591125,0.023166896775364876,-0.06399796158075333,0.01872246339917183,0.17612452805042267,0.07352437824010849,0.014210150577127934,0.20872418582439423,0.06808853894472122,0.05154430866241455,-0.003513760631904006,-0.15737491846084595,0.07905767112970352,-0.035500667989254,0.15042206645011902,0.01585977151989937,0.11151441931724548,-0.142924964427948,-0.08086130768060684,0.20807334780693054,0.17215652763843536,-0.2275223284959793,0.13906238973140717,0.0532078854739666,-0.3242129981517792,0.06715790182352066,0.12010439485311508,-0.11899665743112564,-0.061387598514556885,-0.31635749340057373,0.061803705990314484,-0.07262936234474182,-0.272050142288208,0.004901580046862364,-0.02893243171274662,-0.2348715215921402,0.1364859938621521,0.0835704505443573,-0.07781839370727539,0.04998192936182022,-0.005242250394076109,-0.2051897794008255,-0.062386296689510345,-0.12889327108860016,-0.04942057654261589,0.07760747522115707,0.052516911178827286,-0.06696333736181259,-0.19803868234157562,-0.0859011709690094,0.16456159949302673,0.07066704332828522,-0.06522059440612793,-0.018791813403367996,-0.13275064527988434,0.1492495834827423,0.23562149703502655,-0.17928633093833923,0.018886420875787735,0.2214149832725525,-0.06665361672639847,0.2026606947183609,0.18815970420837402,0.1359771341085434,0.21843169629573822,-0.08418413996696472,0.056050945073366165,0.23931171000003815,-0.011798614636063576,0.0041875955648720264,-0.060467954725027084,0.15481221675872803,0.17387692630290985,0.1793479174375534,-0.1126910001039505,0.13224153220653534,0.18100684881210327,-0.06970084458589554,-0.21053096652030945,-0.06793190538883209,-0.08128055185079575,0.23546504974365234,-0.10297629237174988,0.03170469030737877,0.17791609466075897,-0.026410026475787163,0.010671821422874928,-0.009781084023416042,0.19742989540100098,0.12356361746788025,-0.25278109312057495,-0.1779480278491974,-0.0852736383676529,-0.1424158811569214,0.07228328287601471,-0.001950721605680883,-0.16992154717445374,0.008845780976116657,-0.12256195396184921,-0.05686984211206436,-0.20297658443450928,-0.24948111176490784,0.1859225183725357,-0.09459888935089111,-0.07828939706087112,0.05290913209319115,0.21302878856658936,0.04193258658051491,0.03329233080148697,0.266033411026001,0.20007318258285522,0.26342806220054626,0.24825286865234375,-0.11654359102249146,-0.017661113291978836,-0.10988079756498337,-0.26856058835983276,-0.22060731053352356,-0.20530001819133759,-0.012036770582199097,0.11964355409145355,0.02271001972258091,-0.09160072356462479,0.1802593171596527,0.040529392659664154,0.1416347175836563,-0.1906288117170334,0.01504442933946848,0.3255811035633087,0.19118928909301758,0.286803275346756,0.16335749626159668,0.09552443027496338,-0.012354444712400436,-0.187861368060112,0.39457884430885315,0.32773512601852417,0.36398211121559143,0.1939685046672821,0.30219346284866333,0.29574987292289734,0.27142196893692017,0.07760446518659592,0.06663127988576889,0.006608712486922741,0.1703977733850479,0.008091202937066555,-0.007523751817643642,-0.08248306810855865,-0.010048216208815575,-0.12156118452548981,-0.2743132710456848,0.03331265598535538,-0.062040526419878006,0.001769815105944872,-0.09762687981128693,0.004092312417924404,0.13525432348251343,0.05065791308879852,0.012347549200057983,-0.12244635820388794,0.12254291027784348,0.0699019804596901,0.08667901903390884,0.00021976207790430635,0.1256822794675827,0.09465046226978302,0.04764791205525398,0.2530174255371094,0.0915634036064148,-0.013190270401537418,-0.14132845401763916,0.08250579982995987,-0.014300789684057236,-0.26812097430229187,-0.17602495849132538,0.01592857390642166,-0.3555920422077179,-0.1425863802433014,-0.2776366174221039,0.22868555784225464,0.16813649237155914,0.08846147358417511,0.13928240537643433,-0.16636896133422852,0.027279427275061607,0.10883501172065735,0.04321672394871712,0.2746845483779907,0.020911607891321182,-0.08549430221319199,0.12887170910835266,0.11550532281398773,0.1513070911169052,0.1821170151233673,-0.14722049236297607,0.16273479163646698,0.04084904119372368,0.09024889767169952,0.040733080357313156,-0.17611128091812134,0.09237103909254074,0.06862902641296387,-0.06750552356243134,-0.07860958576202393,-0.08598896861076355,-0.16929373145103455,0.0990547388792038,-0.03226238489151001,-0.10127955675125122,0.043438609689474106,0.3160439133644104,0.09323301166296005,0.17683418095111847,0.05902983993291855,0.3651682138442993,0.011991164647042751,-0.19837184250354767,-0.21760424971580505,-0.1067926362156868,0.03134652227163315,-0.09334520250558853,0.0025485558435320854,0.26096105575561523,0.19094666838645935,0.119053415954113,-0.05660462751984596,0.06285939365625381,0.01161333080381155,0.049594946205616,-0.0531708300113678,0.18406252562999725,0.022405385971069336,-0.019742904230952263,-0.07145699113607407,0.13543422520160675,0.055471811443567276,0.10586584359407425,0.05487377941608429,0.1834942102432251,-0.22205102443695068,0.20522749423980713,0.2193247526884079,0.057683903723955154,-0.15440267324447632,-0.13654272258281708,-0.06491834670305252,-0.05649677291512489,0.053138550370931625,0.07037905603647232,-0.07465434819459915,-0.1899126023054123,-0.013010287657380104,-0.0882141962647438,0.03978732228279114,0.2179044485092163,0.16248494386672974,0.14585056900978088,0.17313067615032196,0.0855613574385643,0.10241595655679703,0.006403019651770592,-0.12015964090824127,0.07922236621379852,0.014029604382812977,-0.11694465577602386,-0.024880293756723404,-0.1604422926902771,-0.17996297776699066,0.16231806576251984,0.12425538897514343,0.11892019957304001,0.17599789798259735,-0.12608139216899872,0.21162134408950806,0.19686396420001984,0.08871640264987946,0.05878932774066925,-0.001999231521040201,-0.057250697165727615,0.20778298377990723,0.051085349172353745,0.035364244133234024,0.12576588988304138,0.1951923966407776,0.3295254409313202,0.23156549036502838,0.20682618021965027,-0.03764458745718002,-0.14040784537792206,0.005076202563941479,-0.03853988274931908,0.05947275087237358,-0.07427641749382019,-0.06263009458780289,0.1602371484041214,0.04799297824501991,-0.03336375579237938,0.03701203688979149,0.15544627606868744,0.005681171547621489,0.0016606759745627642,-0.09793353825807571,0.203913614153862,0.1708153337240219,0.04708898067474365,0.2549017369747162,0.07659191638231277,-0.12688931822776794,0.07360619306564331,0.22184135019779205,0.13859005272388458,-0.0303499698638916,0.0036880888510495424,0.1323547661304474,-0.02288753353059292,-0.3028147220611572,-0.3619575500488281,0.11794906854629517,-0.10910247266292572,-0.1392534226179123,0.13092564046382904,-0.0124719999730587,-0.15542277693748474,0.11362144351005554,-0.27944400906562805,-0.323160320520401,-0.20207202434539795,-0.23884178698062897,-0.4251119792461395,0.06569553911685944,-0.16988548636436462,-0.32502397894859314,0.17305852472782135,-0.08326442539691925,0.0924515649676323,-0.08181541413068771,0.02731664665043354,-0.011895720846951008,0.015446186065673828,-0.013851255178451538,-0.0752139464020729,0.2454013079404831,-0.030570287257432938,-0.3585432171821594,-0.012098967097699642,-0.07606649398803711,-0.15529190003871918,-0.09473498165607452,-0.05125577375292778,-0.2203674614429474,0.1428346037864685,0.1398812234401703,0.0644703358411789,0.05786121264100075,0.2471173107624054,0.2489461600780487,0.2258489727973938,0.1948232352733612,0.21510030329227448,0.13920798897743225,-0.14881426095962524,-0.24814578890800476,0.004202451091259718,0.13036325573921204,-0.13596050441265106,0.14455153048038483,0.13601619005203247,0.16293969750404358,0.025365054607391357,-0.24468660354614258,-0.2255030870437622,-0.15859314799308777,0.047782089561223984,-0.031904131174087524,-0.1983865648508072,-0.11085408180952072,-0.25373321771621704,0.1657564342021942,0.06108853220939636,0.19329245388507843,-0.16523179411888123,0.01484704203903675,0.22500577569007874,0.055405549705028534,-0.11367958784103394,0.10011272132396698,-0.053952887654304504,0.09357014298439026,0.05624363198876381,-0.1370803415775299,0.11001097410917282,-0.16160236299037933,0.08162577450275421,-0.13583649694919586,-0.10286671668291092,-0.22612257301807404,-0.10130717605352402,0.07022803276777267,0.11727003753185272,0.02462214045226574,-0.04542497918009758,-0.10235082358121872,-0.011354994028806686,-0.2306830734014511,-0.1293981522321701,0.01835007406771183,9.155960287898779e-05,0.04503725469112396,0.04934201017022133,-0.014277584850788116,-0.12790916860103607,0.002047905931249261,-0.07921526581048965,0.021595679223537445,-0.0210549458861351,0.1642492711544037,-0.05610239878296852,-0.1335819512605667,0.10033687949180603,-0.041327912360429764,-0.06878012418746948,0.20767050981521606,-0.039455220103263855,0.03966083750128746,-0.055281225591897964,-0.03885580971837044,-0.24322836101055145,-0.03089168108999729,-0.13577674329280853,-0.019381510093808174,-0.2616040110588074,0.2083648294210434,0.07183189690113068,0.044736530631780624,0.1029229536652565,-0.08319847285747528,0.24239662289619446,0.17036838829517365,0.05697312951087952,-0.003176567843183875,-0.41033998131752014,-0.24296443164348602,-0.42626041173934937,-0.3683217465877533,-0.3788801431655884,-0.47445085644721985,-0.504490852355957,-0.4883323013782501,-0.2652338147163391,-0.06538931280374527,-0.20865026116371155,0.026556143537163734,-0.013267757371068,0.0339861698448658,0.11411723494529724,0.20428651571273804,0.23362480103969574,0.27298426628112793,-0.24498596787452698,-0.24211673438549042,-0.048439204692840576,-0.24317684769630432,-0.13659930229187012,-0.25049281120300293,-0.18012647330760956,-0.44833090901374817,-0.2620638310909271,-0.24601024389266968,-0.2879488468170166,-0.4830123782157898,-0.1216319128870964,-0.14358259737491608,-0.3455798327922821,-0.0796135812997818,-0.3625207245349884,-0.20380321145057678,-0.1755794882774353,-0.010389108210802078,-0.010194841772317886,-0.00980858039110899,-0.12185228615999222,0.03041805513203144,0.03529990464448929,0.15159592032432556,0.19726625084877014,0.15367840230464935,0.06451079994440079,-0.05262259766459465,0.11436965316534042,0.24151624739170074,0.029846109449863434,0.0584639273583889,0.21063977479934692,0.18046726286411285,-0.19893208146095276,0.03494013100862503,-0.017296677455306053,-0.3464149534702301,-0.19315452873706818,-0.13871097564697266,-0.2186017632484436,-0.334565669298172,-0.13939711451530457,-0.1606135070323944,0.03283633664250374,-0.20660357177257538,0.10452579706907272,-0.18301348388195038,0.08787092566490173,0.20343904197216034,0.25996342301368713,0.24455222487449646,-0.011147424578666687,0.06088384985923767,-0.012337671592831612,0.2257722020149231,0.1005353108048439,0.049465958029031754,0.029274744912981987,0.13253530859947205,0.2633093297481537,0.07015174627304077,0.12732844054698944,-0.049463268369436264,0.023179499432444572,0.14980332553386688,0.0560869425535202,0.0022191540338099003,-0.01949622668325901,-0.13452792167663574,0.09078629314899445,-0.00449239369481802,0.21530789136886597,-0.007143442984670401,0.27487388253211975,0.010227212682366371,0.09033834934234619,0.13433673977851868,0.03395652770996094,0.20330651104450226,0.26142701506614685,0.23621252179145813,-0.03761889412999153,-0.04949300363659859,0.1488053947687149,-0.16800467669963837,0.05513317510485649,0.09743284434080124,-0.12056447565555573,0.11640656739473343,0.00481021124869585,0.09925568848848343,0.19182683527469635,-0.061430346220731735,0.0924559086561203,-0.09525813907384872,0.15552036464214325,0.09166381508111954,0.07381124794483185,-0.06890463829040527,0.14715029299259186,-0.11764034628868103,-0.13052047789096832,0.00732782855629921,-0.17718537151813507,-0.19185879826545715,0.10076139122247696,0.3677171766757965,0.22996623814105988,0.39516136050224304,0.27356263995170593,0.3794006407260895,0.2522141635417938,0.3379915952682495,0.2512240409851074,-0.20653453469276428,-0.06840487569570541,-0.19009442627429962,-0.19134977459907532,-0.27301445603370667,-0.26641881465911865,-0.3487073481082916,-0.0961056798696518,-0.072386234998703,0.015477494336664677,0.24882012605667114,0.06509008258581161,0.3070417642593384,0.3341117203235626,0.23522959649562836,0.1065870076417923,0.2386075109243393,0.2922267019748688,0.22309108078479767,0.1612289994955063,0.008675525896251202,-0.1100093349814415,0.09650398045778275,-0.13129062950611115,0.10582804679870605,-0.011816061101853848,0.12564152479171753,0.259966641664505,0.09175965934991837,0.20370076596736908,0.13317863643169403,0.13624899089336395,0.05638977140188217,0.0752764642238617,0.07034869492053986,-0.03705565258860588,-0.10475311428308487,-0.12760601937770844,-0.017317919060587883,-0.04170486703515053,0.12351588904857635,0.19804249703884125,0.0010218940442427993,-0.17565572261810303,-0.12103499472141266,0.05878554284572601,-0.008278034627437592,-0.025385884568095207,0.1109561175107956,-0.07223495841026306,0.04572939872741699,0.07893921434879303,-0.011921013705432415,-0.0398828350007534,0.19209346175193787,-0.08462264388799667,-0.0663125291466713,0.11750920861959457,0.048079732805490494,0.13732260465621948,0.14951342344284058,-0.05713025853037834,-0.10214236378669739,0.014401372522115707,-0.05329021438956261,0.09442596882581711,0.18457579612731934,0.11904837191104889,0.3263087570667267,-0.03544868156313896,0.14472275972366333,0.10005971044301987,0.09463009238243103,-0.04099161550402641,0.07183834910392761,-0.04351472854614258,-0.1097685769200325,-0.07146701961755753,-0.24976316094398499,-0.21196389198303223,-0.07807699590921402],
            3:[-0.0968756154179573,0.033399295061826706,-0.11530894786119461,-0.13819602131843567,-0.00796281173825264,0.0005981802241876721,0.11324911564588547,-0.02593669295310974,-0.19774939119815826,0.1314420849084854,-0.15697245299816132,-0.17137284576892853,0.041834235191345215,0.10275556892156601,-0.08566229045391083,0.06167580559849739,-0.07513343542814255,-0.11729585379362106,0.20092886686325073,0.09995162487030029,0.21796800196170807,0.15702958405017853,0.12478131800889969,-0.09984913468360901,0.09822021424770355,0.13632918894290924,0.06365849077701569,0.08729976415634155,0.29007619619369507,0.0639510452747345,-0.01837441697716713,0.03861105069518089,0.06032465770840645,0.1050480306148529,0.13821566104888916,-0.04669342562556267,0.011541184037923813,-0.17512717843055725,-0.1664077341556549,-0.020274678245186806,0.11241622269153595,0.09403346478939056,0.10321580618619919,0.10532687604427338,0.008428212255239487,0.09852452576160431,-0.033012937754392624,0.12020813673734665,0.13766171038150787,-0.03968934342265129,0.012980003841221333,0.10833899676799774,-0.03780128061771393,-0.03346783667802811,0.20089782774448395,-0.021606089547276497,-0.05461018905043602,-0.029811309650540352,0.12185683846473694,-0.08310340344905853,0.0806693509221077,0.17768721282482147,-0.08199543505907059,0.05494549870491028,0.06665965914726257,0.005255246069282293,0.04667414352297783,0.02315485291182995,0.03465826064348221,0.2644304931163788,0.16680195927619934,0.0929371640086174,-0.08592823892831802,-0.19381392002105713,0.031087562441825867,-0.20986612141132355,-0.04111771658062935,-0.04409679397940636,0.018821442499756813,0.0048002335242927074,-0.04223494976758957,0.060325827449560165,-0.0739206075668335,0.1255391538143158,0.06672732532024384,-0.069373719394207,0.11947564035654068,-0.11508101969957352,-0.03224669396877289,-0.07731764018535614,0.07322647422552109,-0.06330729275941849,0.03438117355108261,-0.14553984999656677,0.004477920476347208,-0.1159113273024559,-0.05086188390851021,0.04419679194688797,-0.1825500726699829,0.20387306809425354,0.03469660505652428,0.029717424884438515,0.30020540952682495,0.22967831790447235,-0.019813314080238342,0.3633044958114624,0.3217490613460541,0.060213200747966766,0.04416663199663162,-0.023271778598427773,0.07349967956542969,0.03976477310061455,0.0031422381289303303,-0.06564778089523315,0.08758426457643509,0.15985403954982758,-0.09362476319074631,-0.20048031210899353,-0.20601961016654968,-0.14879027009010315,-0.20233489573001862,-0.0522865355014801,0.018968941643834114,-0.07199673354625702,-0.1545136272907257,-0.008402514271438122,-0.0055109355598688126,0.16821202635765076,0.05777032673358917,0.04649065062403679,-0.01467433013021946,0.24583955109119415,0.004591901786625385,0.09990983456373215,0.13760831952095032,-0.06557776033878326,0.10515549778938293,0.1324595808982849,0.060479797422885895,0.014284586533904076,-0.0230371356010437,0.03925008326768875,0.000509998993948102,-0.17991481721401215,0.18832336366176605,0.16078229248523712,0.2136562615633011,0.11798428744077682,0.11957541108131409,-0.1655786782503128,0.04663367569446564,-0.10832881182432175,-0.026065925136208534,-0.07294437289237976,0.025036469101905823,-0.06154392659664154,0.14152517914772034,-0.11539088934659958,0.06935635209083557,-0.05500853806734085,0.19006898999214172,-0.12553933262825012,0.15664538741111755,0.016531379893422127,0.08495819568634033,0.09429879486560822,-0.08077657222747803,-0.01699679344892502,-0.12677417695522308,0.03258417174220085,-0.07411223649978638,0.025499166920781136,0.10476873815059662,0.025357242673635483,0.07172634452581406,0.15894712507724762,0.11277288943529129,0.24892190098762512,0.27474161982536316,0.03226858749985695,0.13069535791873932,-0.04908038303256035,0.12099843472242355,-0.057271186262369156,-0.03681643679738045,0.17291758954524994,0.1528288573026657,-0.04796944558620453,0.12102621793746948,0.010084198787808418,0.03429244086146355,0.04669788107275963,0.16719216108322144,0.2107773870229721,0.1721612811088562,-0.05504244938492775,0.07040233165025711,-0.15616632997989655,-0.17021775245666504,0.040365610271692276,0.016918933019042015,-0.008881366811692715,0.0238290224224329,0.09440486878156662,-0.23868583142757416,-0.20171748101711273,-0.04620129242539406,-0.2225150316953659,0.06715048104524612,-0.1388462632894516,-0.244726300239563,-0.08397950977087021,-0.07814420759677887,0.10430961847305298,-0.03772915154695511,0.1310180425643921,0.1063169613480568,0.18520572781562805,0.1741054654121399,0.4427196681499481,0.6068803668022156,0.40885835886001587,0.312921941280365,0.4568479657173157,0.286211222410202,-0.11619344353675842,-0.03488536179065704,-0.25955650210380554,0.12675906717777252,0.026166051626205444,-0.15848711133003235,0.014236889779567719,0.13348132371902466,-0.08451323211193085,-0.004965533968061209,0.23796816170215607,0.10294752568006516,0.28399208188056946,0.01838594488799572,0.026566745713353157,-0.17546840012073517,-0.279200941324234,0.008105866611003876,-0.03374725580215454,-0.23226773738861084,0.06728231906890869,-0.09561437368392944,-0.19972728192806244,0.03075297549366951,-0.16253837943077087,-0.1307121366262436,-0.23170384764671326,-0.08137630671262741,-0.0386747308075428,0.16811546683311462,-0.16654643416404724,-0.10510264337062836,0.057326920330524445,-0.10694225877523422,0.043467335402965546,-0.0682067722082138,0.08975721150636673,-0.013036884367465973,-0.19392777979373932,-0.06277476251125336,-0.300035685300827,-0.4076213240623474,-0.3670974671840668,-0.454202264547348,-0.4040997326374054,0.05511391907930374,-0.08390714973211288,-0.09921322762966156,-0.08705804497003555,-0.09942682832479477,-0.04322439804673195,0.3427925705909729,0.13299955427646637,0.08808671683073044,-0.13965781033039093,-0.13899073004722595,-0.02688051201403141,0.15064746141433716,-0.08780591189861298,-0.06480953842401505,0.25192883610725403,0.08217515051364899,-0.05231405422091484,0.03784660995006561,0.20096494257450104,0.18113109469413757,-0.12160142511129379,-0.16279222071170807,-0.027210861444473267,-0.2555887699127197,-0.2493116855621338,-0.2897208333015442,0.15161734819412231,-0.09417899698019028,-0.07193322479724884,-0.09043057262897491,-0.0022253254428505898,0.1377454400062561,-0.11269441246986389,-0.10288642346858978,-0.18067114055156708,-0.011503228917717934,0.011902029626071453,0.004337429068982601,-0.12845994532108307,-0.18116840720176697,-0.16702145338058472,-0.037812259048223495,0.08351777493953705,-0.11872658133506775,-0.051671676337718964,0.08223040401935577,-0.0845847800374031,-0.1750958263874054,-0.08633629977703094,-0.040800366550683975,0.13840726017951965,0.05805734544992447,0.07447447627782822,0.4920392334461212,0.25204572081565857,0.41218724846839905,0.5264866948127747,0.6024478673934937,0.41932809352874756,0.4194110333919525,0.4606908857822418,0.4788576364517212,0.1310616433620453,-0.07809598743915558,0.13188260793685913,-0.029412001371383667,0.11265313625335693,0.2358538955450058,-0.2492426186800003,-0.15768007934093475,-0.07654418051242828,-0.003747631097212434,0.016713947057724,0.010845376178622246,0.07096263021230698,0.083169125020504,-0.07355926185846329,-0.014557218179106712,-0.014083816669881344,-0.12687329947948456,0.06801752746105194,-0.010725532658398151,0.04653511568903923,0.24483129382133484,0.15548944473266602,0.05889071524143219,-0.023681553080677986,0.07521823793649673,0.27933838963508606,0.15699538588523865,0.1500830203294754,0.07783492654561996,-0.10815969854593277,-0.11760695278644562,0.017946727573871613,0.07739290595054626,0.1923423558473587,0.12313435226678848,0.02714165300130844,0.24775849282741547,0.059990592300891876,0.043619152158498764,0.0218430757522583,-0.037852395325899124,-0.3875194787979126,-0.3862532675266266,-0.20471422374248505,0.10404524207115173,0.04435110092163086,0.19210034608840942,0.12364770472049713,0.17325642704963684,0.09912291169166565,-0.20981542766094208,-0.2538299560546875,-0.15476448833942413,0.27958250045776367,0.24761389195919037,-0.025552796199917793,0.15557581186294556,0.13984689116477966,0.17191515862941742,-0.3506312966346741,-0.14353927969932556,-0.08681722730398178,0.006743816193193197,-0.03301827237010002,-0.02905946411192417,0.09427586197853088,0.0951310396194458,0.06871863454580307,-0.10379762947559357,0.059964921325445175,0.07241295278072357,0.15491409599781036,0.09858222305774689,0.19929350912570953,-0.25298523902893066,-0.2752145528793335,-0.16559121012687683,-0.22886034846305847,-0.28752508759498596,-0.3724304437637329,0.1830051690340042,0.014194353483617306,0.1296900063753128,0.022343875840306282,-0.13503919541835785,-0.1678330898284912,-0.1916055679321289,-0.21470467746257782,-0.026084845885634422,-0.03528662398457527,-0.05422196164727211,0.03464513272047043,-0.19262418150901794,-0.03379610925912857,-0.012916060164570808,0.028693726286292076,0.11198257654905319,-0.011105976998806,-0.12549607455730438,0.17612814903259277,0.15743447840213776,-0.11285409331321716,0.007728776894509792,0.002275037346407771,-0.08275463432073593,0.09547030925750732,0.046958621591329575,-0.16918249428272247,-0.056523118168115616,0.12459924817085266,-0.17842626571655273,-0.14965729415416718,0.030290968716144562,-0.2078278362751007,-0.17736919224262238,-0.16325709223747253,0.315021276473999,0.05540742352604866,0.31758466362953186,-0.055146995931863785,0.05225434899330139,-0.026750603690743446,-0.013485134579241276,0.05022639036178589,-0.014824753627181053,0.1394355297088623,0.08312210440635681,0.024840708822011948,0.06096992269158363,0.17101208865642548,0.07062879949808121,-0.11571802943944931,-0.22654728591442108,-0.006273046135902405,-0.023917824029922485,0.029811283573508263,-0.1334933042526245,-0.1384563148021698,-0.07774344831705093,-0.3545939028263092,-0.37849247455596924,-0.20988275110721588,-0.33948755264282227,0.05066448077559471,0.0950811505317688,0.2619172930717468,-0.21593445539474487,0.024081895127892494,0.2294812798500061,-0.32627978920936584,-0.13458724319934845,0.026312800124287605,-0.12016666680574417,0.21340465545654297,0.155152827501297,-0.23221386969089508,0.05125991255044937,0.2089923769235611,-0.002620324492454529,-0.12164034694433212,0.12002987414598465,0.21693558990955353,0.2359798550605774,0.34379681944847107,0.042402107268571854,0.18223178386688232,0.17881686985492706,-0.2415558397769928,0.0374387726187706,0.03440740704536438,-0.20804272592067719,-0.11649812012910843,-0.040487222373485565,0.14405493438243866,0.14661185443401337,0.21445563435554504,-0.31510910391807556,-0.06775461137294769,-0.12821070849895477,-0.0997765064239502,-0.27767860889434814,-0.1543392539024353,-0.10816755890846252,-0.17079703509807587,-0.02800920605659485,0.1823078691959381,0.11025198549032211,-0.013660573400557041,-0.20393361151218414,-0.10450965166091919,0.229458287358284,-0.23182596266269684,-0.0904388576745987,0.0536656379699707,-0.325036883354187,0.030867597088217735,0.023995479568839073,-0.1011912003159523,0.1457381695508957,0.24504490196704865,-0.24877773225307465,-0.13871373236179352,-0.009133112616837025,-0.01636250875890255,-0.06794888526201248,-0.2539287507534027,0.0036663126666098833,0.013204330578446388,0.25085368752479553,-0.10430780798196793,0.04697732999920845,0.15388835966587067,0.02498353086411953,0.10522294789552689,0.14094394445419312,-0.09426265954971313,-0.21136273443698883,-0.06967630237340927,-0.002915845951065421,-0.1136440709233284,0.08183826506137848,0.056184399873018265,0.03464440628886223,-0.07603317499160767,0.11102813482284546,0.08728566020727158,0.06457559764385223,0.12518927454948425,0.1604021191596985,0.18355973064899445,0.09078330546617508,0.13919654488563538,0.06295451521873474,-0.005382725037634373,0.061805400997400284,0.012663119472563267,-0.08568435162305832,0.014664766378700733,0.028562037274241447,0.08183881640434265,-0.01584894023835659,0.0334731787443161,0.020527314394712448,0.011939754709601402,0.07257801294326782,0.06234743818640709,-0.14443491399288177,-0.1912183165550232,0.058338712900877,-0.08155893534421921,-0.05145147070288658,0.11575845628976822,-0.1670272946357727,0.10047555714845657,-0.01054813340306282,0.03839285299181938,0.1304861605167389,0.06551344692707062,0.05347609892487526,0.035135526210069656,0.04746566712856293,0.0908929705619812,-0.12603329122066498,-0.02175418846309185,-0.06459002196788788,0.13111841678619385,0.1170966774225235,-0.07526611536741257,0.00468839006498456,0.013434087857604027,0.10340072959661484,0.1557585746049881,-0.07683614641427994,0.09288125485181808,0.16932132840156555,0.0889751985669136,-0.027448013424873352,-0.02617953158915043,-0.0931130051612854,-0.24544338881969452,-0.23717641830444336,-0.300412654876709,-0.11010553687810898,-0.26482826471328735,-0.1994716227054596,-0.2664746344089508,-0.06008670851588249,0.04131917282938957,-0.11536793410778046,-0.08878366649150848,0.05562795698642731,0.07324854284524918,-0.05530104413628578,-0.1232343316078186,-0.08796077221632004,0.03898424655199051,-0.08823495358228683,-0.06297197937965393,0.11880820244550705,0.02009394019842148,-0.12039697915315628,0.14434342086315155,0.016820156946778297,0.09449199587106705,0.08905727416276932,-0.049106940627098083,-0.003361665178090334,0.10569914430379868,-0.0332229919731617,0.159645214676857,0.216897651553154,-0.013629541732370853,0.0753982663154602,-0.09102584421634674,-0.33504968881607056,-0.16844195127487183,-0.23465053737163544,-0.1416042447090149,-0.14313222467899323,-0.17681589722633362,-0.17518028616905212,-0.13753807544708252,-0.15857785940170288,0.07755106687545776,0.0185487512499094,0.040093474090099335,-0.061475224792957306,-0.08718246221542358,0.08235704153776169,-0.00958141777664423,0.011591978371143341,-0.17029450833797455,0.11792723089456558,-0.041991401463747025,0.06065775454044342,0.05667140707373619,0.07076174765825272,-0.10408823192119598,-0.07069247961044312,0.04707227274775505,0.022148732095956802,-0.16767147183418274,-0.03682582452893257,0.17344991862773895,-0.13962557911872864,-0.1625298410654068,-0.08432456105947495,-0.26734015345573425,-0.03875148668885231,0.045633796602487564,-0.12202786654233932,0.12769794464111328,-0.020000096410512924,-0.15056245028972626,-0.06666264683008194,0.123493991792202,0.021980270743370056,-0.17548124492168427,-0.02203221619129181,0.08684507757425308,0.0018370094476267695,-0.05596230924129486,0.021686898544430733,0.022188540548086166,-0.17600852251052856,-0.05769145116209984,0.03672076761722565,0.07510044425725937,-0.023172695189714432,0.06639211624860764,0.09715542942285538,-0.14067134261131287,-0.05415123328566551,0.09252811968326569,-0.08437005430459976,-0.0598597452044487,0.07995743304491043,-0.018866874277591705,-0.1165846511721611,0.061235927045345306,0.04555471986532211,0.0737745612859726,-0.1455235481262207,0.056635044515132904,-0.12244804203510284,0.02711164578795433,0.010472774505615234,0.10249962657690048,0.05643416941165924,0.040243517607450485,0.108685702085495,-0.011174483224749565,-0.24503827095031738,-0.1605696976184845,-0.0859435573220253,-0.18840831518173218,0.013561369851231575,-0.11895009875297546,0.12365563213825226,0.03512592986226082,0.10836761444807053,-0.180665984749794,0.07342030107975006,0.06151651591062546,-0.11918608844280243,-0.05056479573249817,-0.019263964146375656,-0.09865076839923859,-0.023326924070715904,-0.03365866467356682,-0.19671033322811127,-0.1625175178050995,0.04086259752511978,-0.12268853187561035,-0.027938295155763626,-0.19022689759731293,0.007693116087466478,-0.02706369198858738,0.09505152702331543,-0.11305470764636993,-0.23334252834320068,-0.0732480138540268,-0.03873873129487038,-0.10979494452476501,-0.04787033796310425,0.010399485006928444,-0.08942926675081253,0.12019084393978119,-0.22089040279388428,-0.0722019374370575,0.03659335896372795,-0.08387050777673721,0.011100461706519127,-0.0217256098985672,-0.13723035156726837,0.05278683081269264,-0.18901945650577545,-0.02836713381111622,0.08087457716464996,-0.049331068992614746,0.057579003274440765,0.09564836323261261,-0.13799917697906494,0.025974906980991364,0.018706442788243294,0.05562989041209221,0.10532025992870331,-0.0024198347236961126,0.12251570075750351,0.03032025322318077,-0.12684309482574463,0.11501366645097733,0.057658370584249496,-0.004136903211474419,-0.11836141347885132,0.19981947541236877,0.09001383185386658,0.006427578162401915,-0.3937250077724457,-0.38571321964263916,-0.13447873294353485,-0.07516203075647354,-0.3101479113101959,-0.10106134414672852,-0.2703433036804199,-0.07765297591686249,-0.12310624122619629,0.1454082727432251,0.10643505305051804,0.12398692965507507,-0.09536050260066986,0.13803523778915405,-0.06292721629142761,-0.036422863602638245,-0.09481722861528397,-0.07159385830163956,-0.18810993432998657,-0.09432315826416016,-0.1058335080742836,-0.01320237573236227,0.04430101439356804,-0.11065977811813354,-0.21379250288009644,-0.2544821500778198,0.06368475407361984,0.10126534104347229,-0.046224892139434814,0.23684097826480865,0.08402935415506363,0.01380330789834261,-0.018223710358142853,-0.054708585143089294,-0.16589654982089996,-0.0508778877556324,0.06723117083311081,0.04861248657107353,0.05922580137848854,-0.16701418161392212,-0.12896718084812164,0.11124375462532043,0.1233585774898529,0.07395020127296448,-0.09893926978111267,0.143101766705513,0.016455307602882385,0.08248800039291382,0.07409516721963882,0.09054893255233765,0.0968894436955452,-0.0071053034625947475,0.000310851086396724,-0.08354008197784424,-0.15836714208126068,0.12512028217315674,-0.09900722652673721,-0.14139899611473083,0.08908058702945709,-0.18735402822494507,-0.12843094766139984,-0.07255245745182037,0.05618690326809883,-0.05468400567770004,0.02112027071416378,0.04096255823969841,-0.21146239340305328,-0.16416102647781372,-0.15485776960849762,0.010476035997271538,0.09202125668525696,-0.08516085147857666,-0.19879496097564697,0.0053654201328754425,-0.022406378760933876,-0.24064312875270844,-0.18347135186195374,-0.055107273161411285,-0.23296412825584412,-0.19346092641353607,-0.1561039388179779,0.12715549767017365,-0.052011534571647644,-0.18185217678546906,-0.09998960047960281,0.05370292067527771,0.11115042865276337,-0.08096279203891754,0.07709234952926636,0.1434803456068039,-0.10850618034601212,-0.023585645481944084,-0.01649787276983261,0.16865800321102142,0.0720040500164032,0.12213600426912308,0.05615708976984024,0.0024579630699008703,-0.01087040826678276,-0.16284093260765076,-0.03628595545887947,0.0522121861577034,-0.1698695421218872,-0.05789288505911827,-0.06966406106948853,-0.1944892704486847,-0.09982265532016754,-0.007546251639723778,0.08394671231508255,-0.08407767117023468,-0.4036356210708618,0.4186917245388031,0.07218257337808609,-0.20803876221179962,0.31739336252212524,0.19492213428020477,0.27787259221076965,0.10354764014482498,0.09512373805046082,-0.23501257598400116,0.18963149189949036,-0.043689481914043427,0.017315587028861046,-0.008638542145490646,0.04828477278351784,0.05868208780884743,0.18270470201969147,-0.05324553698301315,-0.05739820376038551,0.0474877655506134,0.0478941909968853,-0.13796569406986237,-0.08703027665615082,0.17672036588191986,0.10484172403812408,0.159958153963089,-0.13772064447402954,-0.21937373280525208,0.3283062279224396,0.023751383647322655,-0.04940022900700569,0.21572227776050568,0.19438205659389496,0.011251766234636307,-0.15649518370628357,-0.20365114510059357,-0.07730098068714142,-0.09525995701551437,-0.25226855278015137,-0.2229691743850708,0.0809076577425003,0.02602512575685978,-0.25629371404647827,0.07790311425924301,-0.006015061400830746,-0.07029784470796585,-0.08790604770183563,0.05889196693897247,0.10032230615615845,-0.13780340552330017,0.03306170552968979,0.1319962441921234,-0.007517716381698847,-0.2450864166021347,-0.29192447662353516,0.04014414921402931,-0.015220826491713524,-0.20124077796936035,-0.1738426834344864,-0.01733916625380516,-0.1080799251794815,0.30314797163009644,0.17145714163780212,-0.10895036906003952,0.3601129949092865,0.1813945472240448,0.10708513110876083,0.20926696062088013,0.3350837230682373,0.23694583773612976,-0.13641124963760376,-0.03956550359725952,-0.26648905873298645,0.1495581716299057,-0.11530829221010208,-0.10623039305210114,0.054547276347875595,0.14725877344608307,-0.0370873287320137,-0.008795550093054771,0.019260047003626823,0.11662223190069199,-0.003962834365665913,0.06985509395599365,0.21006466448307037,-0.15703116357326508,0.06275444477796555,0.17343081533908844,-0.11954401433467865,0.06147225946187973,-0.010325152426958084,-0.10698970407247543,0.0815935954451561,-0.02855827473104,0.010634522885084152,0.127017542719841,0.031189821660518646,-0.029785403981804848,-0.09576531499624252,-0.029198650270700455,0.047883640974760056,-0.1150490790605545,0.06416677683591843,0.11782076954841614,-0.045344941318035126,0.028216132894158363,-0.133452370762825,-0.1535741686820984,-0.12998469173908234,-0.13573545217514038,-0.21800526976585388,-0.2512204349040985,-0.11340273916721344,-0.03941134363412857,-0.20293956995010376,0.08540590852499008,0.16316640377044678,0.012663707137107849,0.038409844040870667,-0.10882037878036499,-0.06921708583831787,0.04774835333228111,0.05400897562503815,-0.15297898650169373,0.2625783383846283,0.012641383334994316,0.0106126070022583,0.1465619057416916,-0.1256803274154663,-0.21648694574832916,-0.16879048943519592,-0.09172526001930237,-0.23664145171642303,0.019845636561512947,0.034123919904232025,-0.039354290813207626,-0.08264902979135513,0.0007056709728203714,-0.06055988743901253,-0.30157768726348877,-0.3018473982810974,-0.26760154962539673,-0.15810513496398926,-0.35028496384620667,-0.21475878357887268,-0.3456217050552368,-0.39056268334388733,-0.38551655411720276,-0.08412903547286987,-0.29840898513793945,-0.4043579697608948,-0.018604718148708344,0.21824461221694946,0.22672472894191742,0.19665470719337463,0.15094411373138428,0.1869588941335678,-0.09601642936468124,-0.06908247619867325,0.040396884083747864,0.042542919516563416,0.15471908450126648,-0.052944429218769073,-0.0808902308344841,0.03201034665107727,0.13541410863399506,0.12843364477157593,0.09624693542718887,-0.050665102899074554,0.3520374596118927,0.40404650568962097,0.25355780124664307,0.1568257063627243,0.19660374522209167,0.10472887009382248,-0.034777309745550156,0.03914109617471695,-0.08267109096050262,-0.33827468752861023,-0.12288673222064972,-0.38158878684043884,-0.11082305759191513,-0.016467534005641937,0.015966977924108505,-0.2257167249917984,-0.2915050685405731,-0.2664492130279541,-0.08389756083488464,-0.0343766063451767,-0.045024700462818146,-0.02327914349734783,0.06901835650205612,-0.002623572014272213,0.13031162321567535,0.13697941601276398,0.027283422648906708,0.00441388413310051,-0.11274158954620361,-0.11032228916883469,0.040612660348415375,0.009295382536947727,-0.1063254177570343,-0.11467892676591873,-0.14502453804016113,-0.15033262968063354,0.13980603218078613,0.1394243985414505,0.24436944723129272,0.3650636672973633,0.2009924352169037,0.3403094410896301,0.12574659287929535,0.2808660566806793,0.2615128755569458,0.05231548100709915,0.0049998946487903595,0.05018121376633644,-0.10370777547359467,-0.012761563062667847,0.012751653790473938,0.10525400191545486,0.05911703780293465,0.02071985974907875,0.10334279388189316,0.0300716832280159,0.09631109982728958,0.11571602523326874,0.15779250860214233,0.007195730227977037,0.15732377767562866,0.15280017256736755,0.07778622210025787,0.14663392305374146,-0.05558105558156967,0.012729444541037083,-0.07933597266674042,-0.11240918189287186,0.17269940674304962,0.051870446652173996,0.1137881800532341,0.06579477339982986,-0.05174229294061661,0.09996171295642853,-0.05504079908132553,0.14234286546707153,-0.11796215921640396,0.1682714819908142,-0.12694008648395538,-0.15997710824012756,0.047191277146339417,0.007589524146169424,0.005053471773862839,-0.04106217250227928,0.0637608990073204,0.02726069837808609,0.10866887122392654,-0.1025240421295166,-0.08523540198802948,0.08735058456659317,0.0930701270699501,-0.0030636501032859087,0.09380367398262024,0.18483835458755493,0.10160426050424576,-0.07735096663236618,0.11296354979276657,-0.0939503088593483,0.04857402294874191,-0.0059170955792069435,-0.09877839684486389,0.13401374220848083,0.15528427064418793,-0.10104390233755112,0.11450372636318207,0.015207015909254551,-0.22113604843616486,-0.11637987941503525,0.026845866814255714,0.16944877803325653,0.09877840429544449,0.1411924511194229,0.15154843032360077,0.015682203695178032,-0.033488448709249496,-0.02822098694741726,0.023794302716851234,-0.3890886604785919,-0.29116290807724,-0.3225310444831848,-0.4823412597179413,-0.24124950170516968,-0.531937837600708,-0.4679346978664398,-0.1309051662683487,-0.22577780485153198,0.2044699341058731,0.2111627608537674,0.09499289095401764,0.1193694919347763,0.06098882108926773,0.09870802611112595,0.15389302372932434,-0.050695933401584625,0.018738511949777603,-0.023710932582616806,0.03620421141386032,-0.06565364450216293,-0.001181271392852068,0.06787413358688354,0.12309639900922775,0.10459793359041214,-0.01252750400453806,0.013598807156085968,0.05847945436835289,0.357876718044281,0.05993850901722908,-0.14180214703083038,0.14009791612625122,-0.21341799199581146,0.03061867132782936,-0.03436300531029701,-0.18146675825119019,-0.11699356138706207,-0.23344729840755463,-0.04391106590628624,-0.190305694937706,-0.10551029443740845,-0.21262669563293457,-0.19856366515159607,-0.31492942571640015,-0.3288443088531494,-0.14915239810943604,-0.12629660964012146,-0.06561309099197388,-0.10167080909013748,-0.05395353212952614,-7.142603863030672e-06,-0.07447659969329834,0.040648724883794785,-0.06947649270296097,0.01166932936757803,-0.007863184437155724,-0.08953659981489182,-0.04192524030804634,-0.1732131838798523,0.03487510606646538,-0.20587942004203796,-0.05449255555868149,-0.20338450372219086,-0.19589866697788239,-0.0428597517311573,-0.019342411309480667,-0.13742247223854065,0.006481699645519257,-0.0029766024090349674,-0.15693044662475586,-0.1330801546573639,0.07443151623010635,0.1123143881559372,0.12839317321777344,0.16950474679470062,-0.005053394008427858,0.09009008854627609,0.018928319215774536,0.18858949840068817,-0.05073025822639465,0.20122632384300232,-0.14479313790798187,-0.16208860278129578,-0.16245242953300476,0.026562126353383064,-0.13446444272994995,-0.038684193044900894,0.15202340483665466,-0.14029882848262787,0.02665005996823311,-0.24486424028873444,0.05328735336661339,-0.04107842221856117,-0.2646922469139099,-0.155255988240242,-0.2246481478214264,-0.07872536033391953,0.013501306995749474,-0.23051917552947998,0.12734568119049072,0.06803929805755615,0.1466127634048462,0.16216425597667694,0.030507825314998627,0.08801215887069702,-0.15267270803451538,0.008290493860840797,-0.011091882362961769,0.004875144921243191,0.030666761100292206,0.1441093236207962,0.1301925927400589,0.11371578276157379,0.02530583366751671,0.10133096575737,-0.09212144464254379,0.10423721373081207,-0.15576396882534027,-0.19003960490226746,-0.2945729196071625,-0.15071387588977814,-0.18282975256443024,-0.18013513088226318,0.02321319840848446,-0.07311834394931793,-0.14075058698654175,0.01647859811782837,-0.02472086437046528,0.06279099732637405,-0.10859165340662003,0.0867486372590065,-0.05959337577223778,0.0059809936210513115,-0.16028331220149994,-0.031030219048261642,0.24981319904327393,0.22938500344753265,0.20927514135837555,0.0824778750538826,0.23496204614639282,0.1917172372341156,0.08953335881233215,0.04698473587632179,0.23387722671031952,-0.2254476249217987,-0.07527396827936172,-0.20705409348011017,0.046421464532613754,-0.1958640068769455,-0.06516556441783905,-0.011273025535047054,-0.14993475377559662,-0.17814776301383972,-0.2008827030658722,-0.12349481880664825,-0.23437078297138214,-0.2021901160478592,-0.16160565614700317,-0.047577403485774994,-0.11978453397750854,-0.21062389016151428,-0.18498128652572632,0.11103843152523041,0.1160772442817688,-0.08429839462041855,-0.033685408532619476,0.11377763748168945,-0.02107335813343525,0.11321388930082321,0.07129210978746414,-0.12944455444812775,-0.17657825350761414,-0.07798222452402115,-0.2878013253211975,-0.125075563788414,-0.17554068565368652,-0.16383616626262665,-0.24771852791309357,-0.0743035301566124,0.05327649042010307,0.16582132875919342,-0.04751841723918915,0.01058095321059227,0.15528210997581482,0.2110716998577118,0.12338808178901672,0.20289425551891327,0.2361287623643875,0.20992636680603027,-0.06707992404699326,-0.10933089256286621,-0.08467927575111389,-0.054086968302726746,-0.11254843324422836,-0.13243792951107025,-0.14827853441238403,0.03823957219719887,-0.1473664492368698,-0.01704607903957367,0.026562342420220375,0.05416503921151161,-0.009038587100803852,0.04929307475686073,0.1452125608921051,-0.038528021425008774,0.1121753454208374,-0.00878607016056776,-0.06448490172624588,-0.027586787939071655,-0.07769159227609634,0.019713304936885834,-0.07340836524963379,0.03381355479359627,-0.09483662992715836,-0.12540468573570251,0.04163416102528572,0.026592852547764778,-0.028846120461821556,-0.07632240653038025,-0.06352218240499496,-0.01511607225984335,0.08371937274932861,0.0031443373300135136,0.13024161756038666,0.13997723162174225,-0.0566861554980278,-0.01163684856146574,-0.07629874348640442,-0.12792430818080902,0.038220927119255066,-0.13230419158935547,-0.19927367568016052,0.0020973999053239822,-0.14844448864459991,0.25609126687049866,0.04894129931926727,0.18178309500217438,0.1410924792289734,0.08825276046991348,0.0314384400844574,0.3490699529647827,0.12496324628591537,0.11579185724258423,0.06985793262720108,-0.12437226623296738,-0.15997423231601715,0.06180891767144203,-0.0896616280078888,-0.07843165844678879,-0.05883136764168739,0.07581927627325058,0.061377089470624924,-0.13077157735824585,-0.14471982419490814,-0.020076075568795204,-0.01985519751906395,-0.13466106355190277,0.09034618735313416,-0.12188655138015747,-0.006592445075511932,-0.03281692415475845,-0.02262823097407818,-0.07062901556491852,0.10263548791408539,-0.062122780829668045,0.15705910325050354,-0.1057453453540802,-0.06842172890901566,-0.08255939930677414,0.17911598086357117,0.043637488037347794,-0.21901214122772217,0.020757213234901428,0.06500296294689178,-0.1826714277267456,0.058712396770715714,0.021364850923419,-0.12425392866134644,-0.014710973016917706,0.07289085537195206,0.2178991585969925,0.034368935972452164,0.1328333169221878,-0.07436980307102203,0.11958236992359161,0.09311200678348541,0.10527529567480087,0.06407566368579865,-0.5460094213485718,-0.2517636716365814,-0.5569965243339539,-0.41702359914779663,-0.3115893602371216,-0.4566957652568817,-0.35666900873184204,-0.3580264151096344,-0.37885308265686035,0.08016767352819443,0.06293885409832001,-0.022210484370589256,-0.08514852821826935,-0.063168466091156,0.1735735982656479,0.08141247183084488,0.08603818714618683,0.0404139868915081,0.08099500089883804,0.07913630455732346,0.12150970101356506,-0.09140640497207642,-0.06584163010120392,-0.013121644034981728,-0.11730360239744186,0.10972560942173004,-0.0908762663602829,0.062323495745658875,0.12977027893066406,0.07150156795978546,-0.014064357616007328,0.05947903171181679,0.02443070150911808,0.12392319738864899,-0.06403441727161407,-0.07992206513881683,-0.4365898370742798,-0.3316574990749359,-0.17612317204475403,-0.3248644471168518,-0.38385504484176636,-0.3410000205039978,-0.2799626588821411,-0.1542353630065918,-0.19332104921340942,0.11724177747964859,0.13676930963993073,-0.008647237904369831,-0.0016819126904010773,0.11973143368959427,-0.01237514242529869,0.06813523173332214,0.0891246572136879,0.0971992164850235,0.10121341794729233,0.06495415419340134,0.016077395528554916,0.02373231016099453,0.1748518943786621,0.27825310826301575,0.11987839639186859,0.07514676451683044,0.04771862179040909,0.05026666820049286,-0.059120047837495804,0.03712235391139984,-0.07104019820690155,0.10042331367731094,-0.030371708795428276,-0.1294163465499878,0.12309171259403229,0.04482823237776756,-0.07001062482595444,0.05846516042947769,0.044238198548555374,-0.1710330694913864,0.05870869383215904,-0.1231599971652031,-0.14541493356227875,-0.12387311458587646,0.13089407980442047,-0.07766307890415192,-0.0010666009038686752,0.16046671569347382,-0.10940619558095932,-0.08469053357839584,-0.06616253405809402,0.10159243643283844,-0.0411301888525486,-0.06768732517957687,0.13084883987903595,0.03150556609034538,0.07264967262744904,0.19821929931640625,0.026365898549556732,0.1666347235441208,-0.039480406790971756,0.05033939704298973,0.050906650722026825,0.1323642134666443,0.044617731124162674,0.012597844004631042,-0.02914266288280487,0.10796899348497391,-0.057498108595609665,0.11706442385911942,0.14161072671413422,0.134842649102211,0.09847792237997055,0.104282908141613,0.19894255697727203,-0.007216210011392832,0.06589627265930176,0.03324003517627716,-0.18140603601932526,-0.0891677737236023,-0.02648915722966194,0.052814699709415436,0.16038092970848083,0.09043923020362854,0.127648264169693,0.01776047982275486,0.047850824892520905,-0.024262219667434692,-0.12826187908649445,0.11829207837581635,-0.015646755695343018,-0.0033454466611146927,0.15318839251995087,-0.14900949597358704,-0.005364338867366314,-0.07109854370355606,-0.17108583450317383,-0.22400681674480438,-0.1268531233072281,0.18907581269741058,0.13214412331581116,0.19497135281562805,0.03918934240937233,0.10248108953237534,-0.03610943257808685,-0.037711892277002335,0.09585203230381012,-0.030499881133437157,0.21105892956256866,0.07627124339342117,0.13170818984508514,0.19059297442436218,0.136647567152977,0.15128840506076813,-0.12204978615045547,-0.005121143069118261,0.16340750455856323,0.22878077626228333,0.029710553586483,-0.06082601100206375,-0.0853525847196579,-0.1942148208618164,-0.14852868020534515,-0.024009238928556442,-0.12357038259506226,-0.17954333126544952,-0.07937594503164291,-0.20877297222614288,-0.22428999841213226,-0.12921008467674255,-0.23499345779418945,-0.19297881424427032,-0.1770462989807129,-0.20751194655895233,-0.2713322043418884,-0.09839394688606262,-0.18811705708503723,-0.15785740315914154,-0.09136631339788437,0.021678775548934937,0.05635339766740799,-0.10374205559492111,-0.1166352778673172,-0.13694371283054352,0.3908606767654419,0.19427724182605743,0.43499213457107544,0.2733432352542877,0.29120534658432007,0.08358566462993622,0.08657654374837875,0.06235291063785553,0.18397288024425507,0.021008063107728958,0.022414308041334152,-0.09854254126548767,0.11428125202655792,0.18038707971572876,0.1281225085258484,0.02982017956674099,0.19769209623336792,0.16048581898212433,-0.058959364891052246,0.03666548430919647,0.07082787901163101,-0.11335521191358566,-0.01256298553198576,0.008043492212891579,0.01674431562423706,-0.043147411197423935,0.0605422668159008,0.3364744484424591,0.23787499964237213,-0.015014455653727055,0.03540797159075737,0.04611581191420555,-0.06360141932964325,-0.08107241243124008,-0.12070305645465851,-0.03545764833688736,-0.04749877750873566,0.03884157910943031,-0.04054948315024376,-0.17145343124866486,-0.20314672589302063,-0.04951893538236618,-0.020458035171031952,-0.20797188580036163,-0.17812570929527283,-0.049085743725299835,0.04462333396077156,-0.010370846837759018,0.030444692820310593,-0.13799349963665009,-0.16941331326961517,0.1045224592089653,0.012311774305999279,-0.0538480244576931,0.015815842896699905,0.20290507376194,0.22563356161117554,0.23052027821540833,0.24996604025363922,0.17365247011184692,0.21532873809337616,0.0842394158244133,0.09409087896347046,-0.036438390612602234,-0.08264991641044617,0.06172407791018486,0.14255067706108093,0.001179973129183054,-0.09000537544488907,0.11909682303667068,-0.07441209256649017,-0.05201423168182373,-0.08196643739938736,0.03790883347392082,-0.030916843563318253,0.07632926106452942,-0.12919387221336365,-0.03579816222190857,-0.12399933487176895,0.0831444039940834,-0.1589873731136322,0.04087582975625992,-0.06382311880588531,-0.06878851354122162,-0.13750706613063812,-0.08535686880350113,-0.03932274132966995,0.025750497356057167,0.10846023261547089,0.07045678794384003,0.046549491584300995,-0.13688573241233826,0.029467154294252396,-0.11105310171842575,-0.11758597195148468,0.097748301923275,0.008212165907025337,0.09865692257881165,0.09938789159059525,-0.21420541405677795,-0.08868775516748428,-0.030285663902759552,-0.14085933566093445,-0.170432448387146,-0.06949439644813538,-0.01991857960820198,0.033801183104515076,0.08533744513988495,0.36993786692619324,0.27035561203956604,0.054082997143268585,0.1202574372291565,0.36087945103645325,0.27356037497520447,-0.12093722820281982,0.3402949571609497,0.27661076188087463,-0.14536410570144653,0.08656281977891922,0.0688665360212326,-0.22400711476802826,-0.07563821226358414,0.027025632560253143,-0.3306567668914795,0.009910456836223602,0.21186695992946625,-0.04613529518246651,-0.01621386595070362,-0.08182619512081146,-0.016796741634607315,-0.12856005132198334,0.12781421840190887,-0.24566824734210968,-0.16355541348457336,0.13142527639865875,0.11461851000785828,0.23445022106170654,0.30738553404808044,0.196679949760437,0.3141850531101227,0.3073682487010956,-0.0008831671439111233,0.2658005356788635,0.2866974472999573,0.2456793189048767,0.2169206589460373,0.3442184627056122,0.24096901714801788,0.44092875719070435,0.13190282881259918,0.27746835350990295,0.3042497932910919,0.0821630135178566,-0.07884880155324936,-0.20446503162384033,0.08525794744491577,-0.17880432307720184,-0.04514854773879051,-0.018376749008893967,-0.027929268777370453,0.1524495929479599,0.03369884938001633,-0.07894231379032135,0.09444200992584229,0.21020080149173737,-0.06761779636144638,0.08735384792089462,0.22261299192905426,-0.04293975606560707,0.17194871604442596,0.14987300336360931,0.03472635895013809,0.25019797682762146,0.08287418633699417,-0.18611262738704681,0.05910973250865936,0.2956227660179138,-0.12821421027183533,0.06267757713794708,0.14104832708835602,0.03374319151043892,0.040403813123703,-0.011840052902698517,-0.14751240611076355,-0.0019064793596044183,0.03141729161143303,-0.1533224731683731,0.08111244440078735,-0.08376315236091614,-0.1453886330127716,0.08376695215702057,0.006589188240468502,-0.09613955765962601,-0.01710849441587925,0.06527900695800781,-0.225750133395195,-0.19637897610664368,-0.06711207330226898,-0.06205803155899048,-0.11671808362007141,-0.15488702058792114,0.06085292622447014,0.11302929371595383,0.16429735720157623,0.19120144844055176,0.16990123689174652,0.06539509445428848,0.04552319645881653,-0.006006740499287844,0.00174606847576797,-0.07153373211622238,0.1285032033920288,0.08930497616529465,-0.08220423758029938,0.0640794187784195,0.0952356681227684,0.11680440604686737,-0.015399441123008728,0.00796156283468008,-0.05973895639181137,0.0157962366938591,0.06181548535823822,0.0007988197030499578,0.06150248274207115,0.06528578698635101,-0.07911811769008636,-0.15822407603263855,0.08911361545324326,-0.11264301836490631,-0.03506777808070183,0.02045563794672489,0.0183625016361475,-0.1169416531920433,-0.12031479179859161,-0.010730021633207798,-0.023776622489094734,0.11747702956199646,-0.10777481645345688,-0.12817350029945374,-0.18844477832317352,0.005334764719009399,-0.14420878887176514,-0.2306133359670639,0.21466940641403198,0.14870662987232208,-0.030885811895132065,0.04019704461097717,0.15564598143100739,0.052530501037836075,-0.07955051958560944,-0.09051822870969772,-0.12310117483139038,0.07287148386240005,0.23978741466999054,0.16622459888458252,-0.16620300710201263,-0.23834384977817535,-0.2752922773361206,0.1706032007932663,0.10532961785793304,0.06869219988584518,-0.07834267616271973,0.17246310412883759,0.03746717795729637,-0.14301417768001556,0.00868820771574974,0.01933732070028782,0.09998249262571335,0.06722970306873322,-0.026047000661492348,-0.016939830034971237,0.15460102260112762,-0.07781857997179031,0.01607036590576172,-0.08605398237705231,-0.15751336514949799,-0.16860036551952362,0.07704336196184158,-0.06287623196840286,0.1816321462392807,0.24709656834602356,0.2747279703617096,0.20242322981357574,0.3585836887359619,0.1807866394519806,0.3098334074020386,0.005107759032398462,-0.19306108355522156,0.12679791450500488,0.24619263410568237,0.5257849097251892,0.44002634286880493,0.63164222240448,0.3908229172229767,0.48185330629348755,0.39142030477523804,0.07193616032600403,-0.03398536145687103,-0.053864944726228714,-0.07510387897491455,-0.1618412286043167,0.1251351535320282,0.10724588483572006,0.053244974464178085,0.0392097570002079,0.05608260631561279,0.09884169697761536,0.14945198595523834,0.05761744827032089,0.25826334953308105,-0.017084339633584023,-0.14481373131275177,-0.05885700881481171,-0.18385285139083862,-0.2105257660150528,-0.09948000311851501,-0.06863296777009964,0.07168307155370712,-0.09552688896656036,-0.059093885123729706,0.03779786080121994,0.08126597851514816,0.16930121183395386,0.11816798895597458,0.06599259376525879,0.09189461916685104,-0.017838211730122566,-0.004024697933346033,-0.047239549458026886,-0.17940977215766907,-0.05616910383105278,-0.14599792659282684,-0.07565654814243317,-0.08404891937971115,-0.08432817459106445,0.1197301521897316,0.017535144463181496,-0.01211498025804758,0.1537378579378128,0.10909420996904373,-0.08854158967733383,0.057397760450839996,0.056645046919584274,-0.05318504571914673,-0.06817404925823212,-0.03717422112822533,-0.0671006515622139,0.09048385918140411,0.10951273888349533,-0.005014987662434578,0.09091924130916595,0.06552068889141083,0.07789772748947144,0.06395061314105988,-0.11330782622098923,-0.016649508848786354,-0.029156368225812912,-0.050185490399599075,-0.07371535897254944,0.12193026393651962,-0.04180975258350372,-0.12533240020275116,0.04834712669253349,-0.15384352207183838,0.020734883844852448,0.0009034642134793103,0.028388896957039833,0.003083382500335574,-0.1034708321094513,0.1887296587228775,0.08241146802902222,0.07839994877576828,0.1747114062309265,0.06905186921358109,0.11900662630796432,0.10706964135169983,0.11770284920930862,0.17144925892353058,-0.03650316968560219,-0.12522970139980316,0.08365879207849503,0.08779492229223251,-0.11081212013959885,-0.19322757422924042,-0.09223894029855728,0.10437988489866257,-0.06747294962406158,0.1803099513053894,0.08594560623168945,0.17368857562541962,-0.015522555448114872,0.12723778188228607,0.1505395770072937,-0.10591936111450195,0.0690632313489914,-0.07046282291412354,0.0327627994120121,-0.1168096587061882,0.12259919196367264,0.10966769605875015,-0.07966958731412888,0.06636535376310349,0.1561506986618042,0.042074643075466156,0.015198166482150555,-0.0015864897286519408,-0.0009643140947446227,0.10953586548566818,0.21769319474697113,0.01862080581486225,-0.10604427754878998,-0.026761339977383614,0.08405370265245438,-0.09869988262653351,0.008895883336663246,-0.1665036976337433,-0.021716788411140442,0.07643156498670578,0.0025250099133700132,-0.12666656076908112,0.015400290489196777,-0.19675517082214355,-0.17731119692325592,-0.10670696198940277,0.016434412449598312,-0.015869611874222755,-0.09614767879247665,0.007481592241674662,-0.03849867358803749,0.09220588207244873,0.0047503309324383736,-0.2546389400959015,0.23053178191184998,0.19993366301059723,0.24185730516910553,0.21390733122825623,0.11743583530187607,0.33524152636528015,0.019120531156659126,0.06861349940299988,0.34895405173301697,-0.07172039151191711,-0.12165358662605286,-0.11761653423309326,0.15810848772525787,-0.07926701009273529,-0.10262779146432877,0.10702589154243469,0.185140922665596,0.04196997359395027,-0.0540584996342659,0.07186688482761383,-0.03936881572008133,0.18969127535820007,-0.05711022764444351,-0.16971397399902344,0.12682028114795685,-0.0227566696703434,-0.09595213085412979,0.11429708451032639,0.16177545487880707,-0.049082934856414795,0.14546605944633484,0.11854396015405655,-0.07009245455265045,0.03392462059855461,0.12962552905082703,-0.03312639147043228,-0.13030210137367249,-0.15661019086837769,-0.24685180187225342,0.03796200454235077,-0.2350172996520996,-0.2989441752433777,-0.19604262709617615,-0.23193787038326263,-0.1971370279788971,0.026601087301969528,-0.0031805329490453005,0.001863557379692793,0.02798473834991455,-0.1772558093070984,-0.2315392643213272,0.11356443911790848,-0.1346186250448227,-0.1504736691713333,0.20650285482406616,0.1205923929810524,0.20382104814052582,0.06939122825860977,0.21687506139278412,0.2109268605709076,0.058273132890462875,0.2520456612110138,0.04991919547319412,0.13391956686973572,0.17617718875408173,0.11210262775421143,0.0789337307214737,0.14286039769649506,-0.004698200151324272,0.07775915414094925,0.0457310788333416,0.16809406876564026,-0.022830437868833542,-0.14933200180530548,-0.08016324043273926,0.1645272672176361,0.21744327247142792,-0.003866164945065975,0.014210578054189682,-0.010002468712627888,-0.00753909582272172,-0.16376715898513794,-0.12124524265527725,-0.024778377264738083,-0.1357448250055313,-0.15573544800281525,0.04519115760922432,-0.006420442834496498,0.11281680315732956,0.05641230195760727,0.05920283868908882,0.10713355243206024,0.07052420824766159,0.08715691417455673,0.1332642138004303,0.19355498254299164,-0.06056565046310425,-0.17163962125778198,0.04418393224477768,0.047446317970752716,-0.033747442066669464,-0.2952866554260254,-0.0729118287563324,0.0789899155497551,-0.1318785548210144,-0.07240531593561172,-0.1736525297164917,0.04261602833867073,0.17742547392845154,-0.07321242243051529,0.129930317401886,-0.3179638683795929,-0.14299535751342773,-0.3513934910297394,0.13398827612400055,0.09489747881889343,0.00554418470710516,-0.11670960485935211,-0.12939617037773132,-0.14797210693359375,0.05514975264668465,-0.17340758442878723,-0.16959720849990845,-0.0015231079887598753,0.1427253931760788,0.061127305030822754,0.017201527953147888,0.12032155692577362,0.04431338608264923,-0.054478131234645844,0.12161161005496979,-0.10004176944494247,0.0074757169932127,0.08000669628381729,0.0140674514696002,0.12125983834266663,0.2527638077735901,0.3131530284881592,0.051121603697538376,-0.1595553308725357,-0.008926362730562687,0.12218930572271347,0.039882171899080276,-0.19854426383972168,-0.029298195615410805,-0.07232630997896194,0.01081225834786892,-0.050521038472652435,-0.23555722832679749,0.04396488144993782,-0.06997130066156387,-0.3209189474582672,0.0052729444578289986,-0.1812916398048401,-0.11815184354782104,-0.12373161315917969,0.07396893203258514,-0.060877475887537,-0.10463444143533707,0.06515087187290192,0.09669497609138489,0.12556423246860504,-0.21262486279010773,-0.2841346859931946,-0.24112500250339508,0.1180809736251831,-0.013941970653831959,-0.10529971867799759,-0.11769021302461624,-0.07280407845973969,0.018856601789593697,-0.2054976373910904,-0.20052143931388855,0.10843710601329803,-0.10423305630683899,-0.08284331113100052,-0.2767067551612854,0.0400596521794796,0.1769421398639679,0.002492313040420413,-0.048950500786304474,0.11353091895580292,0.032966844737529755,-0.05681050196290016,0.030849650502204895,-0.11040324717760086,-0.08061489462852478,-0.16587558388710022,-0.09399884939193726,0.051045436412096024,-0.14122165739536285,0.030144400894641876,0.15011760592460632,0.16540215909481049,-0.045796703547239304,0.16172732412815094,0.17951242625713348,0.1802693009376526,-0.21524494886398315,-0.07958371192216873,-0.268087238073349,-0.10635457932949066,-0.11483390629291534,-0.20789456367492676,-0.09448099136352539,-0.10642652213573456,0.0061286636628210545,0.1030002161860466,0.13195329904556274,0.06910569965839386,0.08242598921060562,-0.161113440990448,-0.11104724556207657,-0.03489558398723602,0.02389739453792572,0.041439615190029144],
            4:[0.30562087893486023,0.1824757605791092,0.23052702844142914,0.25635698437690735,0.1880234181880951,0.2352934181690216,0.12032242119312286,0.09932686388492584,0.08759316802024841,0.05353684723377228,0.12833140790462494,0.09999825805425644,0.08244196325540543,0.12964007258415222,0.28778862953186035,0.10735855996608734,0.07602394372224808,0.08313222229480743,0.09847519546747208,0.07957478612661362,0.00907386839389801,-0.04324567690491676,-0.008234202861785889,-0.07407654821872711,0.03572644665837288,0.058253757655620575,-0.028953222557902336,0.009130598977208138,0.19617082178592682,0.03727834299206734,0.03372641280293465,0.013775787316262722,-0.08030989021062851,-0.049521781504154205,-0.10613521188497543,-0.025134176015853882,-0.0805092379450798,-0.10158941894769669,0.003946435172110796,-0.021672287955880165,-0.037345293909311295,0.027959393337368965,0.16777457296848297,0.012223739176988602,0.07994172722101212,0.005648506805300713,-0.003492188174277544,-0.06625870615243912,0.01822863146662712,0.07916853576898575,-0.06812914460897446,-0.04538398236036301,-0.009794059209525585,-0.06717418134212494,-0.030029233545064926,0.07124065607786179,0.06842176616191864,0.06983538717031479,0.08083640784025192,0.08812228590250015,-0.004476809874176979,-0.015038921497762203,-8.148829510901123e-05,0.022398600354790688,0.018406426534056664,-0.033917248249053955,-0.026791410520672798,-0.049734894186258316,-0.029285691678524017,0.04915718734264374,0.006768050603568554,0.05001862347126007,-0.048471469432115555,-0.04364185407757759,-0.01971403695642948,-0.05624274164438248,-0.07067876309156418,-0.056425780057907104,-0.06045585498213768,-0.04759993031620979,-0.0025195388589054346,-0.017841331660747528,-0.01618354395031929,0.04233082756400108,-0.04251040518283844,-0.026896217837929726,-0.06794924288988113,-0.12856154143810272,-0.053187254816293716,-0.06377915292978287,-0.006713720969855785,-0.05639452487230301,-0.15735602378845215,-0.13182811439037323,-0.06732478737831116,0.02029404416680336,0.003993369173258543,0.05650276690721512,0.0035118188243359327,-0.03897218033671379,-0.02863665297627449,-0.11592678725719452,-0.21899881958961487,-0.0274807158857584,-0.10168957710266113,-0.06625165045261383,-0.07277049124240875,-0.19414065778255463,-0.11358924210071564,-0.031627148389816284,-0.09659439325332642,0.022529875859618187,-0.0382307693362236,-0.05535050481557846,-0.013720834627747536,-0.07879485934972763,-0.1243697926402092,-0.15057741105556488,-0.08555097877979279,-0.13962264358997345,-0.14952702820301056,-0.04711136966943741,-0.058601778000593185,0.00046510776155628264,0.017586084082722664,0.011787398718297482,-0.02687435783445835,-0.027050651609897614,-0.026139656081795692,-0.003838024800643325,-0.09610123187303543,-0.13884013891220093,-0.08907981961965561,-0.14187373220920563,-0.06861436367034912,0.006224959623068571,-0.029708417132496834,-0.02307555265724659,0.04201548546552658,0.06975654512643814,-0.05654874071478844,0.021927485242486,0.06614993512630463,0.0008679029997438192,-0.08358453214168549,-0.07870721817016602,-0.04046674817800522,-0.029066888615489006,-0.0523524135351181,0.015001650899648666,0.02629650942981243,0.040356069803237915,-0.03406921774148941,0.04464755579829216,-0.03268362581729889,0.0772247239947319,0.012045660056173801,-0.022953085601329803,-0.08345474302768707,-0.09977567195892334,0.06778773665428162,0.09335203468799591,-0.0011806486872956157,0.029571831226348877,0.013421056792140007,0.02614937350153923,0.01127792987972498,0.05010978505015373,0.030203767120838165,0.03172542154788971,-0.0769231766462326,-0.014443146996200085,-0.006988691631704569,-0.058559466153383255,-0.0015324725536629558,0.06250637769699097,0.08957813680171967,0.09099245071411133,0.03653217852115631,-0.043696753680706024,-0.02577134594321251,3.435700637055561e-05,0.05915049836039543,0.018036285415291786,0.010070807300508022,-0.05790853127837181,0.036343805491924286,0.0008562053553760052,0.023679440841078758,0.1056213304400444,0.14530502259731293,0.14183276891708374,0.041560836136341095,0.09004256129264832,0.007650298066437244,-0.02532459981739521,-0.02674066834151745,6.472809764090925e-05,0.033296067267656326,-0.035362038761377335,0.06903207302093506,0.00739365816116333,0.041571374982595444,0.03515557199716568,-0.04844904690980911,-0.033919017761945724,-0.01822684332728386,-0.062326304614543915,-0.06333646923303604,0.06199736148118973,-0.022438600659370422,0.009763896465301514,0.037303220480680466,0.16835451126098633,0.20569121837615967,0.09166158735752106,0.11823525279760361,0.14342573285102844,-0.016639087349176407,-0.030833963304758072,-0.009207949973642826,-0.052302271127700806,-0.020987609401345253,0.05694118142127991,-0.11734192073345184,-0.1642620712518692,-0.1630813628435135,-0.08130675554275513,-0.03298339992761612,-0.17941364645957947,-0.09776827692985535,-0.023689385503530502,-0.13851840794086456,-0.04189446568489075,-0.054683536291122437,-0.07641179859638214,-0.06817352771759033,-0.012723199091851711,-0.09845045208930969,-0.17587284743785858,-0.20579123497009277,-0.04066294804215431,-0.10900624096393585,-0.25473979115486145,-0.35196319222450256,-0.15955406427383423,-0.1364264190196991,-0.0641087144613266,0.07647132128477097,-0.051513127982616425,-0.04792286455631256,-0.04830411449074745,0.09518706798553467,0.0735386312007904,-0.02544073946774006,-0.12685802578926086,-0.22511662542819977,-0.2043124884366989,-0.19304092228412628,-0.11268238723278046,-0.3121322989463806,-0.13324283063411713,-0.020292488858103752,-0.06925943493843079,-0.0018913127714768052,0.05080581456422806,0.09004580229520798,-0.06003982573747635,-0.10148178040981293,-0.2662556767463684,-0.3454567790031433,-0.21529771387577057,-0.05238642916083336,0.07097660005092621,-0.1313386857509613,-0.03364550322294235,0.09653378278017044,0.03302161395549774,0.08724518120288849,0.059289395809173584,0.005867400206625462,-0.07403723895549774,-0.015123996883630753,-0.03839784488081932,-0.011548726819455624,0.0991281270980835,0.006795787718147039,0.11408688873052597,0.13922545313835144,-0.0008379925275221467,0.08643010258674622,0.0036726088728755713,-0.02569838985800743,-0.05762552469968796,0.08141540735960007,-0.01190037839114666,-0.14309635758399963,-0.07750663161277771,0.19645284116268158,0.24082742631435394,0.10426873713731766,0.021004753187298775,-0.12990354001522064,-0.07826428860425949,-0.13516095280647278,-0.17918045818805695,-0.108043372631073,-0.05975068733096123,-0.050292033702135086,0.007230634335428476,0.05755425989627838,0.2371906340122223,0.22456085681915283,0.19119174778461456,0.10031116753816605,0.0603768527507782,0.02911568619310856,0.05076233670115471,0.21600838005542755,0.06916432827711105,-0.014511941000819206,0.023269180208444595,-0.09181943535804749,-0.06387665867805481,0.18005339801311493,0.23146198689937592,0.0035386483650654554,-0.08850683271884918,-0.1848006695508957,-0.10360973328351974,-0.09913173317909241,-0.025465117767453194,0.16725590825080872,0.0256009791046381,-0.1001146137714386,-0.13191044330596924,0.010118478909134865,0.07746127247810364,0.10147367417812347,0.08648443222045898,0.053547102957963943,0.045283734798431396,-0.003213686402887106,0.13698115944862366,0.12988923490047455,0.0333322174847126,0.05297732353210449,0.0216327216476202,0.006490493193268776,-0.0026525731664150953,0.2186487764120102,0.09721452742815018,0.09450319409370422,0.12613466382026672,-0.003099494380876422,0.16450318694114685,0.06727069616317749,0.09526746720075607,0.0750034749507904,0.09861478954553604,0.10650595277547836,0.028878042474389076,0.1070621907711029,0.10458892583847046,0.1107015535235405,0.050560738891363144,0.09761859476566315,0.17714770138263702,-0.007130617741495371,0.02770981192588806,-0.035000402480363846,0.02108883112668991,0.01106108259409666,-0.04481365904211998,0.0026191161014139652,-0.07707062363624573,0.09062908589839935,0.09280521422624588,-0.02697846107184887,0.09201881289482117,0.1571446806192398,0.21697287261486053,-0.025338271632790565,0.055742837488651276,0.06137794628739357,0.031682953238487244,-0.01976323500275612,-0.026358960196375847,-0.04740026593208313,-0.027748920023441315,0.08795091509819031,0.11559372395277023,0.1958705633878708,0.11682417243719101,0.04788501188158989,0.03603630140423775,0.021010946482419968,-0.0879082903265953,-0.01617901585996151,-0.03224673867225647,-0.13589391112327576,-0.17779016494750977,-0.13754123449325562,-0.11392801254987717,-0.11851710081100464,-0.055351149290800095,0.11013392359018326,-0.002819925779476762,0.07038546353578568,-0.016797905787825584,-0.12649676203727722,-0.1556963175535202,-0.20010489225387573,-0.15424272418022156,-0.1981113851070404,-0.16429799795150757,-0.07117389887571335,-0.09894167631864548,-0.18587726354599,-0.16155369579792023,0.026922699064016342,-0.06650584936141968,-0.06984779238700867,-0.25171199440956116,-0.3292960226535797,-0.3336807191371918,-0.25701943039894104,-0.1748260110616684,-0.19696401059627533,-0.16921493411064148,-0.18349182605743408,-0.19648492336273193,-0.1602926254272461,-0.2136082798242569,0.13026586174964905,0.16352219879627228,0.0011484265560284257,-0.17959557473659515,-0.34997209906578064,-0.253561794757843,-0.23703445494174957,-0.1675466001033783,-0.27284690737724304,-0.18226484954357147,-0.05472731590270996,-0.09502921998500824,-0.00795058999210596,0.02175389975309372,0.08313322812318802,0.14674289524555206,0.1403074413537979,0.02378746122121811,-0.010923586785793304,0.022592345252633095,-0.14896562695503235,-0.042698126286268234,-0.06285960972309113,0.0066834199242293835,0.04199611395597458,-0.027905914932489395,0.05107983201742172,0.04174923524260521,-0.1359938532114029,-0.0916607677936554,-0.11854492872953415,-0.13044288754463196,-0.26570960879325867,-0.3304695785045624,-0.2473372220993042,-0.20440252125263214,-0.19784776866436005,-0.041408102959394455,0.012737864628434181,-0.14205774664878845,-0.21703648567199707,-0.11528374254703522,-0.032662224024534225,-0.1306135654449463,-0.23930010199546814,-0.12564297020435333,-0.013498370535671711,-0.10118314623832703,0.03144168481230736,-0.0976530909538269,-0.26872313022613525,-0.19585056602954865,-0.06465931981801987,-0.02729511447250843,-0.007057128939777613,0.17077282071113586,0.051996778696775436,-0.07277824729681015,-0.145005464553833,-0.31225550174713135,-0.11150240898132324,0.030206577852368355,-0.1275743693113327,0.02352474443614483,-0.18982094526290894,-0.18735575675964355,-0.010705514810979366,-0.23746894299983978,-0.21163906157016754,-0.03057362511754036,-0.02725944109261036,-0.10483604669570923,-0.2025037258863449,-0.14702941477298737,-0.26786431670188904,-0.22494357824325562,-0.3611501455307007,-0.34712278842926025,-0.1364162415266037,-0.02541365846991539,0.15045180916786194,0.17739181220531464,-0.017215978354215622,-0.0876978188753128,-0.28545984625816345,-0.3069363534450531,-0.216176837682724,-0.17117200791835785,-0.24371519684791565,-0.5997846126556396,-0.9256638288497925,-0.5861994028091431,-0.029279692098498344,-0.0253172405064106,-0.005677815061062574,0.013699439354240894,0.0002681683108676225,0.001594788976944983,-0.31649497151374817,-0.2041444182395935,-0.2499283105134964,-0.20578551292419434,-0.30358558893203735,-0.271134614944458,-0.5799168944358826,-0.4183804392814636,-0.03167853131890297,0.15363922715187073,-0.023599915206432343,-0.00037741620326414704,0.000587336253374815,0.06430429220199585,-0.3530346155166626,-0.27006781101226807,-0.0920567512512207,-0.14773541688919067,-0.23401853442192078,-0.23154610395431519,-0.19650530815124512,-0.24881739914417267,-0.20751801133155823,-0.059712618589401245,0.040152132511138916,0.01225433312356472,-0.008139402605593204,0.013975076377391815,-0.1717275232076645,-0.1025676503777504,-0.13337171077728271,-0.14095629751682281,-0.20042361319065094,-0.2002410590648651,-0.1614486426115036,-0.12094616144895554,-0.09802571684122086,-0.09373776614665985,0.0008055398357100785,0.10860379040241241,0.049524515867233276,0.08490104973316193,-0.28206953406333923,-0.258978933095932,-0.2945244014263153,-0.22259794175624847,-0.25348639488220215,-0.3166780471801758,-0.17316335439682007,-0.07870034873485565,-0.026694001629948616,-0.14546307921409607,-0.10703437775373459,0.03072946146130562,-0.08157220482826233,0.006908061448484659,0.20190908014774323,0.2280384749174118,0.22217285633087158,0.1825343668460846,0.18281956017017365,0.2048507034778595,0.12809592485427856,0.09259321540594101,0.027530495077371597,0.06266129016876221,0.09565100818872452,0.12725666165351868,0.22166556119918823,0.24697093665599823,0.013193146325647831,0.0957847461104393,0.09064537286758423,0.1504771113395691,0.00854227039963007,-0.09175556153059006,-0.06081738322973251,0.027169877663254738,0.05893674120306969,0.026774736121296883,-0.015515296719968319,0.11305514723062515,0.09712541103363037,0.07836426049470901,-0.007400685455650091,0.08070635795593262,0.0828009620308876,0.15538574755191803,-0.004029725678265095,-0.04283499717712402,-0.013603584840893745,0.014100342057645321,0.0017639913130551577,0.0075407857075333595,-0.0008048911695368588,0.03625571355223656,-0.044131532311439514,-0.06985075026750565,0.07347208261489868,0.035023998469114304,0.002983477897942066,0.05011546611785889,-0.05205405130982399,-0.019298776984214783,-0.028239835053682327,0.06719965487718582,-0.01796671748161316,0.03240649774670601,0.014722289517521858,0.05161530524492264,0.061307232826948166,-0.06252823024988174,0.041756771504879,0.09842493385076523,0.061819545924663544,-0.0009789761388674378,-0.02407255582511425,-0.07870963215827942,-0.05370701104402542,-0.1401795595884323,-0.16856341063976288,-0.14501310884952545,-0.14403226971626282,-0.07270997017621994,0.05103500559926033,0.09072902798652649,0.013830529525876045,-0.06907852739095688,-0.06306623667478561,0.01814110204577446,-0.04244714975357056,-0.0054971459321677685,-0.018939238041639328,-0.04118809849023819,-0.13763634860515594,-0.09252363443374634,-0.09367075562477112,-0.1442449539899826,0.007677400019019842,0.04099900275468826,-0.0015159978065639734,-0.12486956268548965,-0.06255758553743362,0.07309466600418091,-0.0036314718890935183,-0.04978794977068901,-0.1180579885840416,-0.34034690260887146,-0.2788669466972351,-0.09189512580633163,-0.0375770665705204,-0.09051799029111862,-0.013408495113253593,0.037983980029821396,0.0752951055765152,-0.07695646584033966,-0.11413836479187012,-0.22336922585964203,-0.05471019074320793,-0.030858246609568596,0.04929301142692566,-0.12354561686515808,-0.288241982460022,-0.2044752836227417,-0.0477704256772995,-0.11541058868169785,-0.01279639732092619,-0.023464398458600044,-0.020107273012399673,-0.05524981766939163,-0.08202903717756271,-0.19017697870731354,-0.28075310587882996,-0.027579395100474358,-0.13304470479488373,0.02534453757107258,-0.07426463812589645,-0.11270574480295181,0.046650175005197525,-0.11285627633333206,-0.08793451637029648,-0.0558251328766346,-0.06970199942588806,-0.1361362636089325,-0.06515572965145111,-0.15464727580547333,-0.11825747787952423,-0.025011342018842697,-0.14688603579998016,-0.15461821854114532,0.035904355347156525,-0.07319692522287369,0.07952136546373367,0.033764004707336426,-0.025530416518449783,-0.009718956425786018,0.0028153781313449144,-0.13180027902126312,-0.01458855252712965,-0.07560697942972183,-0.049840252846479416,0.12476073205471039,-0.08411510288715363,-0.14511436223983765,0.05162210017442703,-0.1006944552063942,-0.017822187393903732,0.03327133506536484,0.0876207947731018,0.09394511580467224,0.03722731024026871,-0.05458785593509674,-0.040607210248708725,0.011003326624631882,-0.11279207468032837,0.1222945973277092,0.02113882452249527,-0.13599491119384766,-0.07222429662942886,-0.05431568622589111,0.02944604493677616,0.05624893307685852,0.07100123167037964,0.16194504499435425,0.06097550690174103,-0.04283500835299492,0.0015807384625077248,0.021353378891944885,-0.035901278257369995,0.032468415796756744,0.0633116289973259,0.0016158272046595812,-0.08682862669229507,0.014603218995034695,-0.02214771695435047,-0.00827111303806305,0.049214646220207214,0.20299404859542847,0.0024876962415874004,0.014822491444647312,0.0072453017346560955,0.1329909712076187,0.10026125609874725,0.09167162328958511,0.14565697312355042,0.0510115921497345,-0.0006166949169710279,0.12666626274585724,0.05595934018492699,-0.04827229678630829,0.07456093281507492,0.171931654214859,0.03965913876891136,0.03271549195051193,-0.005102864932268858,0.0035005228128284216,0.04246627166867256,0.05241652950644493,0.05006866157054901,-0.0004901266074739397,0.03255322948098183,0.024185562506318092,-0.015585117042064667,0.009942476637661457,0.0035940471570938826,-0.025307083502411842,0.02922985330224037,0.020458931103348732,-0.04279244691133499,-0.007776057813316584,-0.027596279978752136,-0.03643273934721947,0.035466112196445465,-0.03643827140331268,-0.012883361428976059,0.07325229793787003,-0.05283001810312271,-0.011974171735346317,0.0001288108469452709,0.005115616135299206,-0.013978863134980202,0.04281209781765938,-0.0042036063969135284,0.033528391271829605,-0.016252418980002403,0.04732559248805046,0.010460715740919113,-0.05031604319810867,-0.017942702397704124,0.016487818211317062,0.06424912810325623,0.009380635805428028,0.04432709142565727,0.012573257088661194,-0.007409714162349701,0.03212415426969528,0.02786160819232464,0.01490845251828432,-0.015343325212597847,0.040304459631443024,0.10539449006319046,0.013615844771265984,0.026674995198845863,-0.01055973768234253,0.028245337307453156,0.03274473175406456,0.03825047239661217,0.024510368704795837,0.016371145844459534,0.02614414133131504,-0.010404182597994804,0.040742773562669754,-0.026122037321329117,-0.03942521661520004,0.01728188246488571,0.04354372248053551,-0.008011714555323124,0.03145210072398186,-0.046546995639801025,0.0018657438922673464,0.036988385021686554,-0.0334426648914814,-0.003447502851486206,0.01972820609807968,0.005704906303435564,-0.05935687944293022,-0.03877405822277069,-0.0048823049291968346,0.0561814047396183,-0.03765840455889702,-0.014158360660076141,-0.050135646015405655,-0.0021424097940325737,-0.01889524608850479,0.0048444559797644615,0.03386707976460457,-0.04327119141817093,0.020125800743699074,0.016311466693878174,-0.059821296483278275,-0.05082210898399353,0.010542815551161766,0.009983942843973637,-0.023900579661130905,0.037415746599435806,-0.007783722132444382,-0.042039211839437485,-0.004104194231331348,-0.026355305686593056,-0.004258223809301853,-0.029576461762189865,0.01886119693517685,0.017448758706450462,-0.012413833290338516,-0.041221942752599716,-0.006755792535841465,0.04571470990777016,0.047536857426166534,-0.0033324232790619135,0.022163860499858856,0.04199626296758652,-0.03264014050364494,-0.014996804296970367,-0.05114716291427612,0.016215568408370018,-0.05743841454386711,-0.049044571816921234,-0.07365251332521439,-0.018421603366732597,-0.018763156607747078,0.010907687246799469,0.03436513617634773,-0.04806182160973549,0.008295799605548382,-0.05071306228637695,0.008204109035432339,-0.019874604418873787,0.00855254102498293,0.04282231628894806,-0.031193602830171585,-0.021959347650408745,-0.0542200542986393,-0.0422583632171154,-0.02891894243657589,-0.04452413693070412,0.03093603067100048,0.021185647696256638,-0.035405006259679794,-0.03301500901579857,0.005516651086509228,0.034905705600976944,0.027142100036144257,0.038274649530649185,0.049626655876636505,-0.006506231613457203,-0.05909227952361107,0.022136660292744637,0.014305055141448975,0.05293083190917969,-0.011656799353659153,0.032868605107069016,-0.03234131634235382,0.04547153413295746,-0.0043363855220377445,-0.009129627607762814,0.0452168844640255,0.08068535476922989,-0.023938169702887535,-0.02394450455904007,0.00313292327336967,0.008894437924027443,0.001514278817921877,0.02813512273132801,-0.008210309781134129,0.012627653777599335,0.03611186519265175,0.02468245103955269,-0.048759445548057556,0.0033035543747246265,-0.014987296424806118,0.02804388292133808,0.03794826939702034,0.015382040292024612,0.03496275097131729,0.013346359133720398,0.06346630305051804,0.0863044485449791,0.016525819897651672,0.07768543064594269,-0.001784822321496904,0.025017986074090004,0.017841912806034088,0.026380186900496483,-0.009566015563905239,0.09911655634641647,0.014741544611752033,0.09758306294679642,0.04433010146021843,0.037611644715070724,0.11297411471605301,0.0597502626478672,0.08831614255905151,0.058076947927474976,0.05760115012526512,0.060048479586839676,0.03688131645321846,0.049203332513570786,0.05492803454399109,0.10626040399074554,0.04879043251276016,-0.012239414267241955,0.06979838758707047,0.026436252519488335,0.03506845980882645,0.063619464635849,0.01360469963401556,0.05996071174740791,0.11211454123258591,0.09893938153982162,0.07012169063091278,0.06988395005464554,0.039483003318309784,0.11968643218278885,0.08559288829565048,0.05440182611346245,0.0735064148902893,0.07368297874927521,0.06562162190675735,0.05881006643176079,0.0277931597083807,0.07126165181398392,0.0940496176481247,0.07191213220357895,0.07367603480815887,0.09631706774234772,0.08577223867177963,0.06127019599080086,0.03237549588084221,-0.00862172432243824,-0.01618056930601597,0.033004723489284515,0.03180370852351189,0.0841510221362114,0.014924437738955021,0.0051308865658938885,-0.0021533677354454994,-0.02541111223399639,-0.005737366154789925,0.10921715945005417,0.0932527557015419,0.041713643819093704,0.059924907982349396,0.09231721609830856,-0.0018629084806889296,0.009700370021164417,0.025181235745549202,0.08637011051177979,0.09676925092935562,0.056607987731695175,-0.06518374383449554,-0.014412866905331612,-0.0055310167372226715,0.06393393129110336,0.04357577860355377,-0.02283407561480999,-0.006544649600982666,0.0788848027586937,0.05028561130166054,0.043282024562358856,0.0174498800188303,0.043562151491642,0.03800446540117264,0.05137283727526665,0.004120574798434973,0.018945785239338875,-0.007028374820947647,-0.054215528070926666,-0.039283689111471176,-0.0029274015687406063,-0.03393601253628731,0.010141469538211823,-0.05058889091014862,-0.03626297414302826,-0.11565637588500977,-0.09518075734376907,-0.17371287941932678,-0.07176151126623154,-0.03786716237664223,0.0029621643479913473,-0.03341618552803993,-0.07327369600534439,-0.03572685271501541,-0.07988002896308899,0.031731635332107544,0.03477298840880394,-0.04853089526295662,0.015191812068223953,0.10026170313358307,0.04620877653360367,0.03704718500375748,-0.06765691190958023,-0.057009257376194,-0.017933551222085953,-0.08334033936262131,-0.014191413298249245,0.057453617453575134,-0.010628076270222664,0.033731307834386826,0.07632816582918167,0.06773409992456436,-0.003954268526285887,0.03667612373828888,0.09680375456809998,0.04130241647362709,0.054558951407670975,-0.03252585977315903,0.02688402682542801,-0.037540726363658905,-0.10754456371068954,-0.0064534894190728664,0.017992641776800156,0.01111078541725874,0.07871569693088531,-0.014914782717823982,0.09243478626012802,0.026293298229575157,0.06265943497419357,-0.003761394415050745,0.0188783910125494,0.039895154535770416,0.007575737312436104,-0.042237378656864166,-0.037311628460884094,0.0022564330138266087,-0.05819613113999367,0.003718989435583353,0.030130792409181595,-0.028270117938518524,-0.01505842711776495,0.07700696587562561,0.07044630497694016,0.11222377419471741,0.050493258982896805,0.0658952072262764,-0.09235437959432602,-0.01749008521437645,-0.008218513801693916,0.035219840705394745,-0.0516434982419014,-0.028687769547104836,0.04907076060771942,0.007813343778252602,0.08283426612615585,0.15648719668388367,0.139759823679924,0.0932195633649826,0.09348767250776291,0.08932486921548843,-0.029350455850362778,-0.013234755955636501,-0.0063284155912697315,0.05235417187213898,-0.036893054842948914,0.02739466167986393,0.057181984186172485,0.05010780319571495,0.12554848194122314,0.07376666367053986,0.07720410078763962,0.05819552391767502,-0.00396149093285203,-0.0025051492266356945,-0.0016896789893507957,-0.02868969924747944,-0.05332430824637413,0.003823729930445552,0.03881821408867836,0.007906980812549591,0.07192446291446686,0.09170544892549515,0.0522599034011364,0.04199930280447006,0.033222876489162445,0.04326099902391434,0.0006822266150265932,0.007021778728812933,-0.012598898261785507,-0.023134691640734673,0.04623262956738472,0.017544014379382133,0.045378413051366806,-0.011095899157226086,0.053421951830387115,0.02744707465171814,0.06193924695253372,0.0428168959915638,0.09749290347099304,0.09564892202615738,0.16182632744312286,0.07373985648155212,0.010408291593194008,-0.020351022481918335,-0.022633329033851624,0.011887573637068272,0.2255593240261078,0.226180300116539,0.17447693645954132,0.17832347750663757,0.24574968218803406,0.09636177122592926,0.07808087021112442,-0.021898053586483,0.056121230125427246,-0.057533148676157,-0.018431097269058228,-0.04536827281117439,-0.08272185921669006,-0.02922912873327732,0.29289793968200684,0.060190748423337936,0.08292555063962936,0.09083953499794006,0.23293890058994293,0.08651092648506165,-0.0027778891380876303,-0.1314278244972229,-0.12128192186355591,-0.0988604798913002,-0.10895099490880966,-0.12634186446666718,-0.12190375477075577,-0.2101648896932602,0.2485152930021286,0.1186184361577034,-0.014014809392392635,-0.047937825322151184,-0.042427316308021545,-0.04323119670152664,0.13097120821475983,0.014017000794410706,-0.13238270580768585,-0.11957331746816635,-0.0302728321403265,-0.14202675223350525,-0.06294582784175873,-0.07799248397350311,0.21895161271095276,0.20207932591438293,0.18463855981826782,0.06262271851301193,0.015220432542264462,-0.18176594376564026,0.1291547268629074,0.026877466589212418,-0.026513496413826942,-0.12182946503162384,0.04633881524205208,-0.08624006062746048,-0.0028829900547862053,0.07532736659049988,0.04866469278931618,0.12208735197782516,0.19834938645362854,0.041033606976270676,-0.0055089546367526054,0.0332450307905674,0.12942226231098175,0.050811465829610825,0.03879186511039734,0.051078520715236664,0.07440105825662613,-0.1333792507648468,-0.09144329279661179,-0.004610664211213589,-0.06049065664410591,-0.0073253377340734005,-0.06067048758268356,0.01322426088154316,-0.05967684090137482,-0.10477114468812943,-0.053238190710544586,-0.13778148591518402,-0.26214203238487244,-0.10115434229373932,0.16716203093528748,-0.04448772594332695,-0.1348472386598587,-0.038169827312231064,-0.14217187464237213,-0.12021712213754654,-0.06599891930818558,-0.02046581357717514,0.20610980689525604,-0.009022489190101624,0.12443973124027252,-0.14348706603050232,-0.30064961314201355,-0.32161399722099304,-0.0751924216747284,-0.0038339493330568075,-0.08281932026147842,-0.014084812253713608,-0.0853809341788292,-0.007963895797729492,-0.035304561257362366,-0.08506014943122864,-0.1369044929742813,0.03137130290269852,-0.0955413281917572,-0.006423669867217541,-0.14663976430892944,-0.16449102759361267,-0.02810552343726158,-0.11597535759210587,-0.15346379578113556,-0.09576573967933655,-0.12306374311447144,-0.07794422656297684,-0.20245376229286194,-0.19836582243442535,-0.2766839265823364,-0.043807532638311386,-0.03661958873271942,-0.26396334171295166,-0.039943065494298935,-0.032232724130153656,-0.018739109858870506,0.14357630908489227,0.016109462827444077,-0.05782491713762283,-0.2449360489845276,-0.15610456466674805,-0.26018112897872925,-0.18924997746944427,-0.2231326401233673,-0.0930100679397583,0.011361414566636086,-0.13253892958164215,-0.0368301197886467,0.06007028743624687,-0.08168255537748337,-0.0603117011487484,-0.0398164764046669,-0.011912682093679905,-0.23709797859191895,-0.010407309979200363,-0.11890532821416855,-0.06675861775875092,-0.18598325550556183,0.08235199749469757,0.07668178528547287,-0.06084089353680611,0.07649378478527069,0.08578763157129288,-0.005644630175083876,-0.04789157211780548,0.03174139931797981,0.08774829655885696,-0.16071784496307373,-0.05412089452147484,-0.09855587780475616,-0.1223028153181076,-0.15086139738559723,0.022467680275440216,0.12430009245872498,0.035664573311805725,-0.011601882986724377,-0.035996243357658386,-0.04945306107401848,0.03494325652718544,0.006895114202052355,0.0850810557603836,-0.10538618266582489,-0.03254332020878792,-0.07873225957155228,-0.14714115858078003,-0.20293743908405304,-0.23697799444198608,0.000556994928047061,0.040391240268945694,0.06250171363353729,0.02749892696738243,0.04119191691279411,0.055397532880306244,0.013668136671185493,0.1182207390666008,-0.1973859816789627,-0.12506720423698425,-0.18591322004795074,-0.16037045419216156,-0.15333132445812225,-0.17267495393753052,0.016496554017066956,0.05743100121617317,0.07947421818971634,0.13385365903377533,0.05070947855710983,0.07757040113210678,0.11636792123317719,0.11975662410259247,0.28184446692466736,0.18323644995689392,0.17010337114334106,0.253219872713089,0.21692045032978058,0.16103044152259827,0.16296513378620148,0.10266935080289841,0.09890467673540115,0.1439083367586136,0.12239544838666916,0.1845884621143341,0.1823505461215973,0.19728045165538788,0.13738016784191132,0.031085625290870667,0.05134640634059906,0.09112317860126495,0.12686236202716827,0.06980004161596298,0.0970422551035881,0.011212673969566822,-0.01839764229953289,0.05045748129487038,0.07441335916519165,0.04011622816324234,0.04071342200040817,0.02756025269627571,0.11219797283411026,-0.007917413488030434,0.015682045370340347,0.04388151317834854,0.11593994498252869,0.04650328680872917,0.04702887311577797,-0.005995956249535084,-0.02725869044661522,-0.023886872455477715,0.03918436914682388,0.03594446927309036,0.05604858323931694,0.02308468706905842,0.005416836589574814,0.08006302267313004,0.11055750399827957,0.06686942279338837,0.051805540919303894,0.05787401273846626,0.042887087911367416,0.0608372762799263,0.04653420299291611,0.006679965648800135,-0.0030977476853877306,0.05575607344508171,0.029667101800441742,0.015743190422654152,0.03304146230220795,0.04415147751569748,0.022063598036766052,0.025149278342723846,0.054365113377571106,0.08720018714666367,0.11587298661470413,0.0327988862991333,0.02468211017549038,-0.10337475687265396,-0.09835347533226013,-0.03977113962173462,-0.03441143035888672,0.04769793152809143,-0.008462178520858288,0.04284866899251938,0.0018553809495642781,-0.007742564659565687,0.057747479528188705,0.08941779285669327,-0.002774098888039589,-0.11300681531429291,-0.07204805314540863,-0.09838517010211945,-0.12897278368473053,-0.06520629674196243,-0.06694044172763824,0.03624329715967178,-0.10084177553653717,-0.0002768251288216561,-0.08066125214099884,-0.02367445081472397,0.10331752151250839,0.039568088948726654,-0.044434767216444016,-0.19416972994804382,-0.12469703704118729,-0.17405304312705994,-0.09655296802520752,-0.053316134959459305,-0.013365451246500015,0.05129816383123398,-0.11112496256828308,0.006068563088774681,-0.06050746887922287,-0.02353610098361969,0.028309432789683342,0.014098168350756168,-0.010543727315962315,-0.11892972886562347,-0.16493578255176544,-0.1184539720416069,-0.1497131884098053,-0.07652973383665085,-0.10120049864053726,0.0016860661562532187,-0.03051389753818512,0.017611132934689522,-0.044827427715063095,-0.02921142801642418,-0.04276633262634277,-0.00779626052826643,-0.011470714583992958,-0.12432127445936203,-0.03526543453335762,-0.10397515445947647,-0.1141514778137207,-0.052213769406080246,-0.04549553617835045,-0.08527959138154984,-0.05012079328298569,-0.12866252660751343,-0.0904577299952507,-0.061306875199079514,-0.08752886950969696,0.038475051522254944,-0.006925724912434816,-0.13144759833812714,-0.037196382880210876,-0.05458986014127731,-0.12831680476665497,-0.06609992682933807,-0.08549194782972336,-0.00985361635684967,-0.022172801196575165,-0.03084268607199192,0.01238504983484745,0.006992905400693417,-0.05874514579772949,0.09352625161409378,0.09935532510280609,0.017417408525943756,0.06694578379392624,-0.08351507037878036,-0.09699055552482605,-0.01104572881013155,-0.029847526922822,0.020129214972257614,0.013782029040157795,0.03875257819890976,-0.0027564777992665768,0.003449010895565152,-0.023605922237038612,0.10950680822134018,0.19621169567108154,-0.0068694124929606915,-0.0012883059680461884,-0.04763370007276535,-0.1052459105849266,-0.011403617449104786,-0.024475285783410072,0.03411804512143135,0.0016536259790882468,0.01058613508939743,-0.05577152967453003,-0.06397096812725067,-0.07216832041740417,0.02157645672559738,0.10883031785488129,-0.034347422420978546,0.011525312438607216,0.0014874970074743032,-0.053267333656549454,-0.03500792011618614,-0.07101907581090927,0.0034475873690098524,0.034756213426589966,-0.023856740444898605,-0.09191367030143738,-0.1047809049487114,-0.008923985995352268,-0.007944809272885323,0.08008985966444016,0.06399181485176086,0.06328681111335754,0.047235239297151566,-0.023583954200148582,-0.01296436507254839,-0.015002332627773285,-0.04005669802427292,0.2017296701669693,0.15431825816631317,0.17369487881660461,0.18430988490581512,0.11531077325344086,0.16499271988868713,0.2269122153520584,0.14869064092636108,0.1433829367160797,0.21714511513710022,0.20259568095207214,0.24487555027008057,0.2025901824235916,0.19982287287712097,0.11768617480993271,0.051256533712148666,0.0658767819404602,0.07904639840126038,0.13066329061985016,0.04088244587182999,0.06063465401530266,0.06844857335090637,0.12979282438755035,0.1368013471364975,0.162474125623703,0.10164842009544373,0.11347140371799469,0.13119065761566162,0.09165335446596146,0.006864192429929972,0.05449702963232994,0.0457889586687088,0.06480880826711655,0.0016430276446044445,0.01765112578868866,0.05712682381272316,0.06757603585720062,0.07470360398292542,0.08709625154733658,0.08645641058683395,0.03190767392516136,0.05642956867814064,0.024670036509633064,0.0764123797416687,0.0614892952144146,0.07193061709403992,0.1025005355477333,0.10019076615571976,0.01890544965863228,0.07783884555101395,0.06689093261957169,0.08533504605293274,0.00711036054417491,0.06960606575012207,0.07180304825305939,0.033032044768333435,-0.002049156231805682,-0.03375299274921417,-0.012539521791040897,0.016676893457770348,0.0743793249130249,0.048257701098918915,0.01535971462726593,0.056436873972415924,0.01293578464537859,0.025227366015315056,0.020247574895620346,0.02060682699084282,0.014262148179113865,0.06037743762135506,0.01545390859246254,0.05567050352692604,-0.018145641312003136,0.026185987517237663,0.07579640299081802,0.0069980015978217125,-0.14308306574821472,-0.07719521224498749,-0.007499330677092075,0.05189521610736847,-0.01490879151970148,0.022947469726204872,0.05267205089330673,0.06329760700464249,-0.010004202835261822,-0.0028993478044867516,0.024809852242469788,0.008187544532120228,-0.03886210918426514,-0.1115434318780899,-0.15796245634555817,-0.07674923539161682,-0.04566404968500137,0.041184812784194946,-0.026693206280469894,0.05491980165243149,-0.009981024079024792,0.0475996695458889,0.02603984996676445,0.04363984614610672,0.028199240565299988,0.005453137215226889,-0.015623969957232475,0.02249225042760372,0.02811533957719803,0.05603853240609169,-0.029477519914507866,-0.055281057953834534,-0.09068189561367035,0.005467006471008062,-0.02611660771071911,-0.030435245484113693,0.0956985205411911,0.03044130466878414,0.062360987067222595,0.051838960498571396,-0.0026594866067171097,-0.03456004709005356,0.03011924773454666,0.03972235321998596,-0.009339969605207443,-0.05553406849503517,-0.08577866107225418,-0.08794771879911423,0.00130541215185076,0.02138388343155384,0.04795566573739052,0.03740888088941574,0.020102908834815025,0.05796349421143532,0.052557218819856644,-0.026477353647351265,-0.0019058854086324573,0.018584124743938446,0.01810593158006668,-0.006664238404482603,-0.0642634704709053,-0.019813016057014465,0.024599023163318634,0.008518092334270477,0.06656425446271896,0.06034301221370697,0.01653379201889038,0.045350395143032074,0.04392780363559723,0.003889154875651002,0.06689249724149704,0.07364100962877274,-0.03696545585989952,-0.011966424062848091,-0.05481036752462387,-0.029703449457883835,-0.03066284768283367,-0.02221805416047573,0.18198733031749725,0.10925120860338211,-0.018887335434556007,-0.006890836637467146,-0.00429248483851552,-0.01717492751777172,0.06712141633033752,0.14742477238178253,-0.004491584375500679,-0.03171820938587189,-0.09654656797647476,-0.030762065201997757,-0.09276730567216873,-0.030256088823080063,0.07798673957586288,0.002229683566838503,-0.07714684307575226,0.00348088052123785,0.02487233839929104,0.013239658437669277,0.10153082758188248,0.060194097459316254,-0.025565218180418015,-0.009337389841675758,-0.08967050164937973,-0.09299460053443909,-0.1672595888376236,-0.17613966763019562,0.03707081452012062,-0.0036557598505169153,-0.021169986575841904,0.05791764333844185,0.09907862544059753,0.06210169941186905,0.15037018060684204,0.06042921170592308,0.09125629812479019,0.0296038668602705,0.03201321139931679,-0.07579011470079422,-0.10279986262321472,-0.1228531002998352,0.10050873458385468,0.07579502463340759,0.11078780144453049,0.11217977106571198,0.07926592975854874,0.01870744861662388,0.007732039783149958,0.08681238442659378,0.04052635282278061,6.0315745940897614e-05,-0.02208995260298252,0.006117735989391804,0.023198120296001434,0.081331767141819,0.053228799253702164,0.07835252583026886,0.08304712176322937,0.05339355766773224,0.111436627805233,0.09963962435722351,0.09047394245862961,0.14205843210220337,-0.016256695613265038,-0.03627559542655945,-0.0838569775223732,-0.07362784445285797,0.002165163168683648,0.12311183661222458,0.025659291073679924,0.05559679865837097,0.0460076667368412,-0.02142423577606678,0.0004270395147614181,-0.059248317033052444,0.04145316034555435,0.14946997165679932,0.06679098308086395,-0.019291523844003677,-0.06813013553619385,-0.08617614209651947,0.02955874428153038,0.06385213881731033,-0.06578291207551956,0.025924118235707283,-0.018685871735215187,0.007714366540312767,-0.07547488063573837,-0.11419723927974701,-0.09248768538236618,-0.06480998545885086,0.05295847728848457,-0.01704009249806404,-0.05214083939790726,-0.06795263290405273,-0.02580145373940468,-0.0371103398501873,0.02592284418642521,-0.015626724809408188,-0.019701050594449043,-0.05791931971907616,-0.14503002166748047,-0.16398902237415314,-0.13345935940742493,-0.1044720709323883,0.0026067951694130898,0.02600276842713356,-0.020564652979373932,0.05166587606072426,0.04211268201470375,0.0945683941245079,0.09255026280879974,0.10520679503679276,0.01715793088078499,-0.03516225516796112,-0.09144154191017151,-0.11219194531440735,-0.039395734667778015,-0.09956831485033035,-0.011050215922296047,0.01855137012898922,0.10942354798316956,0.10142971575260162,0.1446494162082672,0.15639817714691162,0.08729194849729538,0.1450977623462677,0.10320482403039932,-0.038135942071676254,-0.06881510466337204,-0.014165926724672318,0.03150360658764839,0.06336455047130585,0.0826086476445198,0.06655837595462799,0.05692984163761139,0.14318755269050598,0.10996996611356735,0.03938758373260498,0.10086820274591446,0.01868363283574581,0.011818768456578255,0.05919297784566879,0.05564891919493675,-0.04209720343351364,-0.03756009042263031,0.06032828614115715,0.024464435875415802,0.01689581573009491,0.02567409910261631,0.07041014730930328,0.10083159059286118,0.0256082471460104,0.09739227592945099,0.08419933915138245,0.0336039736866951,0.002529110759496689,-0.009865979664027691,0.09081906825304031,-0.051699697971343994,-0.020693179219961166,-0.02278333343565464,-0.00030476495157927275,0.02676880545914173,0.04933365061879158,0.08795498311519623,0.09778949618339539,0.08585003018379211,0.0643450915813446,0.024054763838648796,-0.08790119737386703,-0.0390772819519043,0.03211502358317375,-0.05339912325143814,-0.1097736582159996,-0.09207799285650253,0.0011715178843587637,0.020403895527124405,-0.03582814335823059,0.05438901484012604,0.1353713721036911,0.1111048236489296,0.05890714004635811,0.06334238499403,0.011078719981014729,-0.02279379777610302,0.0035529781598597765,-0.03883577510714531,-0.048329051584005356,0.0014377988409250975,0.042726337909698486,0.009039117954671383,0.04067017510533333,0.04389439523220062,0.11538064479827881,0.12243705242872238,0.12131321430206299,0.10566570609807968,0.04617929831147194,-0.004978356417268515,0.0423419252038002,0.03130272403359413,0.06091563031077385,0.04715627431869507,0.0700499415397644,0.02177390456199646,0.01786564104259014,0.09755664318799973,0.18337658047676086,0.07719901204109192,0.07454048842191696,0.15570859611034393,0.05146920680999756,-0.028567444533109665,-0.03771277889609337,-0.06286395341157913,-0.04314269497990608,0.0012337708612903953,0.029552126303315163,0.05550461262464523,0.029591688886284828,0.06784006208181381,0.12436136603355408,0.04577764496207237,0.07742709666490555,0.14880190789699554,0.12783050537109375,0.042830873280763626,0.05426808446645737,0.04304772615432739,0.007703084032982588,0.06499843299388885,0.025869380682706833,0.007435597945004702,0.05520331859588623,0.07749737054109573,0.11499602347612381,0.12982748448848724,0.14041565358638763,0.10017964243888855,0.025785399600863457,0.09398429840803146,0.07709593325853348,0.059160687029361725,0.1130489632487297,0.08038226515054703,0.08486772328615189,0.05517632141709328,0.07967857271432877,0.17354242503643036,0.12808629870414734,0.11344553530216217,0.04855787381529808,-0.0022387460339814425,0.03429233655333519,-0.01818579062819481,0.021351763978600502,0.06397322565317154,0.04682915285229683,0.03268903121352196,0.0600934736430645,0.06787919998168945,0.10365120321512222,0.13449746370315552,0.11840761452913284,0.04502511024475098,0.08542730659246445,-0.0008835878106765449,0.04198291897773743,0.1049376130104065,0.06893273442983627,0.035386957228183746,0.01700023002922535,0.06258110702037811,0.06465580314397812,0.06748118996620178,0.06633325666189194,0.10452700406312943,0.11463208496570587,0.08924715965986252,0.07451988011598587,0.07743199914693832,0.11754678189754486,0.1598678082227707,0.11998994648456573,0.07888287305831909,0.08701799809932709,0.10559520870447159,0.0848788321018219,0.12373208999633789,0.07044469565153122,0.09236007928848267,0.06583200395107269,0.05605395510792732,-0.010994120500981808,0.05506915971636772,0.10546393692493439,0.08042985945940018,0.09722791612148285,0.08205223828554153,0.1316644847393036,0.10768649727106094,0.1184273436665535,0.08859867602586746,0.08002685755491257,0.07931684702634811,0.10436495393514633,0.09930843859910965,0.08186642080545425,0.04027470946311951,0.014383101835846901,0.11451538652181625,0.1451759934425354,0.15985849499702454,0.12118953466415405,0.0703735202550888,0.0903983861207962,0.08084235340356827,0.044737476855516434,0.01382951159030199,0.04683879390358925,0.04063469171524048,-0.013624561950564384,0.010586455464363098,0.015176340937614441,0.05216224491596222,0.0718436911702156,0.05358252674341202,0.029866309836506844,0.027840934693813324,0.05333833023905754,0.047740451991558075,0.025626754388213158,0.020829815417528152,0.03333412855863571,0.04690176248550415,-0.025760959833860397,0.0321107842028141,-0.017111575230956078,0.0149767454713583,0.05844810605049133,0.015415576286613941,-0.04847543314099312,-0.005746831651777029,0.019886210560798645,-0.00121841870713979,-0.017077770084142685,0.0600263886153698,0.07623424381017685,0.05004242807626724,0.029354097321629524,0.04120301827788353,0.01997281238436699,-0.0759836807847023,-0.03904442861676216,-0.00635515758767724,-0.013498778454959393,0.017252178862690926,0.029365424066781998,-0.06589388102293015,-0.034555479884147644,-0.06835497170686722,-0.0295435581356287,-0.018424516543745995,0.02789757400751114,0.06429415941238403,0.03572723641991615,0.008801024407148361,0.023119976744055748,-0.022862639278173447,0.023959022015333176,0.008653014898300171,-0.019906260073184967,-0.02475648745894432,0.04032853618264198,-0.02781181037425995,0.04991646483540535,0.01904502511024475,0.04535645991563797,0.015678955242037773,0.014625625684857368,-0.06950977444648743,-0.035513997077941895,-0.0033745262771844864,-0.018720634281635284,0.011907920241355896,-0.06412594020366669,-0.009626143611967564,-0.030687473714351654,0.055341776460409164,0.036975257098674774,0.1382092982530594,0.05418933928012848,0.029370632022619247,-0.02112438902258873,-0.03448151797056198,-0.047994643449783325,0.0013147098943591118,0.013826326467096806,-0.015271159820258617,-0.0327942818403244,-0.015391497872769833,-0.033529650419950485,0.017258770763874054,0.07075782865285873,0.07543866336345673,0.07025990635156631,0.0017147919861599803,0.014775414951145649,-0.02817634865641594,-0.020198775455355644,0.07856103032827377,0.04435305297374725,0.003011873457580805,0.049314361065626144,-0.024237360805273056,0.017157865688204765,0.0617251954972744,0.086406409740448,0.10257656127214432,0.04833699390292168,0.057152654975652695,0.10819672048091888,0.11256071925163269,0.07006850838661194,0.09009235352277756,0.10318160802125931,0.09813017398118973,0.12159817665815353,0.017499597743153572,0.07636623084545135,0.06272736191749573,0.08570083975791931,0.05779831111431122,-0.026716023683547974,-0.024597782641649246,-0.082120880484581,-0.07412779331207275,-0.10084563493728638,-0.028621086850762367,-0.008707510307431221,-0.06827902793884277,-0.011940215714275837,-0.025586608797311783,0.020062454044818878,0.052377987653017044,0.04496108368039131,0.002610504627227783,0.058444470167160034,-0.021283742040395737,-0.015136358328163624,-0.04075139760971069,-0.0788905993103981,0.011754950508475304,-0.03832085803151131,-0.07179149985313416,-0.041661348193883896,-0.03707714378833771,0.022081902250647545,0.05216347798705101,0.08233140408992767,0.07600957155227661,0.0025565598625689745,0.01859254203736782,0.1318831741809845,0.05589707940816879,-0.04222907871007919,0.10788605362176895,-0.004509421531111002,-0.01593228057026863,-0.02118687331676483,0.005534758325666189,0.069359190762043,0.052305061370134354,0.07661949098110199,0.025683116167783737,-0.09779710322618484,-0.03529759496450424,0.13802728056907654,0.03843769431114197,-0.01090847048908472,0.04120652377605438,0.0889095589518547,0.04618686065077782,0.05350594222545624,0.016922935843467712,0.048054955899715424,-0.07077336311340332,0.07282151281833649,0.024971218779683113,-0.022543855011463165,-0.012368900701403618,0.09838170558214188,0.13649925589561462,0.08115173876285553,0.059346143156290054,0.06989048421382904,0.1476197987794876,0.12858934700489044,0.019774815067648888,0.041959650814533234,0.028186727315187454,0.054102957248687744,0.10323546081781387,-0.036987245082855225,0.012792612425982952,0.01884157955646515,0.001934132189489901,0.15054692327976227,0.2394864708185196,0.1565459817647934,0.08841077983379364,0.07292782515287399,0.06623218953609467,0.09582604467868805,0.044246651232242584,0.07918752729892731,0.05239955335855484,-0.06365028023719788,-0.013259455561637878,0.0951823890209198,0.03639206662774086,0.0926935002207756,0.3019345700740814,0.2715761363506317,0.0770379826426506,-0.0016352860257029533,0.05890356004238129,0.0726437047123909,-0.028187811374664307,0.03506698086857796,3.189251583535224e-05,-0.05382553115487099,0.03285200148820877,0.05076448619365692,-0.03430536761879921,-0.08105410635471344,-0.011395142413675785,0.060883309692144394,-0.027909494936466217,-0.012712282128632069,0.09926994144916534,0.06158721074461937,0.03969753161072731,0.09456904232501984,0.004115679766982794,-0.09326936304569244,-0.05914970859885216,0.04252229258418083,0.0009042187011800706,-0.02265990898013115,-0.05404480919241905,-0.05983389541506767,-0.041976671665906906,0.01335145253688097,0.03295789286494255,0.06240218132734299,0.031313374638557434,0.023757781833410263,0.015756841748952866,-0.014778169803321362,-0.033535607159137726,0.038465097546577454,0.03765273094177246,0.02298293635249138,-0.04917578026652336,-0.11066711694002151,-0.10913398861885071,0.03767714276909828,0.08268408477306366,0.07169981300830841,-0.015955908223986626,0.029234936460852623,-0.07287473231554031,-0.06236816197633743,-0.012996570207178593,-0.09194714576005936,-0.10350129008293152,-0.11469671875238419,-0.10351058095693588,-0.1125837191939354,-0.059186894446611404,-0.06628728657960892,-0.03787098452448845,0.024746514856815338,-0.03584205359220505,-0.020784689113497734,0.005022238940000534,0.00254244077950716,0.0277085043489933,0.02965892292559147,-0.06950743496417999,-0.03441524878144264,-0.017712200060486794,-0.053147897124290466,-0.021153347566723824,-0.06291544437408447,0.060859862715005875,0.052517667412757874,0.0682072788476944,0.025931473821401596,0.13487809896469116,0.14045242965221405,0.12227918952703476,0.013394678011536598,-0.08434893935918808,-0.010228739120066166,-0.0498165600001812,-0.04936728999018669,0.014004320837557316,-0.024765484035015106,0.04441653564572334,0.05697282776236534,0.07146687060594559,0.1550629734992981,0.09523039311170578,0.07779537886381149,0.08424648642539978,0.061958666890859604,0.04058536887168884,0.027429411187767982,0.03381459787487984,-0.007687355857342482,-0.0007019672193564475,-0.013968565501272678,0.07833543419837952,0.09501603245735168,0.13109004497528076,0.07459121197462082,-0.06817330420017242,-0.06594492495059967,-0.10165513306856155,-0.06063384935259819,-0.1164185032248497,-0.12381540238857269,-0.07770288735628128,-0.11754986643791199,-0.1354026347398758,-0.2013969123363495,-0.11560261249542236,-0.13248290121555328,-0.027022739872336388,-0.03414617478847504,-0.15997812151908875,-0.18341265618801117,-0.08651726692914963,-0.06831581145524979,-0.197061687707901,-0.15029817819595337,-0.20079772174358368,-0.038302529603242874,-0.1330176591873169,-0.19923709332942963,-0.041238363832235336,-0.05299831181764603,-0.051401760429143906,-0.13548436760902405,-0.12298042327165604,-0.17918916046619415,-0.1739080846309662,-0.11025539040565491,-0.2645390033721924,-0.14471541345119476,-0.1999094933271408,0.022941023111343384,-0.13130216300487518,-0.15292145311832428,-0.1322857290506363,-0.09657842665910721,-0.07440442591905594,-0.15774258971214294,-0.054781265556812286,-0.030638590455055237,-0.13734517991542816,-0.10239429026842117,-0.28933265805244446,-0.06467545032501221,-0.04504377394914627,-0.0017185828182846308,-0.2045784890651703,-0.22719678282737732,-0.10513244569301605,-0.07840210199356079,-0.11099600046873093,-0.09825772047042847,-0.030837172642350197,-0.024291669949889183,-0.1002110093832016,-0.04875919595360756,-0.20133548974990845,-0.03509443253278732,-0.059274010360240936,0.09425809234380722,-0.1846161186695099,-0.13416525721549988,-0.09664767980575562,-0.09459477663040161,-0.07181208580732346,-0.07025128602981567,-0.20997701585292816,-0.2706204652786255,-0.10493184626102448,-0.1743534952402115,-0.35643142461776733,9.871251677395776e-05,-0.08529604226350784,-0.06932147592306137,-0.14136888086795807,-0.08632288128137589,-0.13125059008598328,-0.20671990513801575,-0.1137968897819519,-0.09195077419281006,-0.2159167379140854,-0.3390570282936096,-0.3551460802555084,-0.18705427646636963,0.010164588689804077,-0.07332700490951538,-0.1538255661725998,-0.22412168979644775,-0.2784331440925598,-0.29457229375839233,-0.24504604935646057,-0.1752299964427948,-0.0662594586610794,-0.08098994195461273,-0.0385141484439373,-0.24631927907466888,-0.44288718700408936,-0.5499217510223389,-0.16671378910541534,-0.15459971129894257,-0.19397962093353271,-0.08287720382213593,-0.1998414397239685,-0.27574312686920166,-0.17888981103897095,-0.21799004077911377,-0.16165323555469513,-0.1648811250925064,-0.12896014750003815,-0.3472146987915039,-0.39167520403862,-0.26897361874580383,-0.41300252079963684,-0.15367001295089722,-0.25029024481773376,-0.24734723567962646,0.030202478170394897,-0.1478167623281479,0.0739307776093483,0.03380439803004265,-0.08210822939872742,-0.0965690091252327,-0.1963040828704834,-0.35208383202552795,-0.15214283764362335,-0.11162732541561127,-0.2844376862049103,-0.11088936030864716,-0.39152902364730835,-0.3017458915710449,-0.026662632822990417,-0.17566132545471191,-0.08414377272129059,-0.0387326180934906,-0.08017710596323013,-0.10064243525266647,-0.14142049849033356,-0.15628831088542938,-0.1307668685913086,-0.029149191454052925,-0.18978063762187958,-0.038807377219200134,-0.23017559945583344,-0.32813164591789246,0.0026154934894293547,0.02350359596312046,-0.12247070670127869,-0.07963240146636963,-0.036590319126844406,-0.048841774463653564,-0.08597595244646072,-0.16395920515060425,-0.04518384113907814,0.028568750247359276,-0.17819373309612274,0.026589592918753624,-0.05668565258383751,-0.19102762639522552,-0.1487901359796524,-0.12004920095205307,-0.01788664236664772,-0.08265484869480133,0.022820668295025826,0.06470181792974472,-0.017613589763641357,-0.13874071836471558,-0.08893716335296631,-0.0029764261562377214,-0.1078762486577034,-0.07187581807374954,-0.051789540797472,-0.05536898225545883,-0.0778685212135315,-0.08580239862203598,0.05182894319295883,-0.01013637613505125,0.04668360948562622,0.22738298773765564,-0.12775661051273346,-0.15165553987026215,-0.10322711616754532,0.007838951423764229,-0.1153334453701973,-0.12661778926849365,-0.049151282757520676,0.018498435616493225,0.07078889012336731,-0.044519443064928055,0.08633404970169067,0.0892939418554306,0.09223711490631104,0.19332462549209595,0.14127211272716522,0.040127214044332504,0.10201501101255417,0.02595481649041176,0.04628678411245346,0.023112891241908073,0.1283455193042755,-0.020422382280230522,0.058640964329242706,0.05456647276878357,0.13367673754692078,0.2315516322851181,0.23683921992778778,0.33321160078048706,0.0049309455789625645,-0.007464004680514336,0.022968372330069542,0.10908825695514679,0.10799267888069153,0.056996725499629974,0.13095512986183167,0.08636079728603363,0.12263917922973633,0.07206984609365463,0.10659857094287872,0.22061669826507568,0.23628966510295868,0.13966462016105652,-0.07902287691831589,-0.1478191763162613,-0.01960841193795204,0.07816234230995178,0.04052925854921341,-0.00986422412097454,-0.016562988981604576,-0.05693931505084038,0.07090599089860916,-0.0013120602816343307,0.004217096138745546,0.00580588961020112,0.085908442735672,0.019147830083966255,-0.12952151894569397,-0.09732542932033539,0.1291934996843338,0.04594598710536957,-0.04186994582414627,-0.06313406676054001,0.013920494355261326,-0.020070267841219902,0.08367825299501419,-0.033874429762363434,-0.013743635267019272,-0.04326438531279564,-0.026890836656093597,0.008431276306509972,0.02425622195005417,0.12561219930648804,0.17500126361846924,0.016088873147964478,-0.09014878422021866,-0.1515226662158966,0.007986923679709435,-0.058620885014534,-0.04988805204629898,-0.1103215292096138,-0.045456402003765106,0.006368232890963554,0.03872000426054001,0.11137613654136658,0.07784149795770645,-0.02649710327386856,0.0056323157623410225,-0.10919123142957687,-0.19262738525867462,0.0452633872628212,0.13668479025363922,-0.12343373149633408,-0.11615784466266632,-0.12369071692228317,-0.03957498446106911,-0.08095694333314896,0.06752333790063858,0.08625219017267227,0.05590343847870827,-0.0010794984409585595,-0.09605563431978226,-0.11076895892620087,-0.028818346560001373,-0.008198031224310398,-0.004335531499236822,-0.029141563922166824,-0.06146285682916641,-0.15665072202682495,-0.11285675317049026,-0.064033143222332,-0.04278751090168953,0.11380758136510849,0.07141779363155365,0.04496102035045624,-0.2019396424293518,-0.2782047688961029,-0.13399513065814972,-0.16265985369682312,-0.0930076539516449,-0.09665030986070633,-0.16543905436992645,-0.13613182306289673,-0.09922727197408676,-0.08265475183725357,0.031486570835113525,-0.01463589072227478,0.03212051838636398,-0.10032262653112411,-0.18979397416114807,-0.12909092009067535,-0.06382286548614502,-0.1289568841457367,0.006189142819494009,-0.08053576201200485,0.031477902084589005,-0.06269870698451996,-0.024911638349294662,-7.556573837064207e-05,-0.027094021439552307,-0.0381971150636673,-0.0676642656326294,-0.11836712062358856,-0.11452726274728775,0.05752620846033096,-0.04299893602728844,-0.1253698468208313,-0.1388411968946457,-0.39481469988822937,-0.06510594487190247,0.005667565856128931,-0.05469721183180809,0.04823550954461098,-0.06977096945047379,-0.12648551166057587,-0.07837876677513123,-0.09633452445268631,-0.1261352002620697,0.006178664043545723,-0.07640010863542557,-0.01916632428765297,-0.10896231979131699,-0.3148185908794403,-0.07050551474094391,-0.017561886459589005,-0.12076273560523987,-0.1536691039800644,-0.12830883264541626,-0.04803575202822685,-0.08212592452764511,0.019000228494405746,-0.07721103727817535,-0.02009217068552971,-0.060355301946401596,0.01970933936536312,0.03369370102882385,-0.12767431139945984,-0.017087429761886597,-0.018474124372005463,-0.054772257804870605,-0.12280701845884323,-0.06367536634206772,0.008976951241493225,-0.06049526855349541,-0.030034704133868217,-0.08895925432443619,-0.027517171576619148,-0.053959958255290985,-0.06744129210710526,-0.036248113960027695,-0.14792639017105103,-0.13091400265693665,-0.01377965323626995,-0.04109375923871994,-0.03431212529540062,0.021732283756136894,0.053753532469272614,-0.0360892228782177,-0.06289052218198776,-0.10464152693748474,-0.05231154337525368,-0.06022609397768974,-0.04145551845431328,-0.12397311627864838,-0.07048247009515762,-0.024894652888178825,0.020686587318778038,0.0069410051219165325,-0.03094244748353958,0.13451188802719116,0.25778305530548096,0.10783834010362625,0.04002013057470322,0.010525023564696312,-0.032696694135665894,0.04003094509243965,0.02578369528055191,0.019082872197031975,0.04832302778959274,0.0423005148768425,-0.0017803128575906157,-0.05851316452026367,0.02506042644381523,0.0894487127661705,0.13399691879749298,0.06347539275884628,0.04174207150936127,0.0033756576012820005,-0.020646443590521812,-0.035954900085926056,-0.04969986528158188,0.030048059299588203,0.04905353859066963,-0.01512077171355486,-0.03248694911599159,-0.00021750219457317144,0.0013187532313168049,0.04763989523053169,0.05879547446966171,0.0812240019440651,-0.0007047203835099936,0.03593887761235237,-0.013280865736305714,0.06713972985744476,0.010287937708199024,0.06984882801771164,0.0403195358812809,0.0073755350895226,-0.04725890979170799,0.03266085311770439,0.00234811007976532,0.0533885732293129,0.09310322999954224,0.08932379633188248,-0.10078597068786621,-0.10080374032258987,0.06541222333908081,0.07116533815860748,0.023925257846713066,0.12445026636123657,0.09208488464355469,0.11936560273170471,-0.001688470016233623,-0.06938861310482025,-0.055258478969335556,-0.029167260974645615,0.015095517039299011,0.06263916939496994,-0.022777318954467773,-0.02483028545975685,0.01952512562274933,0.137480229139328,0.04479382932186127,0.14017796516418457,0.1382496953010559,0.14019975066184998,0.05388214439153671,-0.062361251562833786,-0.0008811096777208149,0.006060580722987652,0.013491186313331127,0.05444738641381264,-0.0633201077580452,-0.007955960929393768,-0.02777971886098385,0.06192091107368469,0.09137126803398132,0.20706863701343536,0.18230512738227844,0.06901131570339203,0.0140980314463377,0.08353830128908157,-0.0053905313834548,-0.04320180416107178,-0.03500629961490631,-0.05421588197350502,-0.12659890949726105,-0.028219478204846382,0.0031649209558963776,-0.031451158225536346,0.005950588267296553,0.14257694780826569,0.10201280564069748,0.015013090334832668,-0.011141346767544746,0.0019576656632125378,0.05603120103478432,-0.06640090048313141,-0.06083974987268448,-0.04363500326871872,-0.06074201315641403,-0.028439024463295937,0.006982418242841959,-0.08210968971252441,-0.03039073571562767,0.006892127450555563,-0.015665756538510323,-0.04571627080440521,-0.02363007701933384,-0.007967459037899971,0.01809072308242321,-0.035918429493904114,-0.05517292022705078,-0.024972163140773773,-0.11271025985479355,-0.03430115804076195,-0.05520745366811752,-0.05957421660423279,-0.11412828415632248,-0.08890420198440552,-0.08468365669250488,-0.06373772770166397,0.01620713621377945,-0.01287376880645752,-0.022765733301639557,-0.06646009534597397,-0.04142431914806366,-0.013187100179493427,-0.1217026486992836,-0.04477306455373764,0.03257273510098457,-0.003394113853573799,-0.05332614853978157,-0.07994045317173004,-0.10747949779033661,-0.0954328328371048,0.0330527126789093,0.05047569051384926,0.09350014477968216,-0.027775747701525688,-0.01866048015654087,-0.01142337266355753,-0.051892563700675964,0.06605826318264008,-0.03762556612491608,-0.08172819018363953,-0.05164126679301262,-0.028616242110729218,-0.03432965651154518,-0.004441929515451193,-0.07496620714664459,0.003190849907696247,0.08416371792554855,-0.023394664749503136,-0.02302376739680767,0.1078074648976326,0.11264459043741226,0.05406007170677185,-0.043671850115060806,-0.05621881037950516,0.040995653718709946,0.10381290316581726,0.09023261070251465,0.02768966369330883,-0.0055305142886936665,0.031406059861183167,0.08198551833629608,0.07594850659370422,0.03324339538812637,0.17911653220653534,0.16930124163627625,0.07656104862689972,0.06408832967281342,0.02269570901989937,0.09427835792303085,-0.01689068414270878,0.001882868935354054,0.0560716837644577,0.04314282909035683,0.08171790838241577,0.04005908966064453,0.13714417815208435,0.10146763920783997,0.16630946099758148,0.09632185101509094,0.066755510866642,0.11980771273374557,0.02227458730340004,0.05281858891248703,-0.017054010182619095,0.05735541880130768,0.062260862439870834,0.09985209256410599,0.08613407611846924,0.10921776294708252,0.10643380135297775,0.0829271525144577,0.0506586879491806,-0.011171499267220497,0.02750719152390957,0.020908063277602196,0.02916371449828148,0.003769158385694027,0.031960178166627884,-0.11043216288089752,-0.1589571088552475,-0.012499305419623852,0.022334158420562744,0.01827458292245865,0.015999166294932365,-0.05995064228773117,0.02908363938331604,0.0032199283596128225,0.033110689371824265,0.0812411904335022,0.04941609129309654,0.05863342434167862,0.09897741675376892,0.08872628957033157,-0.007886325940489769,-0.009366023354232311,0.03930536285042763,-0.01162753812968731,-0.05201949551701546,-0.013968138955533504,0.03905787318944931,0.06651878356933594,0.09541875869035721,0.14250360429286957,0.13449740409851074,0.08735191076993942,0.17636480927467346,0.14099641144275665,0.0588761605322361,0.11412446200847626,0.08745173364877701,0.10926760733127594,0.009000349789857864,0.012720489874482155,-0.029491694644093513,0.022559380158782005,0.03592207282781601,-0.005257474724203348,0.0723133385181427,0.046670183539390564,0.10430026054382324,0.0994424968957901,0.02304275706410408,0.04080420359969139,0.012313575483858585,-0.003912862855941057,0.006323855835944414,-0.012284272350370884,-0.045856181532144547,-0.05603473260998726,-0.034013938158750534,-0.07431671768426895,-0.14840912818908691,0.06505908071994781,0.07142020016908646,0.030074525624513626,-0.0014716997975483537,0.021427873522043228,-0.014161690138280392,-0.0938192680478096,-0.021828262135386467,-0.029772130772471428,-0.03903903439640999,-0.09173687547445297,-0.012224936857819557,-0.003140929387882352,0.08387614786624908,0.05349390208721161,0.021828019991517067,-0.0329684279859066,0.0012589020188897848,-0.025422751903533936,-0.048330701887607574,-0.03144542872905731,-0.019380783662199974,0.0007927980041131377,-0.008754351176321507,0.14306847751140594,0.10918160527944565,0.059023208916187286,0.025767695158720016,-0.13151785731315613,-0.13704562187194824,-0.038759104907512665,-0.11234867572784424,-0.004595005419105291,0.051656324416399,-0.05265551060438156,-0.05195467919111252,-0.0712747797369957,0.08379305154085159,0.058859508484601974,0.017035074532032013,0.07659420371055603,-0.09444654732942581,-0.08409388363361359,0.07097683846950531,-0.0910191461443901,0.0076511697843670845,-0.08446431159973145,-0.044447556138038635,0.02417483553290367,0.06127617135643959,0.04761116951704025,-0.0227068942040205,0.04746980592608452,0.11494410037994385,0.03814693167805672,0.019291207194328308,0.079208143055439,0.01917133294045925,0.020455028861761093,-0.03336817026138306,0.01312357746064663,0.05615485459566116,0.0721399188041687,-0.02842850424349308,-0.04833123832941055,0.031040290370583534,0.08106217533349991,0.05341598764061928,0.04312684386968613,-0.04042477160692215,0.11221620440483093,0.062307074666023254,0.05168234929442406,0.10041676461696625,0.07912863045930862,0.04051527380943298,-0.012405072338879108,0.01752222329378128,0.01518828421831131,0.006365072913467884,0.020140202715992928,-0.03635239973664284,-0.0992407575249672,0.02033005841076374,-0.0690365731716156,0.03650587424635887,0.015840206295251846,0.021746370941400528,-0.07316488772630692,-0.0023182248696684837,-0.03233183175325394,-0.03125213459134102,-0.07514423131942749,-0.07050655782222748,-0.079039566218853,0.03190461918711662,-0.03040175326168537,-0.021530263125896454,0.010878542438149452,0.02331553027033806,-0.035097651183605194,-0.006944781169295311,-0.01857680454850197,0.06046168878674507,0.009607571177184582,-8.623002213425934e-05,-0.12202484905719757,-0.08510901778936386,-0.08219403028488159,-0.021709702908992767,0.06309502571821213,0.020527685061097145,6.590000702999532e-05,0.06469578295946121,-0.013896721415221691,-0.0044593424536287785,0.05552398040890694,-0.016737839207053185,-0.03533022105693817,-0.03610262647271156,-0.11416824907064438,-0.0424271896481514,0.02553699165582657,-0.05599575862288475,0.059087008237838745,0.05050374194979668,0.11442261934280396,0.07338332384824753,0.027230003848671913,0.05676914006471634,0.07144037634134293,0.073576420545578,0.012905944138765335,0.02571478858590126,0.0341535247862339,-0.43513569235801697,-0.3409757912158966,-0.27819958329200745,-0.2103528529405594,-0.1874486654996872,-0.12066055834293365,-0.10665130615234375,-0.105707548558712,-0.08888965845108032,-0.10624222457408905,-0.13351228833198547,-0.20181135833263397,-0.13271549344062805,-0.12037434428930283,-0.30773937702178955,-0.20450535416603088,-0.1044783666729927,-0.031105590984225273,-0.037258751690387726,-0.034502290189266205,-0.004204206168651581,-0.015475044958293438,-0.055514153093099594,-0.11415351182222366,-0.029250364750623703,-0.04740995913743973,-0.014045396819710732,0.035048358142375946,-0.23104950785636902,-0.15676850080490112,-0.11836066842079163,0.026526352390646935,-0.00844480749219656,0.008807515725493431,-0.027734333649277687,-0.04307528957724571,-0.06484773010015488,-0.01753990724682808,0.00942580308765173,0.03582637012004852,0.012959678657352924,0.060497768223285675,-0.19556473195552826,-0.08096746355295181,-0.04206525906920433,-0.006752680987119675,0.021814044564962387,0.024372423067688942,-0.08899896591901779,-0.15252402424812317,0.01527829747647047,0.022989055141806602,0.06830684095621109,0.029264744371175766,0.05698401853442192,-0.047442249953746796,-0.08316683769226074,-0.044695354998111725,-0.02214251272380352,0.0404370091855526,0.036426737904548645,-0.044201865792274475,-0.05018767714500427,-0.14104899764060974,-0.015996774658560753,0.13023358583450317,0.023593150079250336,0.03672052547335625,0.06677071750164032,0.03614342585206032,-0.01845093071460724,-0.04091867431998253,0.021331408992409706,0.07936760783195496,0.08359586447477341,0.021613121032714844,-0.01187866274267435,-0.10106708109378815,0.06406456977128983,0.197200208902359,0.04840213432908058,0.023784149438142776,0.059957247227430344,0.052803345024585724,0.024788696318864822,-0.05395432934165001,0.054281577467918396,0.08232451975345612,0.11271848529577255,0.010687599889934063,-0.09272624552249908,-0.01448194868862629,0.014105098322033882,0.13157425820827484,0.10022715479135513,0.015225531533360481,-0.03029908984899521,-0.003938484936952591,0.06298664957284927,0.06633268296718597,-0.004605018068104982,0.13997238874435425,0.13140805065631866,0.015076414681971073,-0.04999601095914841,-0.14651307463645935,0.04005623236298561,0.04994526132941246,0.06339112669229507,0.01584109477698803,0.02797548472881317,0.011278126388788223,0.02618958055973053,0.016290683299303055,0.0015984452329576015,0.0335770919919014,0.15429815649986267,0.09277531504631042,0.023266896605491638,-0.03043215721845627,-0.005637829191982746,0.07151439040899277,-0.05458836629986763,-0.027999721467494965,0.02967650815844536,-0.033705588430166245,0.11736402660608292,0.024111023172736168,0.12340783327817917,0.07276690751314163,0.12546300888061523,0.11942952126264572,0.023986825719475746,0.035839006304740906,0.08011768013238907,-0.051593195647001266,-0.12513422966003418,-0.027941983193159103,-0.018542829900979996,0.007116727065294981,0.18233734369277954,0.013628331013023853,0.06512244045734406,0.017697898671030998,0.09367794543504715,-0.018462615087628365,0.03626163303852081,-0.03204875811934471,-0.034445248544216156,-0.062328457832336426,-0.10611002147197723,0.05060422793030739,0.06419111043214798,0.037182506173849106,0.15318059921264648,0.02638143114745617,-0.007280487101525068,0.07065087556838989,0.04341183602809906,-0.03528394177556038,0.029133612290024757,-0.013713532127439976,-0.017636803910136223,-0.003566680708900094,-0.02652982622385025,0.057866305112838745,-0.016466794535517693,0.06589943915605545,0.09239337593317032,-0.01670677214860916,0.06488977372646332,0.086308553814888,0.050177641212940216,0.10572562366724014,0.018631689250469208,0.03182300925254822,0.00921707134693861,-0.018948625773191452,0.0028280906844884157,0.08105133473873138,0.014559156261384487,0.02627418003976345,0.11919765919446945,0.0807972252368927,0.07950646430253983,0.003763986751437187,-0.0031991794239729643,0.06758159399032593,0.015029762871563435,0.0006064417539164424,0.0652138814330101,-0.013544856570661068,-0.021852824836969376,-0.04554208368062973,0.011010663583874702,0.07752250880002975,-0.0034412227105349302,0.03101683221757412,0.01408680621534586,0.02931155636906624,0.026374440640211105,-0.009727034717798233,-0.11772165447473526,-0.14949631690979004,-0.06976113468408585,-0.1225171685218811,-0.2407686859369278,-0.2763742506504059,-0.22190725803375244,-0.25297781825065613,-0.03053640015423298,-0.021441712975502014,0.003781111678108573,-0.0706089437007904,-0.11232395470142365,0.027433885261416435,-0.02823111042380333,-0.007058382499963045,-0.0034472269471734762,-0.011951717548072338,-0.169648215174675,-0.20125672221183777,-0.20780865848064423,-0.24487370252609253,-0.05790575221180916,0.11285698413848877,0.07895778119564056,-0.08015695214271545,-0.09099240601062775,-0.053699690848588943,0.0866425484418869,-0.11247604340314865,0.04173697158694267,-0.03039431758224964,-0.046047430485486984,-0.05038025230169296,-0.2230658084154129,-0.21717768907546997,0.041442904621362686,0.04229942336678505,0.08401839435100555,-0.006732300389558077,0.034710776060819626,-0.13562023639678955,0.06429973244667053,-0.23849642276763916,-0.16464368999004364,0.08067820966243744,0.09143702685832977,-0.004981383215636015,-0.031007271260023117,-0.09881776571273804,0.0839742049574852,0.12643317878246307,0.09834860265254974,0.11719968169927597,0.13643909990787506,0.053341642022132874,0.1446974128484726,-0.17741568386554718,-0.08734637498855591,0.05956112965941429,0.15348291397094727,0.1238267570734024,0.022844383493065834,-0.016244754195213318,0.0009028481435962021,0.05988188087940216,-0.04516270011663437,0.010111955925822258,-0.01393128465861082,0.11508093774318695,-0.2301226407289505,-0.1907554268836975,-0.02028600685298443,-0.04901125654578209,-0.10373473167419434,0.054549675434827805,-0.07909366488456726,-0.05909047648310661,-0.0005936482921242714,-0.034396685659885406,-0.019060859456658363,-0.040533509105443954,0.1312975287437439,0.04247955232858658,-0.3393777310848236,-0.33924660086631775,-0.14668157696723938,-0.0201578252017498,-0.04969407618045807,0.06317634135484695,0.13803686201572418,0.09048093855381012,-0.09153391420841217,-0.048818543553352356,-0.05283258482813835,0.026303313672542572,0.0752851590514183,-0.018706655129790306,-0.22054140269756317,0.021336136385798454,-0.07754561305046082,0.0723498985171318,0.04772571474313736,0.1561712771654129,0.06675069034099579,0.01088305190205574,-0.021342961117625237,-0.1706704944372177,-0.20648975670337677,-0.11195484548807144,0.0312417671084404,0.12625685334205627,-0.10746479034423828,0.1941068321466446,0.16328327357769012,-0.06577276438474655,0.0269861351698637,0.08565185219049454,0.09767067432403564,0.0695648342370987,-0.0529957041144371,-0.039128292351961136,-0.2035478800535202,-0.20576469600200653,0.07944315671920776,0.25423210859298706,0.23404286801815033,0.10486441850662231,0.20193961262702942,0.03121182695031166,-0.12712730467319489,-0.06249736249446869,0.026546137407422066,0.14200074970722198,-0.22260743379592896,-0.041440874338150024,-0.13318228721618652,-0.049291759729385376,0.1969897747039795,0.12827235460281372,0.21901758015155792,0.22451463341712952,0.06262894719839096,0.22148114442825317,0.045501310378313065,-0.031226258724927902,-0.027832381427288055,-0.013952110894024372,-0.31733012199401855,-0.14484624564647675,-0.14529453217983246,0.00735821295529604,0.05709337070584297,0.036708515137434006,0.07277834415435791,-0.045885682106018066,0.10155244171619415,0.1204647496342659,0.05905856937170029,-0.0387607142329216,-0.06324446201324463,0.004806953482329845,-0.3640116751194,-0.21538381278514862,-0.18595632910728455,-0.04945354163646698,-0.06247568130493164,-0.06220988184213638,0.13390257954597473,0.06345608085393906,0.10406146198511124,0.012348526157438755,0.04299648106098175,0.04799390584230423,-0.024555634707212448,-0.024680275470018387,-0.37369614839553833,-0.36396706104278564,-0.26458775997161865,-0.16233687102794647,-0.12246339023113251,-0.0577269122004509,0.1444353461265564,0.04347597807645798,-0.039827991276979446,0.03219984844326973,-0.10421186685562134,0.045978132635354996,-0.09819871932268143,-0.26804831624031067,-0.06656613945960999,0.014982135966420174,-0.05714665353298187,5.678408342646435e-05,0.1741514801979065,0.1622343510389328,0.16895389556884766,0.1942494809627533,0.2325562834739685,0.11756221204996109,0.22307170927524567,0.15232549607753754,0.22171646356582642,0.21538904309272766,0.037326257675886154,0.10597658157348633,0.006294249556958675,0.05323292315006256,0.183821439743042,0.1728372722864151,0.22470450401306152,0.1632087230682373,0.19277316331863403,0.06750970333814621,0.10315869003534317,0.10208191722631454,0.2060333490371704,0.24897393584251404,0.10037576407194138,0.12807263433933258,0.17867660522460938,0.08298678696155548,0.10700628906488419,0.13524289429187775,0.2612687051296234,0.15311077237129211,0.09469380229711533,0.0391322560608387,0.18827436864376068,0.19882641732692719,0.17007392644882202,0.2155890315771103,-0.07987754791975021,0.03724803775548935,0.10112608969211578,0.13970036804676056,0.2068031132221222,0.20371438562870026,0.22655630111694336,0.1528039127588272,0.1242014467716217,0.16134633123874664,0.12320005893707275,0.024863239377737045,0.03694340959191322,0.10529559850692749,-0.1121520921587944,-0.12331493198871613,-0.09273075312376022,-0.011256632395088673,0.01831170544028282,0.06694987416267395,0.26401782035827637,0.1939956396818161,-0.01274792104959488,-0.02564147487282753,0.03536708652973175,0.05975155904889107,-0.016118723899126053,-0.05977971851825714,-0.033718787133693695,0.08861332386732101,0.035481005907058716,0.13179610669612885,0.12916164100170135,0.07985082268714905,0.14239633083343506,0.10706260800361633,0.18796943128108978,0.04343721270561218,-0.04378089681267738,0.06886874884366989,0.0889604389667511,0.014106577262282372,0.15406420826911926,0.12054506689310074,0.054975658655166626,-0.12235914170742035,0.04771757870912552,0.044290635734796524,-0.0391882061958313,0.12515120208263397,0.14834153652191162,0.0972091481089592,-0.08631385117769241,-0.008071782998740673,0.08513541519641876,-0.0023018987849354744,0.13077571988105774,0.0812353566288948,0.08147217333316803,0.09776932001113892,0.16802045702934265,0.06882641464471817,0.17994078993797302,0.06962613016366959,0.18198420107364655,-0.0048219747841358185,-0.06656698882579803,0.07272334396839142,0.04542560130357742,-0.019262447953224182,0.02069375291466713,0.11628737300634384,0.06777533143758774,0.13900962471961975,0.2171887457370758,0.13036057353019714,0.16955173015594482,0.19576440751552582,0.03358067199587822,-0.12718570232391357,-0.1536647081375122,-0.14980702102184296,-0.05208231881260872,-0.026216348633170128,0.07432302832603455,0.1675986796617508,0.055341582745313644,0.1878196895122528,0.32093244791030884,0.47990643978118896,0.5248616337776184,0.3981805145740509,0.107874795794487,-0.03550867363810539,-0.055078260600566864,-0.12388822436332703,-0.020089156925678253,-0.061513613909482956,0.23768998682498932,0.10259608179330826,0.08335200697183609,0.21851395070552826,0.22619199752807617,0.2568913996219635,0.5004333853721619,0.22911787033081055,0.03956454247236252,-0.06652176380157471,-0.05046992003917694,0.0013429868267849088,0.0652124434709549,-0.05495067685842514,0.24080485105514526,0.19163662195205688,0.09657678753137589,0.12771737575531006,0.11740744113922119,0.14921674132347107,0.2976384460926056,0.08129554241895676,0.13022764027118683,0.06802555173635483,0.08862000703811646,-0.0063010407611727715,-0.06653694063425064,-0.09650508314371109,0.21613620221614838,0.10371001809835434,0.18503877520561218,0.12323387712240219,0.1930481493473053,0.1979856640100479,0.1526683121919632,0.16339237987995148,0.1130097359418869,0.14258888363838196,0.15958474576473236,0.008267142809927464,0.02871774695813656,0.04563013091683388,0.2499634176492691,0.18577750027179718,0.25964751839637756,0.1527259200811386,0.22070160508155823,0.22635169327259064,0.14669634401798248,0.1118316724896431,0.07535648345947266,0.1741691529750824,0.16613195836544037,0.128581240773201,0.07351775467395782,0.011532995849847794,-0.30590739846229553,-0.2742704153060913,-0.209222212433815,-0.24939973652362823,-0.20530787110328674,-0.19666112959384918,-0.10251989215612411,0.012936798855662346,0.009761007502675056,-0.03934098407626152,-0.09713339060544968,-0.19531704485416412,-0.20531277358531952,-0.11894906312227249,-0.12880544364452362,-0.22003929316997528,-0.14583319425582886,-0.1009935736656189,-0.11051096767187119,-0.08604837208986282,-0.06315550208091736,-0.007398458197712898,-0.08694035559892654,-0.016412153840065002,-0.046700287610292435,-0.034360241144895554,-0.07929583638906479,-0.017074687406420708,-0.07532893866300583,-0.20383618772029877,-0.07461071759462357,-0.0113566555082798,-0.0265217125415802,-0.06189551204442978,-0.035272084176540375,0.03032681904733181,0.0074827601201832294,0.0059785014018416405,0.04958450794219971,0.026055924594402313,0.033063244074583054,0.04570460319519043,-0.10885610431432724,-0.12865562736988068,-0.05051605403423309,-0.027460923418402672,-0.028923574835062027,-0.14224877953529358,-0.024474285542964935,0.06676965951919556,0.043458059430122375,-0.0037379127461463213,-0.0056477137841284275,-0.04906390979886055,0.022528087720274925,0.06879843771457672,-0.08131495118141174,-0.04657716676592827,-0.022762738168239594,-0.02798929624259472,0.03985993191599846,-0.1151895523071289,-0.07445329427719116,0.09760349243879318,0.14003418385982513,0.11073563247919083,0.13776665925979614,0.043683070689439774,-0.030420735478401184,0.008183286525309086,-0.016486436128616333,-0.020127493888139725,0.07111937552690506,0.0035261644516140223,-0.03796982765197754,-0.037207525223493576,0.01939777471125126,0.18234027922153473,0.25865408778190613,0.05954083055257797,0.048010408878326416,0.05376582592725754,0.009551392868161201,-0.013652603141963482,-0.003592796390876174,0.028611063957214355,0.0523320697247982,-0.05604106932878494,0.10102587193250656,0.10502177476882935,0.07381244003772736,0.207882359623909,0.30350959300994873,0.22059261798858643,0.08706080168485641,0.011660407297313213,-0.04218478128314018,-0.07443417608737946,0.05829888582229614,0.037017714232206345,0.15466007590293884,0.13046714663505554,0.06640410423278809,0.04054504632949829,-0.13209357857704163,0.06980996578931808,0.23567499220371246,0.12665927410125732,0.018367648124694824,0.048710502684116364,0.0026667211204767227,0.008286850526928902,0.01110323891043663,0.009856889955699444,0.11564353853464127,0.14925435185432434,0.1124720349907875,0.06089158356189728,-0.06765104830265045,0.09499300271272659,0.013830544427037239,0.04813676327466965,-0.0005600651493296027,0.052304454147815704,0.09274710714817047,0.033719681203365326,0.054762568324804306,0.10950528085231781,-0.05804218351840973,0.06071333587169647,0.07898522168397903,0.054628707468509674,-0.08767234534025192,-0.008084997534751892,-0.03693201020359993,-0.0017101761186495423,-0.05038099363446236,0.005178861785680056,0.0776657983660698,0.043958067893981934,0.09430739283561707,0.08418722450733185,-0.0642714649438858,-0.0763421431183815,0.04302078112959862,-0.03000321425497532,0.07220602035522461,0.06250506639480591,0.042349815368652344,-0.08356007188558578,0.01288413256406784,0.053038451820611954,0.03362441807985306,-0.06779274344444275,0.10343898087739944,0.0957685187458992,-0.04182121902704239,-0.0025683934800326824,0.02945500798523426,-0.11567988991737366,-0.010232305154204369,-0.03439214453101158,-0.05915350094437599,-0.029812680557370186,-0.061501361429691315,-0.054438985884189606,3.2768002711236477e-06,-0.09138669073581696,0.07247593253850937,0.017667418345808983,0.012898486107587814,0.013049842789769173,0.001986121293157339,-0.1298358291387558,0.034100696444511414,-0.03282398730516434,-0.05906334146857262,-0.04630660638213158,-0.02064567059278488,0.09952840954065323,-0.02671952173113823,-0.0208851657807827,0.08844486624002457,0.05246138572692871,-0.03746601939201355,0.00812471192330122,0.008766073733568192,-0.14398059248924255,-0.04508073255419731,0.00926669780164957,-0.01829368807375431,0.06404317915439606,-0.013846103101968765,0.1334826499223709,0.09491226822137833,0.06825055927038193,-0.03500529006123543,-0.001851457986049354,-0.029620477929711342,-0.03036801889538765,0.006598050706088543,0.021934958174824715,-0.09378646314144135,-0.0006914365221746266,-0.0637531727552414,-0.0018517542630434036,0.0025172566529363394,0.04196241497993469,-0.0087280860170722,-0.011956465430557728,-0.03790583461523056,0.01737394370138645,0.028010694310069084,-0.023476196452975273,-0.01317768543958664,-0.00745344627648592,-0.04729907587170601,-0.06445525586605072,-0.03421233221888542,0.01968773454427719,0.003228067187592387,-0.050654541701078415,-0.004700213670730591,0.005889037624001503,0.01931465044617653,0.031769901514053345,-0.0033427048474550247,0.05793770030140877,-0.02687249518930912,0.002657201373949647,-0.06843647360801697,-0.04777095839381218,-0.024471312761306763,-0.02497393824160099,-0.07633616030216217,-0.03728688135743141,-0.030460495501756668,0.0019988438580185175,0.04743492230772972,0.03224775195121765,-0.03153963014483452,0.004882441367954016,-0.020663773640990257,-0.0735701322555542,-0.10409945249557495,0.008314138278365135,-0.03947821259498596,0.014152864925563335,-0.0068558333441615105,-0.042677778750658035,-0.006353733595460653,0.051109109073877335,-0.017689665779471397,0.0647440254688263,0.052864920347929,0.05351926013827324,0.047233130782842636,-0.011668887920677662,-0.045051299035549164,-0.05587298050522804,0.03662131354212761,0.021615950390696526,-0.018333593383431435,-0.030640296638011932,0.028864607214927673,-0.009219471365213394,0.014392508193850517,-0.018297104164958,0.07793737202882767,0.07636340707540512,-0.002222006442025304,-0.015143314376473427,-0.08344551175832748,0.00908847339451313,0.015270287171006203,0.01123217586427927,0.03412995487451553,0.030852412804961205,0.06270897388458252,0.057029157876968384,0.09397917985916138,0.03667130321264267,0.045171692967414856,0.1302870661020279,-0.00033543165773153305,-0.09240838885307312,-0.07213608920574188,0.07790341973304749,0.05960423871874809,-0.03059198334813118,-0.0053437259048223495,-0.00800735130906105,0.0023221857845783234,0.05772151052951813,0.004965296480804682,0.008900395594537258,-0.016622578725218773,0.008416627533733845,0.0374145582318306,-0.11012865602970123,-0.10586202144622803,0.006681216415017843,0.0630921944975853,0.08559216558933258,-0.006960246246308088,0.03096277080476284,-0.045539725571870804,0.016857964918017387,0.02822033502161503,0.031037842854857445,-0.001262365491129458,0.07238053530454636,0.01889689825475216,0.004180689807981253,-0.03882766515016556,0.036180343478918076,0.06481048464775085,0.030668454244732857,0.006683842279016972,-0.06011355668306351,0.04103078693151474,0.05722059682011604,0.04490220174193382,0.0007402477785944939,0.020980339497327805,0.07256519794464111,0.02663537487387657,-0.05012645572423935,-0.030800238251686096,0.027836874127388,0.03381408378481865,-0.0034927413798868656,0.03850211203098297,0.03532687574625015,0.059343863278627396,-0.050333838909864426,-0.008760310709476471,0.043823838233947754,0.004913858603686094,0.033808279782533646,0.012276108376681805,-0.050269968807697296,-0.05844948813319206,-0.007501347456127405,-0.02956368401646614,0.034743644297122955,0.09938468784093857,0.00965062901377678,0.0465344600379467,-0.03442396968603134,-0.030500994995236397,0.015064206905663013,-0.009953868575394154,0.01326422393321991,0.00911813322454691,0.0013274707598611712,-0.072400763630867,-0.022393308579921722,-0.014269915409386158,-0.006228750105947256,0.07079126685857773,0.010703038424253464,0.025979241356253624,-0.01653745397925377,-0.07051956653594971,-0.018865792080760002,-0.01732706092298031,-0.04996843263506889,0.013901758939027786,-0.0407845713198185,-0.03446364775300026,-0.07493562251329422,0.02762707695364952,-0.0516299270093441,-0.0594586618244648,-0.051545146852731705,0.007539601065218449,-0.07796865701675415,-0.09695042669773102,-0.06826164573431015,-0.037690419703722,-0.12605857849121094,-0.08401896804571152,-0.07485238462686539,-0.16594578325748444,-0.15453815460205078,-0.11664660274982452,-0.11600788682699203,-0.09365470707416534,-0.046768102794885635,-0.043081093579530716,-0.1170351505279541,0.03709935396909714,-0.013286948204040527,-0.031228920444846153,-0.08046190440654755,-0.012726091779768467,-0.0013247685274109244,-0.04990042373538017,-0.051827140152454376,-0.07980861514806747,-0.027593303471803665,-0.11790972948074341,-0.06110627204179764,-0.07559914886951447,0.024772167205810547,0.00981025118380785,0.014012068510055542,-0.03538714721798897,-0.10030173510313034,-0.04343908280134201,-0.05051463469862938,-0.026378076523542404,-0.019861359149217606,0.0156157361343503,-0.03876263648271561,-0.03289183974266052,-0.0781073272228241,-0.011247649788856506,0.004676154348999262,-0.05339399725198746,-0.0602559931576252,-0.09642377495765686,-0.15334032475948334,-0.06349494308233261,-0.04008166491985321,0.002876921556890011,-0.03273097798228264,-0.016583964228630066,-0.11370622366666794,-0.05667426064610481,-0.03476376459002495,-0.01947064697742462,-0.015416801907122135,-0.09048818796873093,-0.10136669129133224,-0.05798175558447838,-0.03525227680802345,-0.10606171190738678,-0.04362976923584938,-0.07886654883623123,-0.0653492733836174,-0.0580778494477272,-0.0414033867418766,-0.04417707771062851,-0.0022313359659165144,0.00490166200324893,-0.07748857140541077,-0.013275434263050556,-0.09169376641511917,-0.06660407781600952,-0.11127113550901413,-0.08512533456087112,-0.03487595170736313,-0.032367199659347534,-0.011081991717219353,0.04584358632564545,-0.07387230545282364,-0.05585557967424393,-0.06540383398532867,0.03177740052342415,-0.014189486391842365,-0.0015010983915999532,-0.10009118914604187,-0.014894145540893078,0.002205470111221075,-0.007035042159259319,-0.01249659899622202,0.07282418012619019,0.08072086423635483,0.08911529183387756,0.07630066573619843,-0.010880810208618641,-0.05693468824028969,-0.04820922017097473,0.018768416717648506,0.010826442390680313,-0.019484711810946465,-0.02048448659479618,0.0035656108520925045,-0.0851564034819603,-0.10769981145858765,-0.041277118027210236,0.033692654222249985,-0.017621813341975212,0.04180451110005379,0.0015056263655424118,0.07773774117231369,0.03937038779258728,0.021394575014710426,-0.003239925717934966,-0.06359061598777771,0.03550046309828758,-0.048888932913541794,-0.014741480350494385,0.042488984763622284,0.15104267001152039,0.08410560339689255,0.06306158751249313,0.03577158972620964,0.061505433171987534,0.07828832417726517,-0.020370889455080032,-0.011357548646628857,0.10080346465110779,0.05760824307799339,0.012483136728405952,0.04526686295866966,0.0349339060485363,-0.023108908906579018,0.028273524716496468,0.028543800115585327,0.005929775070399046,-0.03392721340060234,0.06822694838047028,0.08316267281770706,0.05534583702683449,0.0476270392537117,0.07201474159955978,0.030866973102092743,0.06084686517715454,0.05394866317510605,-0.018130717799067497,0.020496290177106857,0.07282041013240814,0.029749684035778046,-0.04243633523583412,-0.03202023357152939,0.06950339674949646,0.07259944081306458,0.09863179922103882,0.025618944317102432,0.05178912729024887,0.046536147594451904,0.012937970459461212,0.0570988729596138,0.029642676934599876,-0.08711954206228256,-0.07648930698633194,-0.07390325516462326,-0.0285494327545166,-0.00023612547374796122,0.06510656327009201,0.05141375958919525,0.029897533357143402,0.0037319241091609,0.12918642163276672,0.020099680870771408,-0.026156719774007797,-0.006766894832253456,-0.03388902544975281,0.02222546935081482,-0.002932299394160509,0.02841852605342865,0.03426671400666237,0.01281986478716135,0.07454406470060349,0.09710297733545303,0.00037155477912165225,0.033127881586551666,0.08130285143852234,0.1045263260602951,-0.0031892277766019106,-0.04113852232694626,-0.031430020928382874,-0.04524049907922745,-0.00031289528124034405,0.0005069906474091113,0.027771063148975372,-0.06056799367070198,-0.027205290272831917,-0.013681615702807903,0.008104238659143448,0.04438866302371025,0.12538768351078033,0.0712745413184166,0.014003745280206203,0.0023042717948555946,-0.07228817790746689,-0.0767294242978096,-0.04580796882510185,-0.09057300537824631,-0.045516058802604675,-0.0882965475320816,0.02103310264647007,0.03185199201107025,-0.0049441419541835785,-0.0424557588994503,-0.36588796973228455,-0.33857446908950806,-0.26027053594589233,-0.22429940104484558,-0.11752908676862717,-0.08020319044589996,-0.052443359047174454,0.019752580672502518,0.07178216427564621,-0.07569677382707596,-0.07427146285772324,-0.07607070356607437,0.01350756548345089,0.030596155673265457,-0.16681022942066193,-0.11950330436229706,-0.14218276739120483,-0.1018720492720604,-0.08187510073184967,0.05223739147186279,0.1271377056837082,0.10542852431535721,0.1238936260342598,-0.02464427426457405,0.06256842613220215,0.05385517701506615,0.03788751736283302,0.11713851988315582,-0.18340353667736053,-0.09806162863969803,-0.14176487922668457,-0.11418084800243378,-0.06433044373989105,-0.07111069560050964,0.04309021309018135,0.06716130673885345,-0.011120913550257683,-0.04837850481271744,0.06030844524502754,0.11271750181913376,0.10735546052455902,0.03563060238957405,-0.16048327088356018,-0.08489659428596497,-0.07981913536787033,-0.12417688220739365,-0.0997631624341011,0.021716978400945663,-0.10089392215013504,-0.02052196115255356,0.00764587614685297,0.010492324829101562,0.10868130624294281,0.053987354040145874,0.060159169137477875,0.021565793082118034,-0.1246042400598526,-0.09232179075479507,-0.1624094694852829,-0.058769483119249344,-0.12849083542823792,-0.19003045558929443,-0.13519152998924255,-0.016391027718782425,0.03724323958158493,-0.008132826536893845,0.03669063746929169,0.12963241338729858,0.06312166899442673,0.007356367073953152,-0.07693102955818176,-0.06749734282493591,0.04138539731502533,0.04767122119665146,0.023861879482865334,-0.08120811730623245,0.01852356642484665,0.062328897416591644,0.05512731522321701,0.32890117168426514,-0.0645345002412796,-0.04542402923107147,0.0966825857758522,0.0065576182678341866,0.04583323746919632,0.10140229016542435,0.028170756995677948,0.01213375385850668,-0.04389874264597893,0.005959457252174616,-0.027910351753234863,0.10014106333255768,0.056484661996364594,0.29174014925956726,0.11270356923341751,0.011652752757072449,0.013252376578748226,0.015545733273029327,0.045265063643455505,0.04274434596300125,0.08635355532169342,0.1812123954296112,0.060043707489967346,0.01335175335407257,0.08468427509069443,-0.037962429225444794,0.02078811638057232,-0.03337499871850014,0.02255665697157383,0.020962176844477654,0.1229366883635521,0.04996078461408615,0.04262726381421089,0.14085087180137634,0.13130465149879456,0.1897098273038864,0.15476489067077637,-0.005832517985254526,-0.04688820242881775,0.006048683077096939,-0.03847896307706833,-0.17633214592933655,-0.11353612691164017,-0.05206315591931343,0.0011198226129636168,0.02376643754541874,0.13180388510227203,0.12007187306880951,0.1317826807498932,0.1380896121263504,0.20216535031795502,0.15971527993679047,-0.022943150252103806,0.01210705004632473,0.000288309995085001,-0.16251599788665771,-0.10221201926469803,0.0028102514334023,-0.0018592093838378787,-0.023584235459566116,0.27096936106681824,0.03329378366470337,0.05786162614822388,0.17496265470981598,0.14587967097759247,-0.003333225380629301,-0.06753826886415482,-0.06465574353933334,0.017333930358290672,-0.09633607417345047,-0.1611509770154953,-0.054000526666641235,-0.026059400290250778,-0.12804880738258362,0.22267700731754303,0.04335050657391548,-0.034082937985658646,0.03679567575454712,0.0580902099609375,-0.06789350509643555,-0.08542263507843018,-0.09285249561071396,-0.13544081151485443,0.01134929433465004,-0.04813707619905472,0.03364400938153267,-0.1407909095287323,-0.17417530715465546,0.20915567874908447,0.05620553717017174,-0.01526812743395567,0.03907479718327522,0.12172549962997437,0.11334431916475296,-0.015302740037441254,0.013067580759525299,-0.05974600836634636,-0.05270381644368172,-0.02502761036157608,0.01701885089278221,0.042386047542095184,-0.04473496600985527,0.19109608232975006,0.1022418737411499,0.09370819479227066,0.04640090465545654,0.004481656942516565,0.10148876160383224,-0.07165825366973877,-0.1069321408867836,0.002095224568620324,-0.03244009613990784,-0.022563511505723,-0.17141354084014893,-0.004002103582024574,-0.07830069214105606,-0.3254830241203308,-0.23183511197566986,-0.18755492568016052,-0.1892085075378418,-0.19262303411960602,-0.153245747089386,-0.15963268280029297,-0.08441203832626343,-0.11366451531648636,-0.12462252378463745,-0.21145020425319672,-0.2507626712322235,-0.1616809070110321,-0.1254972368478775,-0.11641824245452881,-0.06540298461914062,-0.07600230723619461,-0.12898772954940796,-0.13983185589313507,-0.08195643872022629,-0.07831443846225739,-0.07162890583276749,-0.10813408344984055,-0.09053584188222885,-0.07844957709312439,-0.09016796201467514,-0.04475817456841469,-0.06320257484912872,-0.05400591343641281,-0.10720764100551605,-0.1362333595752716,-0.10144638270139694,-0.049197807908058167,-0.023369481787085533,-0.08214160054922104,-0.03899480029940605,-0.10084843635559082,-0.035527706146240234,-0.01590033993124962,-0.05058467760682106,-0.020165512338280678,-0.043588992208242416,-0.0773753672838211,-0.10901900380849838,-0.09982115030288696,-0.1345863789319992,-0.08595764636993408,-0.00979683082550764,-0.08841165155172348,-0.031553298234939575,-0.02935740537941456,-0.00013921887148171663,-0.07528351247310638,-0.03453496843576431,-0.06785192340612411,-0.027526820078492165,-0.09123840928077698,-0.07640688121318817,-0.06035371869802475,-0.09070718288421631,-0.05190771073102951,-0.08439261466264725,-0.1396508663892746,-0.1225796490907669,0.039563536643981934,0.058081965893507004,0.024065889418125153,0.02414919063448906,-0.015434345230460167,-0.00868085864931345,-0.07941920310258865,-0.049503497779369354,0.02035260945558548,0.027236776426434517,-0.09709805250167847,-0.002666485495865345,0.011269478127360344,-0.014873746782541275,0.12078065425157547,0.1372838169336319,0.09510398656129837,-0.014449494890868664,0.0021687475964426994,0.03979700058698654,0.026414556428790092,0.028005234897136688,0.05227988585829735,0.006788778118789196,-0.019045064225792885,-0.0010472398716956377,0.008309499360620975,-0.03055863082408905,0.12307856976985931,0.1873551458120346,0.08356974273920059,0.0488794706761837,0.011890594847500324,-0.03326958045363426,0.0429985374212265,0.028040507808327675,0.07666146010160446,0.10021110624074936,0.07991382479667664,0.0653461366891861,-0.06494764983654022,-0.05793525278568268,0.03620031848549843,0.1414967030286789,0.16949331760406494,0.063109390437603,0.034467872232198715,0.0265332218259573,-0.03258516639471054,-0.0017483077244833112,0.08803457021713257,0.13746631145477295,0.11485140025615692,0.07773444056510925,0.00038841646164655685,-0.01491391472518444,0.08039306849241257,0.1377290040254593,0.12879079580307007,0.07757018506526947,0.03789949044585228,0.010679795406758785,0.06484708935022354,0.12464381009340286,0.08959183841943741,0.00284777395427227,0.003373285522684455,-0.004744336940348148,-0.05543583258986473,0.028040194883942604,-0.00371481291949749,0.006998526398092508,0.09365437179803848,0.04655235633254051,0.0906793475151062,0.09006094187498093,0.1244385838508606,0.14082694053649902,0.09525033086538315,-0.023668816313147545,-0.03723926842212677,-0.07755475491285324,-0.08118708431720734,-0.021408749744296074,-0.04477119445800781,0.02063916251063347,0.06541232019662857,0.04195953160524368,0.02704009786248207,0.02269037812948227,0.10500269383192062,0.10657127946615219,0.0745893120765686,0.051276903599500656,-0.02783190831542015,-0.11147389560937881,-0.12783658504486084,-0.0840994119644165,0.009783021174371243,-0.020577697083353996,0.07700668275356293,0.0909695029258728,0.06345436722040176,0.0444188117980957,0.1391064077615738,0.08177091926336288,0.011353257112205029,-0.004672566428780556,-0.011231227777898312,-0.05095045268535614,0.010383930057287216,-0.01360662654042244,0.03405213728547096,0.02680121921002865,0.05265234783291817,0.04228148236870766,0.03494889661669731,0.03759993240237236,0.16575902700424194,0.08039125800132751,0.06203555688261986,-0.04885854199528694,-0.06976623088121414,-0.06667114049196243,-0.03435250371694565,-0.057951971888542175,-0.05741901695728302,-0.0003744692075997591,-0.022805284708738327,0.03635484352707863,0.08260119706392288,0.057714223861694336,-0.32796403765678406,-0.22637313604354858,-0.1770772784948349,-0.16924546658992767,-0.16212332248687744,-0.1741148680448532,-0.27560222148895264,-0.19557176530361176,-0.21133004128932953,-0.20186848938465118,-0.29954656958580017,-0.3412359952926636,-0.25143909454345703,-0.27764952182769775,-0.20949018001556396,-0.10205744206905365,-0.10518061369657516,-0.08262641727924347,-0.04799174889922142,-0.051072634756565094,-0.0645410493016243,-0.10018239915370941,-0.14058023691177368,-0.11379754543304443,-0.15670746564865112,-0.17460261285305023,-0.20778462290763855,-0.15169847011566162,-0.11347536742687225,-0.07111670076847076,-0.09959115833044052,-0.013804169371724129,-0.02742912247776985,0.006676047574728727,-0.040834371000528336,-0.09347278624773026,-0.008721674792468548,-0.1194685772061348,-0.13104774057865143,-0.13153676688671112,-0.11747338622808456,-0.14685218036174774,-0.12402553111314774,-0.17997227609157562,-0.13528034090995789,-0.06336266547441483,0.043102674186229706,0.09311708807945251,0.02649816870689392,-0.041222650557756424,-0.011376259848475456,-0.05677112936973572,-0.11469072103500366,-0.06947614997625351,-0.0667293518781662,-0.1378358006477356,-0.05468503758311272,-0.08971867710351944,-0.03150179982185364,0.04380010440945625,0.08629972487688065,0.07020002603530884,-0.053138379007577896,-0.12170876562595367,-0.026291027665138245,0.0948878675699234,0.03979609161615372,-0.03633489832282066,0.01890992745757103,-0.08642492443323135,-0.0779137909412384,-0.07846899330615997,0.010797213762998581,0.06504717469215393,0.10271327942609787,0.17652885615825653,0.08559019863605499,0.08310292661190033,0.06912396848201752,0.06977909058332443,0.024782424792647362,-0.04355359449982643,0.03965921327471733,-0.04208991676568985,-0.07894899696111679,-0.041951123625040054,0.00010105634282808751,0.05921918898820877,0.08646805584430695,0.17576223611831665,0.06425771862268448,0.007389809470623732,0.07802312821149826,0.05687049776315689,0.0033022346906363964,-0.07543576508760452,0.013215357437729836,0.00445640878751874,0.0013068582629784942,-0.032541025429964066,-0.0574314221739769,0.0727759599685669,0.05507266893982887,-0.05352982133626938,-0.06812683492898941,-0.1363229900598526,-0.037881091237068176,0.08468998223543167,0.06014933064579964,0.021635860204696655,-0.03701227903366089,-0.021420802921056747,-0.0042674122378230095,-0.08485328406095505,-0.0022516599856317043,0.040269725024700165,0.06765229254961014,0.12447678297758102,0.0022937317844480276,-0.04145011305809021,0.08276796340942383,0.09413868933916092,0.03991653397679329,0.0021809255704283714,0.01867014542222023,-0.021080808714032173,0.10244590789079666,0.014249826781451702,-0.030297493562102318,-0.06741733849048615,-0.0029803700745105743,-0.00162316404748708,-0.0074363211169838905,0.0014260621974244714,0.07619426399469376,0.005675772670656443,0.04346078634262085,0.001589509192854166,0.004672475159168243,0.0742126926779747,0.15325701236724854,0.11673155426979065,0.04955365136265755,-0.036951836198568344,0.0014942949637770653,-0.01501494087278843,-0.004312135279178619,0.07711135596036911,0.04388374835252762,0.008183321915566921,0.025352435186505318,0.0906057357788086,0.1310218870639801,0.11945323646068573,0.08508297801017761,0.07204669713973999,0.09923440963029861,0.02573721669614315,0.019171202555298805,0.03755883872509003,-0.06225259602069855,0.012032657861709595,0.017073243856430054,0.004852620419114828,0.05335511639714241,0.11851596087217331,0.1820307970046997,0.14632804691791534,0.12293604016304016,0.09436211735010147,0.09453301876783371,-0.010200685821473598,-0.043415095657110214,-0.06629015505313873,-0.07582780718803406,-0.025181453675031662,0.09877309948205948,0.06113141030073166,0.0561041459441185,0.16418986022472382,0.1761515587568283,0.22240081429481506,0.10787083953619003,0.09340794384479523,0.04197058454155922,-0.003955337218940258,-0.09254804998636246,-0.027773302048444748,-0.033862464129924774,0.028439996764063835,0.003310598898679018,-0.01870802789926529,0.04351328685879707,0.10803013294935226,0.2529893219470978,0.3256690502166748,-0.22271625697612762,-0.12165901809930801,-0.12794046103954315,-0.09297764301300049,-0.10128603130578995,-0.030515208840370178,0.01991337537765503,-0.014850625768303871,0.017448367550969124,-0.0508015975356102,-0.1405939757823944,-0.1271880567073822,-0.16701820492744446,-0.19331645965576172,-0.11386530846357346,-0.09752996265888214,-0.0004764324112329632,-0.023918427526950836,-0.009967370890080929,0.030300401151180267,0.04920237883925438,0.025579791516065598,0.08114896714687347,0.006589931435883045,-0.018979353830218315,-0.052449245005846024,-0.07044623047113419,-0.1243162453174591,-0.09711828827857971,-0.025858448818325996,-0.03798120841383934,-0.04672244191169739,0.009719884023070335,0.05739981681108475,0.06667443364858627,0.08749095350503922,0.024694271385669708,0.04941019415855408,-0.0010417683515697718,-0.07907923310995102,-0.09469158947467804,-0.14125584065914154,-0.09846124798059464,-0.08946900814771652,-0.031427182257175446,-0.03773822262883186,0.04101010784506798,0.033338356763124466,0.005949651822447777,-0.027074970304965973,-0.07275501638650894,-0.06988319754600525,-0.059185221791267395,-0.06960733234882355,-0.11086557805538177,-0.11045827716588974,-0.07595322281122208,-0.008232707157731056,-0.028634939342737198,0.010124490596354008,0.12006356567144394,0.10616547614336014,0.011373592540621758,-0.05411143973469734,-0.14026178419589996,-0.14662674069404602,-0.1362888365983963,-0.14395450055599213,-0.08396227657794952,-0.05873022973537445,-0.16901829838752747,-0.20708265900611877,-0.1410178393125534,-0.01523114088922739,0.16096645593643188,0.20067593455314636,0.02579342946410179,-0.04861991107463837,-0.18339329957962036,-0.13716532289981842,-0.2635456919670105,-0.3010954260826111,-0.16650384664535522,-0.08764258027076721,-0.14773139357566833,-0.12834976613521576,-0.030344391241669655,0.0950118899345398,0.15442748367786407,0.12604780495166779,-0.0937499850988388,-0.15040339529514313,-0.20829744637012482,-0.03174320235848427,-0.14048127830028534,-0.17038021981716156,-0.1575256735086441,-0.09410947561264038,-0.054526813328266144,-0.07484446465969086,-0.00622091069817543,0.08208554238080978,0.10772585868835449,-0.0682445764541626,-0.11417420953512192,-0.025620512664318085,0.038388293236494064,0.010575644671916962,-0.008864015340805054,-0.16481667757034302,-0.14429183304309845,-0.11125046759843826,-0.10061237215995789,-0.028632687404751778,-0.11041570454835892,-0.012369956821203232,0.0187662523239851,-0.07446705549955368,-0.0446869432926178,0.027048418298363686,0.0548020638525486,-0.03006948158144951,-0.08023421466350555,-0.1376071274280548,-0.18376903235912323,-0.1672423630952835,-0.09533829241991043,-0.14037668704986572,-0.14354732632637024,-0.09750249981880188,0.02142503671348095,0.10456817597150803,0.06598183512687683,0.03622343763709068,-0.03271954879164696,-0.140231192111969,-0.11394422501325607,-0.15592826902866364,-0.18285104632377625,-0.22049480676651,-0.13497218489646912,-0.09938494116067886,-0.061359431594610214,-0.024972081184387207,0.008143134415149689,-0.018739856779575348,0.04701673239469528,0.049062326550483704,0.010094123892486095,-0.05044781416654587,-0.0897945985198021,-0.03809288144111633,-0.08471525460481644,-0.12351398915052414,-0.06640872359275818,-0.10525926202535629,-0.15137505531311035,-0.12008420377969742,-0.050427671521902084,-0.12817153334617615,-0.18542158603668213,-0.0714215561747551,-0.09546071290969849,-0.10430026799440384,-0.18330226838588715,-0.10052831470966339,-0.04943940415978432,0.006131583359092474,-0.025897610932588577,-0.16176152229309082,-0.10440248250961304,-0.06253107637166977,-0.02758893370628357,-0.11137332767248154,-0.17673738300800323,-0.13089126348495483,-0.08574721962213516,-0.06011904403567314,-0.08633753657341003,0.02736264280974865,0.018556855618953705,0.0658235177397728,-0.02945125289261341,-0.12591193616390228,-0.10847119241952896,-0.11995818465948105,-0.09358879923820496,-0.05988190323114395,-0.15018051862716675,-0.075437992811203,-0.025727814063429832,-0.025644060224294662,0.012382875196635723,0.01924864575266838,0.0839080810546875,0.1065736636519432,-0.04821626469492912,-0.050727274268865585,-0.03411569073796272,-0.05209372937679291,-0.07603072375059128,-0.12781764566898346,-0.0895448848605156,-0.13546565175056458,-0.10940350592136383,-0.05115191265940666,-0.08999650180339813,-0.02690863609313965,-0.0341491661965847,0.008052105084061623,-0.051384713500738144,-0.014783773571252823,-0.005801668856292963,-0.03807034716010094,-0.09548520296812057,-0.12362470477819443,-0.07050994783639908,-0.10521223396062851,-0.06872127950191498,-0.08021008223295212,-0.03354306519031525,-0.0697626993060112,0.0023589441552758217,-0.024429818615317345,-0.010751266963779926,-0.05392962694168091,0.009700500406324863,0.007986917160451412,-0.04391653090715408,-0.12597598135471344,-0.09265931695699692,-0.11855996400117874,-0.07861534506082535,-0.024473220109939575,-0.05969666317105293,-0.05317347124218941,-0.09020394086837769,-0.02613268978893757,-7.335159170906991e-05,-0.022741571068763733,-0.07379487156867981,-0.04502655193209648,-0.08812021464109421,-0.18665224313735962,-0.17371700704097748,-0.15712183713912964,-0.08741851150989532,-0.07472516596317291,-0.011882457882165909,-0.0014512583147734404,0.006212364416569471,0.01562027633190155,0.04737309366464615,0.036309897899627686,0.03278055414557457,0.028412217274308205,-0.06651797890663147,-0.08859530836343765,-0.138093963265419,-0.09716865420341492,0.025367461144924164,0.026583857834339142,-0.05145210400223732,0.045053038746118546,-0.004413288552314043,0.03254628926515579,0.01546549517661333,0.03550085797905922,0.09862732142210007,0.059298571199178696,-0.02774585783481598,-0.05271860584616661,-0.15100981295108795,-0.11762289702892303,-0.058336641639471054,0.007178886793553829,0.02537415362894535,0.09492937475442886,0.09315351396799088,0.09641723334789276,0.010380622930824757,0.02091681957244873,0.04009344056248665,0.07851600646972656,-0.030557360500097275,-0.08835554867982864,-0.022931300103664398,-0.05225369334220886,-0.038498543202877045,0.016494527459144592,0.007643484510481358,0.04692929983139038,-0.004933570511639118,-0.016299456357955933,0.05883456766605377,0.04527874290943146,0.08259329199790955,0.05996216833591461,0.006882351357489824,-0.0480465292930603,-0.046907346695661545,-0.09502270072698593,0.012567972764372826,0.07653288543224335,0.001011035987176001,-0.03177938237786293,0.0008279150351881981,0.030237343162298203,-0.01705879345536232,0.00806343276053667,-0.013044137507677078,0.03906974568963051,0.10477413982152939,0.08864403516054153,0.08859868347644806,0.026285624131560326,0.03720976412296295,0.03590751811861992,-0.0223798006772995,-0.031074007973074913,-0.002097097225487232,0.04954094812273979,0.04358912259340286,-0.028001485392451286,0.011876723729074001,0.012683399021625519,-0.03340562433004379,-0.030011707916855812,-0.04473654180765152,-0.01882799342274666,0.022305842489004135,0.0556105338037014,-0.02376766875386238,-0.0275077186524868,0.0022756806574761868,0.05291033536195755,-0.031001850962638855,0.04645754024386406,0.0836179330945015,-0.038279835134744644,-0.02485516294836998,-0.06053825467824936,-0.032540153712034225,0.04642973840236664,-0.0017422433011233807,0.03346528857946396,0.032096944749355316,0.002658615354448557,0.012157225050032139,0.020873036235570908,-0.09813202917575836,-0.060120243579149246,0.04367389157414436,0.01756899431347847,-0.00047141825780272484,-0.020298119634389877,-0.028322767466306686,-0.013732349500060081,-0.06174603849649429,-0.021160230040550232,0.06277857720851898,0.051208216696977615,-0.030057407915592194,-0.016992470249533653,-0.15959922969341278,-0.12264194339513779,-0.02390255779027939,-0.10532219707965851,-0.07428206503391266,-0.08879990130662918,-0.05338415130972862,-0.07665907591581345,-0.02316351793706417,-0.1118699312210083,0.0005656928988173604,-0.013895858079195023,-0.03909202292561531,-0.14951030910015106,-0.23544035851955414,-0.14114458858966827,-0.11643380671739578,-0.1587655246257782,-0.15657161176204681,-0.13676893711090088,-0.20656414330005646,-0.07411684840917587,-0.1728382259607315,-0.2230839729309082,-0.10581324249505997,-0.1639561504125595,-0.16886815428733826,-0.2604769468307495,-0.2457435429096222,-0.2430710643529892,-0.08670689910650253,-0.05352932959794998,-0.03811752796173096,0.02215578593313694,0.008970827795565128,0.0018852147040888667,0.04031182825565338,5.90575800742954e-05,-0.06626430153846741,-0.06280609220266342,-0.13733530044555664,-0.16195052862167358,-0.23440705239772797,-0.15368276834487915,-0.05926704406738281,-0.09017281234264374,-0.13174672424793243,-0.05388621985912323,-0.12108860909938812,-0.10970760881900787,-0.1492849439382553,-0.08341361582279205,0.0008167186751961708,-0.10558348894119263,-0.16544687747955322,-0.19083921611309052,-0.20117081701755524,-0.08528155833482742,-0.000523716735187918,-0.05059083178639412,-0.060996562242507935,-0.09768518060445786,-0.21625658869743347,-0.12721668183803558,-0.1272747665643692,-0.0937979444861412,-0.05618365854024887,-0.13537871837615967,-0.20121654868125916,-0.17816980183124542,-0.12799206376075745,-0.007671897299587727,-0.10860487073659897,-0.26621678471565247,-0.2284611314535141,-0.11166295409202576,-0.10692384093999863,-0.12972450256347656,-0.038110628724098206,-0.06876952946186066,-0.11981038004159927,-0.07439005374908447,-0.22814874351024628,-0.24241837859153748,0.014668050222098827,-0.03644770383834839,-0.20036721229553223,-0.27480441331863403,-0.24869579076766968,-0.190004363656044,-0.10139025747776031,-0.10724475979804993,-0.0942704975605011,-0.22233286499977112,-0.12270677089691162,0.019619852304458618,-0.1180155947804451,-0.031745363026857376,-0.1293763369321823,0.03556925058364868,-0.12839974462985992,-0.2035840004682541,-0.2340545952320099,-0.31518739461898804,-0.21946416795253754,-0.16810020804405212,-0.1991853564977646,-0.24885037541389465,-0.16704170405864716,-0.1258975714445114,-0.036264996975660324,-0.02143036015331745,-0.14191769063472748,-0.05645814165472984,-0.07711420208215714,-0.12805891036987305,-0.1483473777770996,-0.13145239651203156,-0.2834758758544922,-0.31363099813461304,-0.09216104447841644,-0.1428571492433548,-0.2106216549873352,-0.17584063112735748,-0.062399666756391525,-0.13175949454307556,-0.13959874212741852,0.0016135784098878503,0.026785841211676598,-0.03595302626490593,0.050825394690036774,0.022129472345113754,-0.254751592874527,-0.1344272643327713,-0.02331881783902645,0.06346222013235092,-0.14869475364685059,-0.16908657550811768,-0.03228525444865227,0.03342534601688385,-0.375254362821579,-0.12623445689678192,-0.04592083394527435,-0.04474502056837082,0.003794264979660511,-0.07055207341909409,0.03622167930006981,0.1158975288271904,0.0625913217663765,-0.03779350966215134,-0.037390708923339844,-0.0825960710644722,-0.11788025498390198,-0.25074896216392517,-0.32013338804244995,-0.10240721702575684,-0.13923873007297516,-0.19954892992973328,-0.06756286323070526,-0.1760711967945099,-0.027073921635746956,0.06584538519382477,0.034945420920848846,-0.0786447674036026,-0.15211321413516998,-0.1647919863462448,-0.2086118906736374,-0.31258532404899597,-0.3894176781177521,-0.2309330850839615,-0.23364338278770447,-0.09781213104724884,0.025619208812713623,0.1884840726852417,0.19199596345424652,0.16644513607025146,0.11958736181259155,0.07091590017080307,-0.02833925187587738,-0.1938628852367401,-0.15255558490753174,-0.21797510981559753,-0.48624277114868164,-0.38837647438049316,-0.1989799588918686,0.015804853290319443,0.1340019851922989,0.10114174336194992,0.062337324023246765,0.009242605417966843,0.11287688463926315,0.031884439289569855,-0.022945640608668327,-0.14462028443813324,-0.15638388693332672,-0.37371066212654114,-0.633001983165741,-0.4190801978111267,-0.27452853322029114,-0.10284052789211273,-0.044967152178287506,0.028755733743309975,-0.07139160484075546,-0.018582100048661232,-0.08536142855882645,-0.13326531648635864,-0.19226767122745514,-0.24991197884082794,-0.348466694355011,-0.4217680096626282,-0.6502793431282043,-0.5260105133056641,-0.46018263697624207,-0.39625120162963867,-0.15215545892715454,-0.07955002784729004,0.005865252111107111,-0.018868338316679,-0.14079348742961884,-0.2600468695163727,-0.34798452258110046,-0.43869921565055847,-0.4906442165374756,-0.6212345361709595,-0.10449657589197159,-0.08967474848031998,-0.02022048458456993,0.09588015079498291,0.12766830623149872,0.06599705666303635,0.022545374929904938,0.2133396416902542,0.1489376425743103,0.0508897639811039,0.14119921624660492,0.06503233313560486,0.02707894891500473,0.12293048202991486,0.0641416683793068,0.037939053028821945,0.05747370049357414,-0.018558941781520844,0.03841320797801018,-0.038289234042167664,0.07638508826494217,0.04966527596116066,0.01642565242946148,0.04641102999448776,0.11257362365722656,0.06546066701412201,0.10885496437549591,0.16313140094280243,0.07197469472885132,0.06060471013188362,0.16973251104354858,0.07983428984880447,0.033607300370931625,-0.1295788288116455,0.11760587245225906,-0.007215050980448723,-0.0040756212547421455,0.1453312635421753,0.1961556226015091,0.20183047652244568,0.15191593766212463,0.15552392601966858,0.11112941056489944,0.07324958592653275,0.152248814702034,0.12428376823663712,0.11675763875246048,-0.23066097497940063,-0.09478331357240677,-0.02727178856730461,-0.006255181506276131,0.2302182912826538,0.24429087340831757,0.07646484673023224,0.03888712450861931,0.09824191778898239,0.09645403921604156,0.1486133337020874,0.1145332008600235,0.0878465473651886,-0.028828686103224754,-0.08916791528463364,-0.061137136071920395,-0.06126880273222923,0.08611810207366943,0.1143094077706337,0.1765156239271164,0.16062696278095245,0.018583931028842926,0.09752639383077621,0.04478833079338074,0.11185063421726227,0.05012805759906769,0.020000014454126358,0.048428431153297424,-0.11965495347976685,-0.07865334302186966,-0.10187619179487228,0.1256057173013687,0.1258297562599182,0.09607428312301636,0.10483531653881073,-0.000155344809172675,-0.02853209152817726,0.13168850541114807,0.07547508925199509,-0.02677120640873909,-0.040203679352998734,0.06339500099420547,-0.05087309330701828,-0.022080454975366592,-0.07381107658147812,0.055720649659633636,0.07640234380960464,0.06681226938962936,0.0520097091794014,-0.004322309046983719,0.00902253482490778,0.03811696916818619,-0.001357420813292265,-0.0018790874164551497,0.1306057721376419,-0.015844784677028656,-0.0028826994821429253,-0.09700403362512589,-0.113426074385643,0.03710321709513664,0.04956686496734619,0.012652341276407242,0.13190768659114838,0.14327336847782135,0.036238692700862885,0.04322953149676323,0.10047627240419388,0.002195658627897501,0.11064620316028595,-0.015924669802188873,-0.005472081713378429,-0.05530691146850586,0.020909883081912994,-0.04169779270887375,0.025780415162444115,-0.019425101578235626,0.018301524221897125,0.040308814495801926,0.06383411586284637,-0.007413136772811413,0.01779974065721035,-0.054813988506793976,-0.009051858447492123,0.0020726120565086603,-0.12498059123754501,-0.03572952002286911,-0.029577692970633507,-0.07120724767446518,-0.00026969649479724467,-0.07097892463207245,-0.0528545081615448,-0.08049875497817993,0.010359514504671097,0.037017952650785446,-0.07987257838249207,-0.03857753053307533,0.004666197579354048,-0.016107389703392982,-0.11705028265714645,0.05470963194966316,-0.0022755665704607964,0.062118902802467346,-0.07913145422935486,-0.04874204099178314,-0.07222114503383636,-0.12614381313323975,-0.21622025966644287,-0.008660628460347652,0.00439589936286211,-0.08325540274381638,-0.08343730121850967,0.05500444769859314,-0.06357720494270325,0.08812731504440308,-0.12295341491699219,-0.0012108496157452464,0.06984464079141617,-0.08190660923719406,-0.16922853887081146,-0.16036050021648407,-0.23547598719596863,-0.006167137995362282,0.023102018982172012,0.07769235968589783,0.01448196079581976,-0.007922839373350143,-0.11215537041425705,0.06573159247636795,-0.048055242747068405,-0.11458813399076462,0.08628804236650467,-0.012571575120091438,-0.06137068569660187,-0.19808541238307953,-0.23326314985752106,-0.011333559639751911,0.08127108216285706,0.09367270767688751,-0.018750308081507683,0.06543993204832077,0.005274628754705191,-0.05409194901585579,0.008617047220468521,-0.012520446442067623,0.007410005666315556,-0.15076221525669098,-0.13059720396995544,-0.21744808554649353,-0.1326092630624771,-0.18477056920528412,-0.04010434076189995,-0.03333084285259247,0.008740227669477463,-0.015517289750277996,-0.01551097258925438,-0.06662730127573013,-0.06232823058962822,0.007601799909025431,-0.05749904364347458,-0.13283871114253998,-0.1818479746580124,-0.2297877073287964,-0.2047310471534729,-0.052259065210819244,-0.022565754130482674,-0.11868966370820999,-0.05812261998653412,-0.066612608730793,-0.06641342490911484,-0.07734101265668869,-0.08279963582754135,-0.04579155147075653,-0.1194671094417572,-0.026179106906056404,-0.10418517887592316,-0.11210034042596817,-0.06091506406664848,-0.08632231503725052,-0.04740085452795029,0.01126320194453001,0.018813185393810272,-0.002404947532340884,0.09163136035203934,0.042164288461208344,0.06263208389282227,0.019171999767422676,0.0047964733093976974,0.05662703141570091,-0.003839047159999609,-0.0028899251483380795,-0.088412344455719,-0.07380765676498413,-0.012045447714626789,-0.03951047360897064,0.030719252303242683,0.0665116086602211,0.06373108923435211,-0.028191974386572838,-0.08673398196697235,-0.1272343546152115,0.0013477186439558864,0.10344240814447403,-0.032453473657369614,-0.04461166635155678,-0.1163293868303299,-0.03723445162177086,-0.002811710350215435,0.005907457787543535,0.12050101161003113,0.1130322813987732,0.0294254831969738,-0.04531547799706459,-0.0827360674738884,-0.1315394788980484,0.09846731275320053,0.06990838795900345,0.13711011409759521,0.045843709260225296,-0.07133226096630096,-0.03955886512994766,-0.014814800582826138,0.012492629699409008,0.040394335985183716,-0.07593541592359543,0.1425338238477707,0.02673487924039364,0.028837142512202263,0.03167198970913887,0.14569716155529022,0.05272495374083519,-0.010801807977259159,-0.020521890372037888,-0.04000278189778328,-0.05335867404937744,-0.1412510722875595,-0.13323143124580383,-0.12554070353507996,0.13175827264785767,0.17407387495040894,0.03306332975625992,-0.06622587889432907,0.016551943495869637,0.09489496052265167,0.00718639325350523,0.04289163276553154,0.01453198492527008,-0.03587184473872185,-0.06051221489906311,-0.1341060847043991,-0.06477600336074829,0.09578325599431992,0.04748671501874924,0.02126379683613777,-0.0484464094042778,-0.1621319204568863,0.0031661444809287786,0.18167652189731598,0.17307333648204803,0.13629426062107086,0.042773906141519547,0.05200452730059624,-0.07327739149332047,0.00720559386536479,-0.023372594267129898,0.07437343895435333,-0.026897383853793144,-0.021479401737451553,-0.01273359078913927,-0.10250827670097351,0.05101411044597626,0.1721661239862442,0.17966143786907196,0.016205526888370514,-0.03187333419919014,-0.06643279641866684,0.026668235659599304,0.024732304736971855,-0.04720299690961838,-0.21913184225559235,-0.0909285917878151,0.0015811275225132704,-0.0006881351582705975,0.11271768063306808,0.13563469052314758,0.05829992517828941,-0.015925951302051544,-0.02988574281334877,0.019208477810025215,0.034117113798856735,0.08964888751506805,0.08126261830329895,0.0607818141579628,-0.17349962890148163,-0.02651960588991642,0.07134180516004562,0.16256888210773468,0.17457354068756104,0.15780101716518402,0.0832558274269104,-0.003961531445384026,0.058930203318595886,0.03566606342792511,0.006737519055604935,0.11058258265256882,0.00211128662340343,0.04997048154473305,-0.07004909217357635,0.027505844831466675,-0.046996716409921646,0.017098531126976013,0.02351037785410881,0.03142829239368439,0.0505443774163723,-0.023657366633415222,-0.03892120346426964,-0.013402117416262627,-0.02633659355342388,0.10116363316774368,0.03359430283308029,0.005488678812980652,0.020897282287478447,0.014189819805324078,-0.10517075657844543,0.012529410421848297,0.02664138190448284,0.034059781581163406,0.012111942283809185,0.0267329104244709,-0.08256478607654572,-0.08696286380290985,-0.09457170218229294,0.0778135284781456,-0.005401379428803921,0.03926123306155205,0.0109754903241992,-0.05944369360804558,-0.007049124222248793,0.04163520783185959,0.002820487366989255,-0.024743767455220222,-0.044306542724370956,-0.044714272022247314,-0.1551835685968399,-0.18424668908119202,-0.31353268027305603,-0.3727366328239441,-0.2727481424808502,-0.15685921907424927,-0.16429534554481506,-0.08759921789169312,-0.10610141605138779,-0.24480266869068146,-0.3521096110343933,-0.25132033228874207,-0.10652824491262436,-0.2031598538160324,-0.20484547317028046,-0.2219248265028,-0.183253675699234,-0.21536707878112793,-0.1363700032234192,-0.0697605088353157,-0.08147986233234406,-0.0341641828417778,-0.17399811744689941,-0.2730737328529358,-0.19760450720787048,-0.16179905831813812,-0.12591169774532318,-0.2680656611919403,-0.1905321329832077,-0.182625412940979,-0.2059267908334732,-0.21575550734996796,-0.05981812626123428,-0.03061050921678543,-0.0911843553185463,-0.15897545218467712,-0.17240910232067108,-0.2074187994003296,-0.10890282690525055,-0.23993320763111115,-0.18297621607780457,-0.24638314545154572,-0.26031288504600525,-0.20962116122245789,-0.22759459912776947,-0.12996019423007965,-0.07640355825424194,-0.21615180373191833,-0.24267934262752533,-0.259537935256958,-0.25121140480041504,-0.32092979550361633,-0.19636327028274536,-0.22209502756595612,-0.1215895414352417,-0.13924449682235718,-0.2343684583902359,-0.21784234046936035,-0.2841455638408661,0.1147049069404602,-0.012900877743959427,-0.1707967221736908,-0.1415361762046814,-0.20797370374202728,-0.22631700336933136,-0.18961268663406372,-0.2615956962108612,-0.21763518452644348,-0.06470667570829391,0.013455993495881557,-0.0834827870130539,-0.07325819134712219,-0.10265473276376724,0.14713329076766968,0.12326011061668396,0.01860363781452179,0.03414400294423103,-0.08570753782987595,-0.045753154903650284,-0.18779759109020233,-0.05323810130357742,-0.08222890645265579,-0.08716527372598648,-0.05859372764825821,-0.0203635785728693,0.001069917343556881,-0.02429935894906521,-0.09446881711483002,0.013287490233778954,-0.07431578636169434,-0.08952134847640991,0.025577470660209656,-0.09475427865982056,-0.3956962823867798,-0.27395009994506836,-0.0217968188226223,0.10611627250909805,-0.06545199453830719,-0.10649814456701279,-0.08895249664783478,-0.11436166614294052,-0.025355840101838112,0.08918583393096924,-0.021876152604818344,-0.14236395061016083,0.07882309705018997,-0.15854454040527344,-0.27122262120246887,-0.3977709710597992,-0.23740923404693604,0.11959309875965118,0.06778451800346375,-0.11297215521335602,-0.08403202891349792,-0.09144303947687149,-0.09930158406496048,0.02558809705078602,-0.13578397035598755,-0.1981952041387558,-0.044161491096019745,-0.1606682687997818,0.03547796607017517,-0.10743699222803116,-0.21633730828762054,-0.0662255585193634,-0.07571858167648315,-0.058928053826093674,-0.13738802075386047,-0.19635751843452454,0.0034320680424571037,-0.004728727042675018,-0.10785672068595886,-0.22744497656822205,-0.08853837102651596,-0.18986178934574127,-0.005252521485090256,0.06756405532360077,-0.10010313987731934,-0.1686965525150299,-0.25280264019966125,-0.21381351351737976,-0.03983638435602188,-0.07525116950273514,-0.02805623970925808,-0.051769059151411057,-0.08219324797391891,-0.025813566520810127,0.03921540081501007,0.06618126481771469,0.012390418909490108,0.07426899671554565,0.055601418018341064,0.03953349217772484,-0.09411806613206863,-0.1552535742521286,-0.04160526022315025,0.004742750432342291,-0.2617879807949066,-0.27276304364204407,-0.09679979085922241,0.08720619231462479,0.17526397109031677,0.16916847229003906,-0.000848693132866174,0.051573969423770905,-0.009990161284804344,0.00806397944688797,0.02127797156572342,-0.044873498380184174,-0.1178954690694809,-0.14932772517204285,-0.34580811858177185,-0.29198703169822693,-0.06278325617313385,0.0030450178310275078,0.11241410672664642,0.12795555591583252,0.0741538405418396,0.045420922338962555,0.10937056690454483,-0.088421531021595,-0.012660841457545757,-0.10899027436971664,-0.1209246888756752,-0.24026620388031006,-0.45314085483551025,-0.28621670603752136,-0.2269844114780426,-0.18939602375030518,-0.14982031285762787,0.12783628702163696,0.06150110810995102,0.01451139710843563,0.001639699563384056,-0.1774018257856369,-0.1870642751455307,-0.22537969052791595,-0.31853970885276794,-0.37585920095443726,-0.06289781630039215,-0.0030844712164252996,-0.008140669204294682,-0.012124374508857727,-0.023982586339116096,0.02085200883448124,0.028797566890716553,0.0006407044129446149,-0.0016184223350137472,-0.046253133565187454,-0.05725438520312309,-0.02264389768242836,-0.012575986795127392,-0.05452800169587135,0.0034121812786906958,-0.026889681816101074,-0.0956253856420517,-0.16387176513671875,-0.037896737456321716,0.01653931848704815,0.0133128073066473,0.0022182667162269354,0.010029304772615433,-0.1033053919672966,-0.020016079768538475,-0.008454015478491783,0.045262694358825684,-0.047576360404491425,-0.060024112462997437,-0.04065519943833351,-0.10266255587339401,-0.09199448674917221,-0.004702229984104633,0.09918159991502762,0.06859079748392105,0.002366890199482441,0.016426553949713707,0.0075991833582520485,-0.03807174414396286,-0.02028077468276024,-0.010477938689291477,-0.011910009197890759,-0.012533852830529213,-0.14525485038757324,-0.10918036848306656,-0.0960162878036499,0.0019956084433943033,0.06615103036165237,0.060492318123579025,-0.03597594425082207,0.00201450870372355,-0.0086754085496068,0.02456243336200714,-0.03063620813190937,0.04543963447213173,0.018561121076345444,-0.04615239426493645,-0.11827953904867172,-0.05526445060968399,0.005030321888625622,0.005963666830211878,0.05168263614177704,0.07055342197418213,0.1089765653014183,0.152799591422081,0.07802475243806839,-0.056248195469379425,-0.018602963536977768,-0.0014541661366820335,0.006204587407410145,-0.06945887953042984,-0.06399178504943848,-0.02777869626879692,-0.041253842413425446,-0.054065488278865814,-0.05890022590756416,0.10701041668653488,0.190789133310318,0.10996266454458237,0.1232142522931099,0.010999665595591068,-0.01638023741543293,-0.029208611696958542,-0.03691607713699341,-0.021583814173936844,-0.09523085504770279,-0.12642750144004822,-0.07010828703641891,0.017074227333068848,0.042577676475048065,0.14669868350028992,0.1844482123851776,0.17863275110721588,0.008196212351322174,-0.01108349859714508,0.03415684774518013,0.035715971142053604,0.0034301220439374447,0.0030537063721567392,-0.042094260454177856,0.04158090427517891,0.06991716474294662,0.12067671865224838,0.07304199784994125,0.11075703799724579,0.13283243775367737,0.1920967400074005,0.10838240385055542,-0.006966358050704002,-0.016811273992061615,-0.05993318557739258,0.06273278594017029,0.00047287915367633104,-0.026786455884575844,-0.01972516067326069,-0.01972082629799843,-0.05187699943780899,0.00016921546193771064,0.04393118619918823,0.038738083094358444,0.05714653804898262,-0.0027843378484249115,0.06805219501256943,0.03980138152837753,0.045511748641729355,-0.04679065942764282,0.04671178013086319,0.10038400441408157,0.08006288856267929,0.11691102385520935,0.12193772941827774,-0.016190068796277046,0.0453982949256897,0.037285175174474716,0.047878164798021317,0.05066089704632759,0.013456225395202637,0.013464371673762798,0.008022814989089966,-0.024498773738741875,0.07457708567380905,0.05861861631274223,0.01892162673175335,0.08453287929296494,0.08471950143575668,0.11992213875055313,0.10302471369504929,0.07004150003194809,0.04374008998274803,0.07357832789421082,0.07704069465398788,0.04442813992500305,0.00972286518663168,-0.03378446772694588,0.10640446096658707,0.04115409776568413,0.025789083912968636,0.015317504294216633,0.07449441403150558,-0.005604740232229233,0.05630401149392128,0.03346242755651474,0.03283223509788513,0.016343049705028534,0.06251628696918488,0.06072460487484932,0.03853200376033783,0.13118959963321686,0.19029352068901062,0.07692055404186249,0.03697832673788071,0.03400410711765289,0.020975332707166672,0.013672370463609695,0.0757218599319458,0.020751463249325752,0.05386076495051384,-0.016381340101361275,0.0343366414308548,0.021028771996498108,0.05206049606204033,0.06463783234357834,0.03039143607020378,0.06235578656196594,0.09389849752187729,0.034960292279720306,-0.04894446209073067,-0.038637541234493256,-0.024962879717350006,-0.05526193976402283,-0.08416369557380676,-0.007428804878145456,-0.0463893748819828,-0.012850102968513966,-0.01957651600241661,-0.03436797484755516,0.01284933090209961,0.04389169067144394,0.015008848160505295,-0.0599980503320694,-0.04430036619305611,-0.11000440269708633,-0.06472234427928925,0.05930718779563904,0.020839007571339607,0.0149186747148633,0.003489074995741248,0.07223829627037048,0.03702576830983162,-0.02733955718576908,0.006929856259375811,0.050745975226163864,0.04378465563058853,-0.05665043741464615,-0.06050698831677437,-0.025552313774824142,0.004755547270178795,0.040846966207027435,0.026765840128064156,0.10200651735067368,0.03690493479371071,-0.0032813274301588535,0.05573863536119461,-0.03180691599845886,-0.05670633912086487,0.015242786146700382,0.0016478864708915353,-0.037847746163606644,0.01764829456806183,0.05618822202086449,0.05405076965689659,0.11131131649017334,0.07142215222120285,0.07987015694379807,-0.01451981719583273,-0.0014011685270816088,0.027395600453019142,-0.026707572862505913,-0.04250694811344147,-0.01163356751203537,-0.03285939246416092,-0.05689076706767082,-0.03265795484185219,-0.002437582239508629,0.05953654646873474,0.09062768518924713,0.09095294773578644,0.0054849255830049515,-0.01503688469529152,0.03806810826063156,0.015216800384223461,-0.052989110350608826,0.05142552778124809,0.00844130665063858,0.024672865867614746,-0.12096589058637619,0.027721567079424858,0.0845150426030159,0.033086273819208145,0.10934886336326599,0.04070490226149559,-0.06368599086999893,-0.08176425099372864,-0.009795714169740677,-0.000341012462740764,-0.11480847746133804,-0.004617877770215273,-0.00814793724566698,0.020027652382850647,-0.014856918714940548,-0.07910974323749542,-0.0007719364948570728,0.13172860443592072,0.1121923103928566,-0.04816886782646179,-0.1660252958536148,-0.08199143409729004,-0.008180896751582623,-0.001882921322248876,-0.04501722380518913,-0.04852445051074028,-0.01244149636477232,0.010201314464211464,-0.0007060010102577507,-0.05016973242163658,-0.008635733276605606,0.08997516334056854,0.0974857434630394,0.062308765947818756,-0.07070649415254593,-0.09735742956399918,0.0386914387345314,0.020661786198616028,-0.07118707150220871,-0.019647950306534767,0.02975299023091793,0.05707520619034767,-0.04161212965846062,-0.04396440088748932,-0.04779437556862831,0.09184077382087708,0.11999524384737015,0.06090307980775833,0.02294759452342987,0.007756190374493599,-0.0004816177242901176,-0.007946016266942024,-0.04142579063773155,0.054877869784832,-0.01712839864194393,-0.004555891267955303,-0.024270765483379364,-0.04281916469335556,-0.010824408382177353,-0.029238037765026093,0.14888738095760345,0.06307592988014221,0.03159191459417343,0.1258210837841034,0.08533558249473572,-0.02358582802116871,-0.012995847500860691,-0.06076778471469879,0.014765040017664433,-0.05965088680386543,-0.0806608721613884,-0.03787454590201378,-0.0023249834775924683,0.04469649866223335,0.04926523193717003,0.03081190213561058,0.0685059130191803,0.11712922155857086,0.0038338243030011654,0.0035093287006020546,-0.01819964125752449,-0.10587315261363983,-0.09341910481452942,-0.05784190446138382,-0.02201123535633087,0.03875090181827545,0.012736915610730648,-0.010987598448991776,0.003631003201007843,0.00825175829231739,-0.006431963294744492,0.09349428862333298,0.016537800431251526,-0.06990087777376175,-0.058130502700805664,-0.15130740404129028,-0.04873385652899742,-0.04181738197803497,-0.001602353062480688,0.03377437964081764,0.017724422737956047,-0.07138288021087646,-0.019071446731686592,0.04447769746184349,0.0032634406816214323,0.023818062618374825,0.006094104610383511,0.02954852394759655,-0.04847533255815506,-0.17951002717018127,-0.049107737839221954,0.013696531765162945,-0.04708714410662651,-0.02911413460969925,-0.01449314784258604,-0.0077950432896614075,-0.05129661038517952,-0.05017400532960892,-0.03203941881656647,-0.040309611707925797,-0.05382583290338516,0.017848370596766472,-0.11447913944721222,-0.1705215722322464,-0.03275148198008537,-0.009689515456557274,-0.018079353496432304,0.01268830057233572,-0.0686645433306694,-0.04906735569238663,-0.12751854956150055,-0.11015128344297409,-0.16271530091762543,-0.1254766285419464,-0.04315512627363205,-0.05086209997534752,-0.1077406108379364,-2.1194728105911054e-05,0.016136767342686653,0.005445514339953661,0.027300983667373657,-0.02805185690522194,-0.018903901800513268,0.09892567247152328,0.0912487655878067,0.1011245846748352,0.01399283017963171,0.14441579580307007,0.2513993978500366,0.1257028579711914,0.0605817586183548,-0.014773101545870304,-0.07637758553028107,-0.018792366608977318,-0.09619089961051941,-0.12787926197052002,-0.1260204315185547,-0.0880565345287323,-0.07376521825790405,0.03515727072954178,0.0525706522166729,0.09392160922288895,0.15804995596408844,0.11544101685285568,0.029087139293551445,0.08417986333370209,-0.01386194583028555,0.026886746287345886,0.1724088042974472,0.06638138741254807,0.13593101501464844,0.008198296651244164,0.07163505256175995,0.03787362203001976,0.04442902281880379,0.029684152454137802,0.09149739146232605,0.20008650422096252,0.19672636687755585,0.07744809240102768,0.04708082973957062,0.029393479228019714,0.022308295592665672,0.027247188612818718,0.21413998305797577,0.13049086928367615,0.2905707061290741,0.22005769610404968,-0.02363959699869156,-0.07032183557748795,0.05896200239658356,0.15012696385383606,0.11108779907226562,-0.1865263432264328,-0.08936820924282074,-0.005537297576665878,-0.07103502005338669,0.03817172348499298,0.041092757135629654,0.009253252297639847,0.25139114260673523,0.3145888149738312,0.039640817791223526,-0.0904327780008316,-0.01966678537428379,-0.031976357102394104,-0.01028482150286436,-0.08729206770658493,-0.017007101327180862,0.12128715962171555,0.18372899293899536,0.22689953446388245,-0.006784542929381132,0.233718603849411,0.16360361874103546,0.05314617604017258,0.04552585259079933,0.06568586826324463,-0.030907662585377693,0.03608616441488266,0.00443300511687994,0.026838045567274094,0.06299775838851929,0.07255543768405914,0.008694414980709553,-0.1161375567317009,-0.06174108013510704,0.30558687448501587,0.2642221748828888,0.14606355130672455,-0.0027756751514971256,0.0057595958933234215,-0.04010557383298874,-0.07516548037528992,-0.020251458510756493,0.006919761188328266,0.06652730703353882,0.15282100439071655,0.06450844556093216,-0.20897987484931946,-0.15563808381557465,0.191853329539299,0.04151466116309166,0.11654389649629593,-0.026602571830153465,-0.03584801405668259,-0.04880367964506149,0.011691123247146606,0.07235433161258698,0.07440761476755142,0.0727279856801033,0.10191614180803299,-0.027189649641513824,-0.2338804006576538,-0.23458966612815857,0.05208209156990051,-0.1931532472372055,-0.10049482434988022,-0.016280924901366234,-0.09432994574308395,-0.10244297236204147,-0.08966932445764542,-0.0026614824309945107,0.10143464803695679,0.03960023447871208,0.03762196749448776,0.02227851189672947,-0.03440653532743454,-0.16209375858306885,-0.012675660662353039,-0.05564304068684578,-0.0906108170747757,0.009853560477495193,-0.008524679578840733,0.02935875579714775,-0.03750211000442505,-0.007422724738717079,0.14990387856960297,-0.005991063080728054,0.03694941848516464,-0.007572047878056765,-0.14800314605236053,-0.1085440069437027,-0.16523775458335876,-0.28404146432876587,-0.2066005915403366,-0.2525320053100586,-0.0846979022026062,0.03206157684326172,0.03262403607368469,-0.026469890028238297,0.09637526422739029,0.020710431039333344,0.023671627044677734,-0.016974549740552902,0.002941141137853265,-0.09933564811944962,-0.08402993530035019,-0.034302178770303726,-0.159653902053833,-0.1925947666168213,-0.13885462284088135,-0.03769013658165932,-0.015551827847957611,-0.15359850227832794,0.13604703545570374,0.07777553796768188,0.025583311915397644,-0.09187508374452591,0.07106960564851761,0.013469464145600796,-0.07212924212217331,0.003222564235329628,-0.07350686192512512,-0.002593665150925517,0.030470065772533417,-0.023646622896194458,-0.11610925197601318,-0.07544781267642975,0.2327999621629715,0.24544015526771545,0.12905898690223694,-0.05988088250160217,0.09447242319583893,0.012741763144731522,-0.18622387945652008,-0.0020950825419276953,0.019666176289319992,-0.00922856293618679,0.08582314103841782,-0.036147717386484146,0.03573021665215492,0.05498397350311279,-0.1301422268152237,-0.10143107175827026,-0.04655515030026436,-0.024647727608680725,-0.10758215934038162,-0.13123765587806702,-0.1384858936071396,-0.1783030778169632,-0.10377368330955505,0.01198309101164341,-0.06632400304079056,-0.057358887046575546,-0.019279757514595985,-0.14663633704185486,-0.13832421600818634,-0.0591939277946949,-0.06953588128089905,-0.04455346614122391,-0.054873473942279816,-0.06682697683572769,-0.12466419488191605,-0.1281260848045349,-0.016947360709309578,0.02867954783141613,-0.054666634649038315,-0.007608455140143633,-0.03620085120201111,-0.10629533231258392,-0.08431562036275864,-0.05090653523802757,-0.09281610697507858,0.087362140417099,0.11154096573591232,0.1270635426044464,-0.11522749066352844,-0.025241533294320107,0.07738655060529709,0.12790732085704803,-0.04531877487897873,-0.09439700096845627,-0.01472361572086811,0.052998173981904984,-0.1276414543390274,-0.13571171462535858,-0.08835743367671967,-0.02449057810008526,-0.016100691631436348,0.037906158715486526,-0.03645429015159607,-0.033626358956098557,0.10729002207517624,-0.05471063032746315,-0.1114489957690239,-0.005756088066846132,0.010693260468542576,-0.04884903132915497,0.03196883201599121,-0.05500826984643936,-0.013602057471871376,-0.00868320558220148,-0.058372773230075836,-0.10812583565711975,-0.13474051654338837,-0.11455088108778,0.008404652588069439,-0.03200558200478554,-0.1319243162870407,-0.11161148548126221,-0.05939745530486107,0.02767416462302208,0.0749921202659607,0.022851713001728058,0.036272481083869934,-0.06967185437679291,0.02846973016858101,0.1974567323923111,0.01551448367536068,0.016214841976761818,0.0059390440583229065,-0.04568607732653618,-0.010813171043992043,-0.0047428421676158905,0.054311949759721756,0.06404177099466324,-0.14873483777046204,-0.03692351654171944,0.06925862282514572,0.1723507195711136,-0.005293410737067461,-0.02171170525252819,-0.041950151324272156,-0.04490736126899719,0.024251021444797516,0.06536392867565155,0.17311641573905945,0.009962362237274647,-0.04249872267246246,-0.1392216831445694,-0.16552874445915222,-0.09795787930488586,0.0037509535904973745,0.13377492129802704,-0.09193027764558792,-0.0646156296133995,-0.12105542421340942,-0.1144225150346756,-0.06763661652803421,0.08539871126413345,0.06159791350364685,0.06040746718645096,0.057537276297807693,0.0642307922244072,-0.017599603161215782,0.003625972429290414,0.05859178304672241,-0.035840798169374466,0.03628614544868469,0.01402865257114172,0.06240011006593704,0.04531237110495567,0.053894996643066406,0.13759450614452362,0.0588025264441967,0.015812145546078682,0.06066819652915001,0.07167666405439377,0.09222235530614853,0.028957292437553406,0.011806026101112366,-0.03711068630218506,-0.14884443581104279,-0.12848088145256042,0.00040490913670510054,-0.020810943096876144,-0.0986790880560875,0.04922957718372345,0.08514350652694702,0.07971292734146118,0.06174488365650177,0.06427539139986038,0.04359220713376999,0.03019442781805992,0.04380301758646965,-0.056346453726291656,-0.0611155703663826,-0.12945999205112457,-0.061381809413433075,0.013309904374182224,-0.05808272957801819,0.03629881143569946,0.08136864006519318,0.00511454651132226,-0.02713637799024582,0.035560332238674164,0.1489374041557312,0.10341785848140717,0.03493419662117958,0.011434324085712433,0.04892835393548012,0.00488684419542551,-0.12954874336719513,0.043702632188797,0.037451986223459244,-0.015079713426530361,-0.12210038304328918,-0.044412024319171906,0.048858728259801865,0.053440697491168976,-0.011283982545137405,-0.050180912017822266,-0.09108481556177139,-0.030772540718317032,0.0377366729080677,0.047234710305929184,-0.0810481384396553,-0.06675009429454803,0.012840496376156807,-0.029129186645150185,-0.1694902926683426,-0.14127807319164276,0.001405401504598558,-0.15581925213336945,0.01878003031015396,-0.007625393103808165,0.009379904717206955,-0.030646909028291702,0.04919559508562088,0.013539752922952175,0.07515165954828262,0.022612350061535835,-0.034584905952215195,-0.07960810512304306,-0.06489913910627365,-0.13541054725646973,-0.10304417461156845,-0.04518405348062515,0.11017482727766037,0.023373570293188095,-0.03448480740189552,0.011056873947381973,0.022181851789355278,-0.036563631147146225,0.017587346956133842,-0.07156167179346085,-0.011414003558456898,-0.021710354834794998,0.015775620937347412,0.048001643270254135,-0.07005200535058975,-0.17848315834999084,0.13598963618278503,0.0677933469414711,0.03979182243347168,0.014938291162252426,0.051480989903211594,0.1329612135887146,0.07297953963279724,0.03469929099082947,0.07012854516506195,0.06516328454017639,0.0027875914238393307,-0.021354978904128075,-0.015705520287156105,-0.0406055822968483,0.06179213151335716,0.12438026070594788,-0.05456678569316864,-0.06874247640371323,-0.045782532542943954,0.08739597350358963,-0.030642755329608917,-0.09188491106033325,-0.012403417378664017,-0.04726841300725937,0.0008685413631610572,-0.003311493434011936,-0.02009471133351326,-0.005020785611122847,0.014151716604828835,0.10828910022974014,0.020255396142601967,-0.03125249221920967,0.08866916596889496,0.11001074314117432,0.012183908373117447,-0.06841961294412613,0.02043512649834156,-0.051809150725603104,0.04803317412734032,0.02963363751769066,-0.018851401284337044,0.03596118837594986,-0.031248321756720543,0.012790165841579437,-0.04341353103518486,0.03633419796824455,-0.003088549245148897,0.06766416132450104,0.15280269086360931,0.03396007418632507,-0.04240027815103531,-0.03154545649886131,-0.03473396971821785,0.0332438200712204,-0.008649300783872604,-0.13734598457813263,-0.0003299158997833729,0.02113346941769123,-0.01673104614019394,0.02688489854335785,0.03555537760257721,0.025389160960912704,-0.023068418726325035,-0.10083553940057755,-0.18176548182964325,-0.06988127529621124,0.011173143051564693,-0.020094290375709534,0.004969665315002203,-0.06151116266846657,0.04225043207406998,0.028100693598389626,0.052737072110176086,0.025034910067915916,-0.02682327851653099,-0.021872127428650856,0.023948445916175842,0.09234806150197983,-0.07191278785467148,-0.11700456589460373,-0.01599588245153427,0.003077913075685501,0.030906064435839653,-0.02633015252649784,-0.028560355305671692,-0.03364017233252525,-0.023033350706100464,0.0049221632070839405,-0.030877191573381424,-0.05376066640019417,0.1026827022433281,-0.0041055744513869286,0.005651262588799,0.0024669538252055645,0.021933328360319138,0.08204257488250732,0.0336935892701149,0.013385213911533356,-0.01727588288486004,0.0650334358215332,-0.025252623483538628,-0.05935138463973999,0.04930586367845535,-0.0529504120349884,0.18744681775569916,-0.06690820306539536,-0.041472963988780975,0.04136680066585541,-0.07679907977581024,-0.0004810151003766805,0.0017397792544215918,0.044595859944820404,0.0065228939056396484,0.03306635096669197,0.15728676319122314,0.11796308308839798,-0.031069692224264145,-0.048397041857242584,0.15127120912075043,0.15449903905391693,0.03974253311753273,0.09169473499059677,0.03225221112370491,-0.04663218930363655,0.024243196472525597,0.017505502328276634,-0.09724506735801697,0.030367475003004074,0.07769227027893066,0.05269761383533478,0.06899967044591904,-0.07966965436935425,-0.011763978749513626,0.09998907148838043,-0.07784178107976913,0.17439770698547363,-0.004937492311000824,-0.05858495086431503,-0.07245645672082901,-0.020209120586514473,-0.19898250699043274,-0.015400063246488571,-0.007858390919864178,-0.030711667612195015,0.01416477095335722,-0.006657811813056469,-0.001133755431510508,0.1269870549440384,0.11488324403762817,0.02854072116315365,0.0013855202123522758,-0.037928640842437744,-0.04931703582406044,-0.14328640699386597,-0.16212868690490723,0.022562677040696144,0.05812215432524681,-0.07217410206794739,0.010493995621800423,0.08311375230550766,-0.08732279390096664,0.055869754403829575,0.1350574493408203,0.008864329196512699,-0.010690293274819851,-0.04281744733452797,-0.08093777298927307,-0.22637203335762024,-0.10271812230348587,-0.1027863398194313,-0.031193574890494347,-0.09498869627714157,-0.04066256433725357,0.08300363272428513,-0.07881070673465729,-0.11312038451433182,0.017839917913079262,-0.15909667313098907,-0.07562887668609619,-0.0859854593873024,-0.11197074502706528,-0.2042660415172577,0.01314628403633833,-0.05231134220957756,0.010170449502766132,-0.016462650150060654,0.019452491775155067,-0.00537096057087183,0.06536152958869934,0.009538954123854637,0.009728599339723587,-0.056597307324409485,0.019440947100520134,0.0008662081090733409,-0.021841173991560936,0.009448335506021976,-0.03959783539175987,-0.05497365817427635,-0.037110161036252975,-0.034931913018226624,0.008072791621088982,0.03212785720825195,0.05192439630627632,0.10589205473661423,0.0653638243675232,-0.039065636694431305,0.012170604430139065,-0.02428954280912876,-0.024191975593566895,0.018482379615306854,-0.020093614235520363,-0.026625830680131912,0.009431865997612476,-0.03378426656126976,-0.003961826674640179,0.02665194682776928,0.015434457920491695,0.03028017468750477,0.006116208154708147,0.017777588218450546,0.04479934275150299,0.007034588139504194,0.004798194393515587,0.01067560724914074,-0.04783445969223976,0.04362494498491287,0.06726079434156418,-0.054136212915182114,-0.0039030546322464943,0.0025689085014164448,0.014427555724978447,0.0169814582914114,0.01781422644853592,0.041996557265520096,0.04958942532539368,0.007178923115134239,0.007503713946789503,-0.051527880132198334,0.013176396489143372,0.0009747589356265962,0.042650006711483,-0.09471320360898972,-0.03791474923491478,-0.002437614370137453,0.014582076109945774,-0.001606677076779306,-0.07890372723340988,-0.04190917685627937,0.0290843416005373,0.046609535813331604,-0.02426784858107567,-0.007863329723477364,-0.027780305594205856,-0.06363589316606522,-0.05085725337266922,-0.005375344306230545,-0.06763509660959244,0.00045905017759650946,-0.03554610535502434,-0.02743574045598507,-0.04159349203109741,0.035272806882858276,0.044208306819200516,-0.0010686911409720778,-0.07897388935089111,-0.07902924716472626,-0.04972221702337265,-0.018063398078083992,-0.053871411830186844,-0.025675088167190552,-0.003983817994594574,0.04319941997528076,0.014843734912574291,-0.020565137267112732,-0.10380665212869644,0.007279248908162117,0.04571720212697983,-0.04908803477883339,-0.07261787354946136,0.004860484506934881,-0.001955582993105054,-0.019465595483779907,0.013621271587908268,-0.04169400781393051,0.0033617690205574036,0.07707031071186066,-0.020136389881372452,-0.006169149652123451,-0.017435185611248016,-0.0061288755387067795,0.006093435455113649,0.05333429202437401,-0.03854496404528618,0.016507474705576897,-0.019221777096390724,-0.02442590519785881,-0.006480827461928129,0.013112696819007397,-0.00031730555929243565,-0.019972477108240128,-0.02501421421766281,-0.0026718752924352884,-0.038745902478694916,0.024822937324643135,-0.004744494799524546,0.06945908069610596,0.0016361582092940807,-0.03171399608254433,-0.06492128223180771,-0.012721541337668896,-0.009337627328932285,-0.009806973859667778,0.029156746342778206,0.04971233010292053,0.038933780044317245,-0.04515156149864197,-0.005303904879838228,0.03906507045030594,-0.0416407436132431,0.04483933746814728,0.014110509306192398,0.017692619934678078,-0.0775514468550682,-0.06381674110889435,-0.048899710178375244,0.012080149725079536,0.0157786812633276,0.05051523074507713,0.031924180686473846,-0.014112585224211216,-0.013250037096440792,-0.050072573125362396,-0.08791246265172958,-0.06067598611116409,-0.029855865985155106,0.02446402981877327,-0.06512537598609924,-0.013006542809307575,0.02043825201690197,-0.042055893689394,0.02746228501200676,0.005649605765938759,2.7217498427489772e-05,0.03188091889023781,-0.042688678950071335,0.00546632893383503,0.010155441239476204,-0.03914346173405647,0.00013188857701607049,0.05931873619556427,-0.08569521456956863,-0.03626643866300583,-0.004431984852999449,-0.024162733927369118,-0.006357025355100632,-0.009693099185824394,-0.03588087111711502,0.005995928309857845,-0.025556262582540512,-0.028431029990315437,0.004758470691740513,0.016849447041749954,0.016207976266741753,-0.004486673045903444,0.013858335092663765,0.015533313155174255,-0.056849218904972076,0.01686595007777214,-0.03818539157509804,-0.023060863837599754,-0.014870532788336277,0.03189728409051895,0.011052598245441914,-0.027322333306074142,-0.03031531721353531,0.0032577887177467346,-0.010329769924283028,0.08580754697322845,-0.12250273674726486,-0.04033956304192543,-0.049413833767175674,-0.03836117312312126,-0.01900199055671692,0.06398293375968933,-0.028681088238954544,0.0544709637761116,0.02519003115594387,-0.00941898301243782,-0.06221456453204155,-0.04819786548614502,-0.03873874247074127,0.020504234358668327,-0.09134126454591751,-0.03456462547183037,-0.021580712869763374,-0.0038042443338781595,-0.013249751180410385,-0.05272991210222244,-0.05629291757941246,-0.04381297528743744,-0.05436853691935539,-0.07315228879451752,-0.0340600423514843,-0.06032417342066765,-0.08271251618862152,-0.011405689641833305,-0.060046348720788956,-0.019073236733675003,0.07924039661884308,0.1261328160762787,0.04489314183592796,0.04303693026304245,-0.01522736344486475,-0.0018437933176755905,0.020409831777215004,0.09881947189569473,0.05851132795214653,0.07808662950992584,-0.02623330056667328,0.02360985055565834,0.0020975801162421703,0.03126216679811478,0.04156435281038284,0.040012698620557785,0.06201532110571861,-0.00569293973967433,-0.09164118021726608,-0.020593488588929176,0.010126018896698952,0.004170219413936138,0.049635790288448334,0.02510823681950569,-0.01196246687322855,-0.022121792659163475,0.024256683886051178,0.10075829923152924,0.08489729464054108,0.14105834066867828,0.025200463831424713,-0.002243897644802928,-0.06297881156206131,-0.07046806067228317,-0.06523226201534271,0.09154095500707626,0.07336527854204178,0.045221734791994095,0.026011191308498383,0.08264877647161484,0.06568284332752228,0.04013809189200401,0.02425992861390114,-0.004302362445741892,0.009035026654601097,0.15891610085964203,0.0045212083496153355,0.01301492564380169,-0.035264354199171066,-0.014471958391368389,0.02669314667582512,0.08841872215270996,0.09087300300598145,0.03977971151471138,-0.0051803747192025185,-0.0023509091697633266,0.04063724726438522,0.05511701852083206,0.037407685071229935,0.017482517287135124,-0.013306585140526295,-0.053638942539691925,0.02834920398890972,0.015434829518198967,0.025382811203598976,-0.001360389986075461,0.018657797947525978,0.009305991232395172,0.06141671538352966,-0.007051090244203806,-0.10855226218700409,-0.05000830069184303,-0.004464687313884497,-0.0422343984246254,-0.14845243096351624,-0.12690369784832,-0.1158733144402504,-0.053779132664203644,-0.043510206043720245,0.020295623689889908,0.058385808020830154,0.035985417664051056,-0.11940715461969376,-0.0938558354973793,-0.11027221381664276,-0.11513402312994003,-0.06156216561794281,0.008825695142149925,-0.06944099813699722,0.0037131360732018948,-0.08248285949230194,0.0038505324628204107,-0.038412269204854965,-0.0046956585720181465,-0.01838294044137001,-0.07940461486577988,-0.04278590530157089,-0.08442722260951996,-0.0710858553647995,-0.06677411496639252,0.02483547478914261,-0.12085015326738358,-0.1562604457139969,-0.09576781094074249,-0.0716094821691513,0.02925563044846058,-0.03553449362516403,-0.037984833121299744,-0.061125461012125015,-0.042977117002010345,-0.11506450176239014,-0.061940886080265045,-0.05804372578859329,-0.08853314816951752,-0.011640015989542007,-0.1022474467754364,-0.08142673969268799,-0.03194539248943329,-0.06165948882699013,-0.07523589581251144,-0.017170874401926994,-0.047371797263622284,-0.06584621965885162,-0.005194730591028929,-0.050403892993927,-0.046572454273700714,-0.08742263168096542,-0.05227698013186455,-0.08055375516414642,-0.10866831243038177,-0.08632395416498184,-0.10749407112598419,-0.04062456265091896,-0.02571324072778225,-0.06355737149715424,0.0032670681830495596,0.035685811191797256,0.05545521900057793,-0.18671320378780365,-0.13287855684757233,-0.051723070442676544,-0.004728944040834904,-0.033758364617824554,-0.026406358927488327,-0.04927242919802666,-0.0033243908546864986,-0.048059552907943726,0.029094086959958076,0.00130895571783185,-0.027566108852624893,-0.06775952130556107,-0.07878291606903076,-0.13431033492088318,-0.125599205493927,-0.04060472920536995,-0.014093209989368916,-0.0018656260799616575,0.007896370254456997,-0.024546628817915916,-0.026100246235728264,-0.0746709406375885,-0.010082609951496124,-0.01076708734035492,-0.03020843304693699,-0.05218056961894035,-0.04633475840091705,0.008589384146034718,0.03734470531344414,0.015160306356847286,0.017545480281114578,-0.04596131667494774,-0.03476141020655632,-0.04821382090449333,-0.0389857180416584,-0.0417022630572319,0.03858724236488342,0.033860787749290466,0.058321017771959305,0.08713825047016144,0.04030349478125572,-0.10652285814285278,0.007189718075096607,0.06892621517181396,0.029393063858151436,-0.0618068091571331,-0.06406919658184052,-0.07331569492816925,0.024022776633501053,-0.030953150242567062,0.08429057896137238,0.01366119273006916,0.04862377792596817,0.02985912747681141,0.08127879351377487,-0.11090540885925293,0.03141187131404877,0.0361376591026783,0.1023038774728775,0.10093189775943756,0.031940609216690063,-0.07671760022640228,-0.07434796541929245,0.06296345591545105,0.08993247896432877,0.011544333770871162,-0.025765612721443176,0.0160656850785017,-0.01241716556251049,-0.12281523644924164,-0.09637689590454102,-0.09455648809671402,0.013826023787260056,0.14372360706329346,0.04904783144593239,-0.008800804615020752,-0.0009680018411017954,0.026278745383024216,0.03439312428236008,-0.0475858636200428,-0.019422776997089386,0.017783576622605324,-0.08140367269515991,0.03413929417729378,0.02239924483001232,-0.04776300489902496,-0.02207973226904869,0.14875119924545288,0.11535576730966568,0.01208733394742012,0.06653965264558792,0.012286853976547718,-0.004674625117331743,-0.0871216282248497,-0.015147903934121132,-0.02700721099972725,0.002698279917240143,0.01371874287724495,0.04295102134346962,-0.020933378487825394,-0.02023446373641491,0.034723371267318726,0.16257962584495544,0.01287353876978159,0.038989488035440445,0.0918131172657013,-0.18714676797389984,-0.07485894113779068,0.06599877029657364,0.0783901959657669,-0.017510637640953064,-0.016233976930379868,0.030635498464107513,0.008073627017438412,-0.01231672428548336,-0.10808462649583817,-0.008971068076789379,-0.11243915557861328,0.027925219386816025,0.15044493973255157,-0.06998034566640854,-0.018532557412981987,0.0035014350432902575,0.048290323466062546,0.009511645883321762,0.011142197996377945,-0.08716529607772827,-0.06365469843149185,-0.16420772671699524,0.04421934857964516,-0.03588532283902168,-0.03802841901779175,0.015739740803837776,0.05428696423768997,0.1542029082775116,0.015912657603621483,-0.014383831061422825,0.014796285890042782,0.056768909096717834,0.05325112119317055,-0.037073537707328796,0.012269146740436554,-0.004275083541870117,-0.029149074107408524,0.04964599758386612,0.0629754364490509,0.10783319175243378,0.10103945434093475,0.17718729376792908,0.13714520633220673,-0.00481259124353528,-0.022784817963838577,0.01411521714180708,0.045294370502233505,0.005808611866086721,0.021107522770762444,0.06420772522687912,-0.05217907205224037,-0.10052712261676788,-0.003628193400800228,0.054300591349601746,0.025555972009897232,0.07557667791843414,0.14639605581760406,0.126198410987854,-0.0267572533339262,0.0459805503487587,-0.0460599809885025,0.04353751614689827,-0.02572077326476574,-0.10223560780286789,-0.027674661949276924,-0.005491153337061405,0.04585658013820648,0.12205081433057785,0.006429081317037344,0.11845742911100388,0.12513332068920135,0.048545971512794495,0.04561443626880646,0.017730725929141045,-0.02213118225336075,0.05291975662112236,0.07409393042325974,0.0008536580135114491,0.009784727357327938,0.10411114245653152,-0.04728870093822479,0.04720710963010788,0.12894947826862335,0.032735489308834076,0.05825326219201088,-0.04674473777413368,0.05356971547007561,0.03863903880119324,-0.03963783383369446,0.015021917410194874,0.08801472932100296,-0.021379366517066956,0.04666316509246826,0.10019577294588089,0.020996665582060814,-0.0019967970438301563,-0.027649808675050735,-0.0064627667888998985,-0.00782075710594654,-0.07990604639053345,-0.04019206017255783,-0.08606574684381485,0.04200805723667145,0.027904145419597626,0.0583990104496479,0.11558336764574051,0.03435995429754257,0.08112004399299622,0.061776913702487946,-0.004067567177116871,-0.09343573451042175,-0.0966407060623169,-0.02727571129798889,0.10199931263923645,-0.06801256537437439,-0.03428760543465614,-0.018060855567455292,0.06740564852952957,0.012955822050571442,-0.029178863391280174,-0.08203072100877762,0.005779880098998547,-0.07846959680318832,0.041330333799123764,-0.0494610071182251,0.08843518048524857,0.02020660974085331,0.052237607538700104,-0.04150744900107384,-0.09106284379959106,0.007899359799921513,0.08600693196058273,0.03988984227180481,0.038719542324543,0.04563930258154869,-0.0608140230178833,0.00021564908092841506,0.03067992627620697,0.09947196394205093,0.051032938063144684,0.037827856838703156,0.056539978832006454,-0.012959646992385387,-0.001642731949687004,-0.0065504033118486404,0.07444483786821365,0.056387219578027725,0.07259519398212433,0.026894060894846916,-0.0579097643494606,-0.022528311237692833,0.05184083804488182,0.08155002444982529,0.07722830027341843,0.008099007420241833,0.023669539019465446,0.006709488574415445,-0.015811001881957054,0.048298198729753494,0.018323007971048355,0.0364704504609108,0.0014255172573029995,0.03992479294538498,-0.004948524292558432,0.0177024956792593,0.000640151381958276,-0.005104857962578535,0.003932001534849405,0.06551692634820938,0.03576846420764923,0.022884896025061607,0.041192587465047836,0.06101212278008461,0.06399024277925491,-0.012131576426327229,0.00579878082498908,0.026266051456332207,-0.013774937950074673,0.06941467523574829,0.06372884660959244,-0.05286040157079697,0.0009949181694537401,0.05033562704920769,0.10056913644075394,-0.007911485619843006,0.025730181485414505,0.05719028413295746,0.02940482832491398,0.016515817493200302,0.017167937010526657,0.07185585051774979,-0.022954657673835754,-0.024368582293391228,0.07931744307279587,0.0005613206885755062,-0.1305817812681198,-0.04554925113916397,0.053233835846185684,-0.007958832196891308,-0.025240810588002205,0.03285129740834236,-0.03278717026114464,-0.05869857966899872,0.03284073993563652,-0.020216161385178566,-0.017481744289398193,-0.008107278496026993,0.22133152186870575,0.058531343936920166,-0.079330213367939,0.01519311498850584,0.026451196521520615,-0.01894070766866207,-0.06936664134263992,-0.018179146572947502,-0.00024122813192661852,-0.011137563735246658,-0.07084926217794418,-0.0959220752120018,-0.036996785551309586,0.002435990609228611,0.19045262038707733,0.05209631845355034,-0.05235244706273079,-0.01846158504486084,0.05736367031931877,0.023851320147514343,-0.006209714338183403,0.002269667573273182,-0.0031807003542780876,-0.023515410721302032,-0.06776302307844162,0.014232538640499115,-0.023197976872324944,-0.021670479327440262,0.10693473368883133,-0.0463542714715004,0.029038166627287865,0.039917413145303726,0.05395413562655449,-0.0029329799581319094,-0.00466932775452733,0.02671564370393753,-0.007014785893261433,-0.017149556428194046,0.023307258263230324,0.10291288048028946,-0.01692447066307068,0.08609180152416229,0.09757442772388458,0.036794617772102356,-0.002220262074843049,0.02668730542063713,-0.012308137491345406,-0.004573257174342871,-0.036310408264398575,-0.06525877118110657,-0.07396895438432693,-0.13542091846466064,0.028596187010407448,0.08394362777471542,-0.013935515657067299,-0.0004513389430940151,0.014958984218537807,-0.05558047071099281,0.049552351236343384,0.03297537565231323,-0.04235384613275528,-0.07660654932260513,-0.05216134712100029,-0.2245485931634903,-0.1317720115184784,-0.0717250406742096,0.004652082920074463,0.01073586568236351,0.05005085840821266,-0.0727759525179863,0.03695867583155632,-0.007161160930991173,0.06960439682006836,0.09596463292837143,-0.04854918643832207,0.018665893003344536,-0.06221536546945572,-0.1799703985452652,-0.025838105008006096,0.02967522107064724,0.02093183621764183,0.035972196608781815,0.006277794484049082,-0.008064890280365944,-0.014455800876021385,0.0616297610104084,0.07462599128484726,-0.010526754893362522,-0.021872779354453087,-0.03494077920913696,-0.011524718254804611,-0.18909405171871185,-0.07442891597747803,0.048295024782419205,0.08741716295480728,0.09228454530239105,0.1205899640917778,0.05710751563310623,0.021904105320572853,-0.007331425324082375,-0.05293511599302292,-0.0009041476878337562,0.031068814918398857,-0.043684545904397964,-0.009435894899070263,0.16103997826576233,0.09785091131925583,0.02214992791414261,0.03620174527168274,0.05169244110584259,0.06488087773323059,0.02879146672785282,0.10655676573514938,0.12974947690963745,0.04303227737545967,0.0828375518321991,0.10075567662715912,0.082746721804142,0.0682351142168045,0.09184341877698898,0.09798279404640198,-0.002202207688242197,-0.06342713534832001,-0.05161654204130173,0.005011121276766062,0.04356963559985161,-0.007071915548294783,-0.03381255269050598,-0.004942349623888731,0.0037787477485835552,-0.021367084234952927,0.022482912987470627,0.06417068839073181,0.004667794797569513,0.08335048705339432,0.05165569856762886,-0.052234064787626266,-0.08731567114591599,-0.059828151017427444,0.015128975734114647,0.06386631727218628,0.0077408007346093655,0.038840316236019135,0.006202595308423042,0.043299850076436996,0.0376502200961113,0.005412976257503033,0.16806142032146454,0.11366581171751022,0.03219220042228699,-0.07024138420820236,-0.1539207100868225,-0.11610087007284164,0.027940725907683372,-0.0021243642549961805,-0.02246950939297676,0.03642774000763893,0.06647603958845139,0.044962555170059204,0.0715753361582756,0.017894694581627846,0.09661589562892914,0.05354853346943855,-0.008212742395699024,-0.12314732372760773,-0.13879473507404327,-0.1460319310426712,0.07526640594005585,0.06447450816631317,-0.0503716766834259,-0.06276742368936539,0.026369107887148857,0.03198302164673805,-0.03538301959633827,0.042876504361629486,0.02631140686571598,0.023343022912740707,-0.016017643734812737,-0.13669095933437347,-0.1443103849887848,-0.20366798341274261,0.04371541738510132,0.07641000300645828,-0.11888986080884933,-0.11464719474315643,-0.045228004455566406,0.012320375069975853,-0.036247462034225464,0.0011780073400586843,0.050523608922958374,0.047365400940179825,-0.038012370467185974,-0.11556097865104675,-0.1343722939491272,-0.06697573512792587,0.025378836318850517,0.12119701504707336,-0.032030027359724045,-0.06136360391974449,0.0010856693843379617,0.04709114134311676,0.00010050530545413494,0.01808716356754303,-0.0096025001257658,-0.018328551203012466,-0.037598010152578354,0.002359034027904272,-0.08690164983272552,0.05507570877671242,0.01931055821478367,0.06575509905815125,0.05770047381520271,-0.024715246632695198,0.01249495055526495,0.02922912873327732,-0.008949434384703636,0.05369323119521141,-0.07635810226202011,-0.021986257284879684,-0.05915231257677078,-0.06425334513187408,-0.06769445538520813,-0.03682573884725571,-0.05802207067608833,0.06169651076197624,0.004708150401711464,-0.048151273280382156,0.02591024711728096,-0.019209859892725945,0.06390450894832611,0.07703513652086258,-0.22300602495670319,-0.0994776263833046,-0.02909490466117859,0.011119101196527481,-0.009462844580411911,0.05624430254101753,0.031677816063165665,0.03684298321604729,-0.0017020925879478455,-0.026572147384285927,0.08039940148591995,0.0011395207839086652,-0.06268636137247086,-0.04860638827085495,-0.2217799574136734,-0.2450953722000122,-0.11864965409040451,0.00645738048478961,0.018760330975055695,-0.004932922776788473,-0.12603211402893066,-0.15062490105628967,-0.04363198205828667,-0.02898605354130268,-0.04682500660419464,-0.011242455802857876,-0.14468155801296234,-0.1527663767337799,-0.20740431547164917,-0.18287678062915802,-0.09834033250808716,-0.028315622359514236,0.016051147133111954,-0.014604691416025162,-0.06868990510702133,-0.14341527223587036,-0.0028804768808186054,0.01619833894073963,-0.00669547775760293,-0.06885190308094025,-0.1180691048502922,-0.1409887969493866,-0.21799558401107788,-0.07785632461309433,-0.07777148485183716,0.041584890335798264,-0.05346808582544327,-0.02995472401380539,-0.09480615705251694,-0.06082650274038315,-0.09404125809669495,-0.0063298167660832405,0.014495286159217358,-0.03254832699894905,-0.0604369193315506,-0.10427391529083252,-0.20063139498233795,-0.12319669127464294,-0.02880869060754776,-0.08607065677642822,-0.07213772088289261,-0.02930345945060253,-0.11697211861610413,-0.13537149131298065,-0.042592063546180725,-0.048803891986608505,-0.07547253370285034,-0.03671244531869888,-0.11239833384752274,-0.1675691306591034,0.12628760933876038,0.056180376559495926,0.011078236624598503,0.00576257286593318,-0.009049469605088234,0.018446162343025208,-0.04183201864361763,-0.05958133935928345,-0.04645238816738129,0.055804088711738586,0.17369116842746735,0.1675444394350052,0.15178120136260986,0.10599309206008911,0.0587560310959816,-0.07129421830177307,-0.014411352574825287,-0.020489851012825966,-0.05515405908226967,-0.09286288172006607,-0.1559138000011444,-0.13904765248298645,-0.026872199028730392,0.02759641222655773,0.06713854521512985,0.15459561347961426,0.08211160451173782,0.017744461074471474,0.10236889868974686,-0.04783790558576584,0.02143286168575287,0.05699098855257034,-0.04844115301966667,-0.08950237184762955,-0.09317142516374588,-0.1706623136997223,-0.08809571713209152,0.006781493779271841,0.08296433091163635,0.15876372158527374,0.12031583487987518,0.01491608191281557,0.11783371865749359,0.07089241594076157,0.04726947098970413,0.02892165072262287,0.006650565657764673,0.05708344280719757,0.03376343846321106,-0.014392494224011898,0.042206376791000366,0.018312208354473114,0.061735961586236954,0.13431327044963837,0.20032581686973572,0.08638089150190353,0.011735690757632256,0.03653939440846443,0.04173051565885544,0.05647105723619461,-0.013521322049200535,0.10450360924005508,0.11752768605947495,0.1512327641248703,0.1324017345905304,0.0659036636352539,0.1120956540107727,0.0583559013903141,-0.01664518006145954,0.013330412097275257,0.010219486430287361,0.08407694846391678,0.079383485019207,0.06097541004419327,0.003205203451216221,-0.08705101907253265,-0.007837465032935143,0.14499531686306,0.1413910835981369,0.06275219470262527,0.06600213795900345,0.07378817349672318,-0.006389226298779249,-0.06419334560632706,0.042659658938646317,0.031025176867842674,0.001759305945597589,-0.05339423194527626,-0.0829031765460968,-0.033378154039382935,0.03129925951361656,0.08625761419534683,0.04666402190923691,0.028015319257974625,0.04460252821445465,0.06872738897800446,0.036027297377586365,0.013341959565877914,-0.03367900475859642,0.05544808879494667,-0.04955601692199707,-0.15817773342132568,-0.11326107382774353,0.06318733096122742,0.19247862696647644,0.06310105323791504,-0.0171969011425972,-0.03494459018111229,0.040439423173666,0.03908228129148483,0.05540487542748451,-0.02060871385037899,0.012843696400523186,0.02260880172252655,0.04923145845532417,0.05411643162369728,-0.008676992729306221,0.009071728214621544,0.06895306706428528,-0.0060631087981164455,-0.01653900556266308,0.002422293880954385,0.06637005507946014,0.028734466060996056,0.03468324989080429,0.04755236953496933,0.01564531773328781,0.0617479532957077,0.14497394859790802,0.13120827078819275,0.04446331411600113,-0.07320711016654968,-0.006306501105427742,0.05189969390630722,0.05426252260804176,0.12083333730697632,0.08004284650087357,0.14363601803779602,0.0651477500796318,0.06412836909294128,0.009813901036977768,0.05525682866573334,0.08643955737352371,0.02020648680627346,1.7964339349418879e-06,0.011450427584350109,-0.006877379957586527,0.02118227444589138,0.021636448800563812,-0.0013884739018976688,0.02213846519589424,0.010514209978282452,-0.02655978500843048,-0.11249081045389175,-0.07345112413167953,-0.07880215346813202,0.00369607238098979,0.010864169336855412,0.025726020336151123,0.10003679245710373,0.10901235044002533,0.09710362553596497,0.0010786789935082197,-0.015681155025959015,0.019212789833545685,0.02929738722741604,-0.05610869452357292,-0.18015293776988983,-0.01764722168445587,0.026558293029665947,-0.017934884876012802,0.007842299528419971,0.10351944714784622,0.07429073005914688,0.14632299542427063,0.130643829703331,0.009944122284650803,0.06515993922948837,0.05861533433198929,0.008545644581317902,-0.0894119143486023,-0.1959969699382782,0.016244899481534958,-0.0019177538342773914,-0.033643897622823715,-0.042287006974220276,-0.05123183876276016,0.04584098979830742,0.06136911362409592,0.03863244876265526,-0.038140669465065,-0.006227430421859026,0.043619025498628616,-0.02140524797141552,-0.13753940165042877,-0.21461449563503265,-0.07618885487318039,-0.06106538325548172,-0.034223053604364395,-0.04584256932139397,-0.04197392985224724,-0.002852697391062975,0.002407944994047284,0.016478683799505234,-0.000580521416850388,-0.005314473062753677,-0.06571364402770996,-0.027441341429948807,-0.09055116027593613,-0.09836343675851822,-0.07266101986169815,-0.03568556159734726,-0.025154942646622658,-0.04570788890123367,0.06315254420042038,0.07855627685785294,0.026631727814674377,0.07775924354791641,0.07002447545528412,0.024478981271386147,-0.02496982179582119,-0.0018522700993344188,-0.09516013413667679,-0.13055887818336487,-0.04261260852217674,0.0007629868923686445,-0.007342638913542032,-0.06921347230672836,-0.04680951312184334,0.06170554459095001,0.013005993328988552,0.053884562104940414,0.06520382314920425,0.03402965888381004,-0.06549517810344696,-0.03710081800818443,-0.042507655918598175,-0.14717252552509308,-0.09667731076478958,-0.008078174665570259,-0.05052253603935242,-0.12440391629934311,-0.09289350360631943,0.026294177398085594,-0.017872489988803864,-0.024486271664500237,-0.045549243688583374,-0.04275088384747505,-0.08405488729476929,-0.09722527861595154,-0.08361455798149109,-0.06804794073104858,-0.06897013634443283,-0.08759525418281555,-0.11513353884220123,-0.08430605381727219,-0.05071856081485748,0.027713285759091377,-0.003745416644960642,-0.0011719351168721914,-0.12610140442848206,-0.06648463755846024,-0.04167768731713295,-0.08114985376596451,-0.10783179104328156,-0.06763045489788055,-0.13033288717269897,-0.06530141830444336,-0.08780789375305176,-0.10584443062543869,-0.018368538469076157,-0.06466886401176453,-0.06513655185699463,-0.038266852498054504,-0.04304642975330353,-0.09484878927469254,-0.0952744111418724,-0.06621164083480835,-0.09423349797725677,-0.13655367493629456,-0.03203872591257095,-0.008667873218655586,-0.10449420660734177,-0.09064283221960068,-0.05362764745950699,-0.030308255925774574,-0.05272521823644638,-0.01667776145040989,-0.08432485163211823,-0.041684575378894806,-0.05780549347400665,-0.08171649277210236,-0.04166534170508385,-0.014166589826345444,-0.03998323529958725,-0.041991326957941055,-0.02127034030854702,-0.040086567401885986,-0.05885355547070503,-0.026935424655675888,0.00683369766920805,0.04651441425085068,-0.02754947543144226,-0.043784406036138535,-0.014617988839745522,-0.004600438289344311,-0.04345781356096268,-0.10052203387022018,-0.019189614802598953,0.04679979756474495,-0.042999595403671265,-0.04952739179134369,-0.02620680443942547,-0.05401543155312538,-0.0314074270427227,0.0015506719937548041,-0.0476936511695385,-0.028159121051430702,0.03096833825111389,0.03044598177075386,0.04164789617061615,-0.021825458854436874,-0.06509225815534592,-0.05722613260149956,-0.02226441167294979,-0.053014785051345825,0.038207024335861206,-0.000577351194806397,0.0030287988483905792,0.05831314995884895,-0.013183053582906723,0.014768881723284721,0.010316789150238037,-0.005830476991832256,0.01912832446396351,-0.013832766562700272,-0.0188677366822958,-0.033456381410360336,-0.053486261516809464,0.037134453654289246,0.009636935777962208,0.05677265301346779,0.012498404830694199,0.015607575885951519,0.01584239862859249,0.040826376527547836,0.007543426007032394,0.01618998311460018,0.014964167959988117,-0.03780267760157585,-0.016229037195444107,-0.03258968144655228,-0.004079043865203857,-0.02563055418431759,0.06129296496510506,0.02336031198501587,0.003521502949297428,-0.02125481329858303,0.0019120208453387022,0.06562762707471848,0.04961392655968666,0.004253495018929243,-0.018915513530373573,0.03587570786476135,-0.0016991428565233946,0.027614062651991844,0.05880661681294441,0.046732738614082336,0.08426600694656372,0.019915640354156494,-0.02196699194610119,-0.010481084696948528,0.03606032207608223,0.014044017530977726,-0.011172767728567123,0.00869560707360506,0.0010300592985004187,0.06786107271909714,0.04361233115196228,0.03734970465302467,0.06959790736436844,0.025128474459052086,0.03679206594824791,0.09799985587596893,-0.008084201253950596,-0.010708926245570183,0.061984434723854065,0.02115575410425663,0.0138043612241745,0.061332207173109055,0.0833371952176094,0.10270649194717407,0.028542760759592056,0.08318367600440979,0.10990643501281738,0.0608813539147377,0.05663198605179787,0.03243013843894005,0.03746316209435463,0.023473123088479042,0.023206958547234535,0.021426470950245857,0.05558974668383598,0.04485953226685524,0.02031536214053631,-0.002402127720415592,0.07492747157812119,0.04200831055641174,0.03350116312503815,-0.002328663831576705,0.11745121330022812,0.09357213228940964,0.04888501390814781,-0.009416154585778713,0.1599537581205368,0.11193320155143738,0.06392211467027664,0.06341317296028137,-0.03051961213350296,0.02782890386879444,0.01832675188779831,0.0024160260800272226,-0.08235480636358261,-0.06643271446228027,0.0068078539334237576,0.061697956174612045,-0.01634214259684086,0.03787732496857643,0.05193329602479935,0.06878756731748581,0.004834233783185482,-0.020791657269001007,0.03916686028242111,0.026008201763033867,0.043489087373018265,0.10011941939592361,0.010058092884719372,0.0002420495729893446,0.05146448686718941,0.0510430783033371,-0.03337083011865616,-0.015748485922813416,-0.03975724056363106,-0.005011319182813168,-0.007469429634511471,0.05195271968841553,0.12094028294086456,0.004005781840533018,-0.022981397807598114,0.04004473611712456,0.052080340683460236,0.09236889332532883,-0.009283654391765594,0.01215396262705326,0.014396191574633121,0.028108054772019386,-0.03826645016670227,-0.005477627739310265,0.035781096667051315,0.02235116995871067,0.02971009723842144,0.006069671828299761,-0.01822725124657154,0.05820607766509056,0.006714900955557823,0.08269388228654861,0.07194837927818298,-0.02484670653939247,-0.0936594158411026,-0.056044768542051315,-0.002655257238075137,0.09262413531541824,-0.01880815625190735,-0.05718051269650459,0.0485028475522995,-0.07022683322429657,0.014842638745903969,0.09351979941129684,0.026038071140646935,0.025666289031505585,0.08335792273283005,-0.04455891624093056,-0.17585903406143188,-0.11951974034309387,-0.02833477407693863,0.05253778025507927,-0.04611513763666153,0.015093058347702026,0.046091172844171524,0.035810552537441254,-0.012139683589339256,-0.004373590927571058,-0.06633307039737701,0.017321856692433357,0.00932212546467781,0.06430588662624359,0.038308922201395035,0.04674534127116203,0.019288772717118263,0.037261102348566055,-0.0049641127698123455,-0.06560590118169785,-0.06379509717226028,-0.06164299324154854,0.10296265035867691,0.12588930130004883,0.03468722477555275,0.039900362491607666,-0.04088035970926285,0.032455187290906906,0.03846028819680214,0.07541953027248383,0.04698650538921356,-0.0022152091842144728,0.00765525596216321,-0.052542608231306076,0.03842229023575783,0.07590050995349884,0.12979209423065186,0.12422974407672882,0.14246083796024323,-0.00931501667946577,0.007489795330911875,0.07762221246957779,0.05433133617043495,0.0936584547162056,0.08815319836139679,0.004446058534085751,-0.05321979150176048,-0.01073287334293127,0.07008009403944016,0.06590469926595688,0.22015361487865448,0.14304357767105103,0.12485381960868835,0.10675191879272461,0.05134905129671097,0.0904252752661705,0.13730503618717194,0.14128649234771729,0.11439263820648193,0.07322487980127335,0.04419756308197975,0.09911896288394928,0.1366187185049057,0.12820874154567719,0.19888344407081604,0.08821672201156616,0.06183570995926857,0.037458714097738266,0.035767097026109695,0.029699217528104782,0.04935402423143387,0.08451655507087708,-0.010030508041381836,0.019758129492402077,-0.02722949907183647,-0.009901203215122223,0.06316839158535004,0.10624611377716064,0.10040459781885147,0.08431214094161987,0.05050854757428169,0.010098522529006004,0.09794274717569351,0.03068469651043415,0.031007472425699234,0.05248864367604256,0.05141320824623108,0.027191953733563423,0.0014682520413771272,0.04823390021920204,0.0697162002325058,0.09615206718444824,0.20247140526771545,0.04941384121775627,0.07074083387851715,0.10689384490251541,0.08449549227952957,0.0863756462931633,0.0015790300676599145,0.07201619446277618,0.1249968409538269,0.0919974073767662,0.08181710541248322,0.053891535848379135,0.09990178048610687,0.11525437235832214,0.15635769069194794,0.13923662900924683,0.12463391572237015,0.04282901808619499,-0.03884626924991608,-0.00562302814796567,-0.00764470687136054,-0.06251884996891022,0.0371723435819149,0.048515111207962036,-0.06923219561576843,0.03410690277814865,0.049801770597696304,-0.11755366623401642,0.1654559075832367,0.13774260878562927,-0.0034265322610735893,0.06546245515346527,0.14589616656303406,0.08837739378213882,0.09576558321714401,-0.0035337284207344055,0.06132589280605316,0.07745280116796494,-0.045058317482471466,-0.01383915264159441,0.026275863870978355,0.040223363786935806,0.09406676143407822,0.11527150869369507,0.001821368234232068,-0.04831864684820175,0.1630415916442871,0.18519294261932373,0.11065845191478729,-0.021673841401934624,0.08412355929613113,0.0002532338257879019,-0.09559846669435501,-0.08334238082170486,-0.09654631465673447,0.07415662705898285,0.0022545158863067627,-0.023551741614937782,-0.054279159754514694,-0.047112591564655304,0.09140139818191528,0.2502175569534302,0.15621067583560944,0.0433308444917202,0.14247828722000122,-0.07307806611061096,-0.06893212348222733,-0.003424291266128421,-0.03692370653152466,-0.03822669759392738,-0.130440816283226,-0.12547050416469574,-0.01454996969550848,-0.02459390088915825,0.13915510475635529,0.11611257493495941,0.09847850352525711,0.04055129364132881,0.05406451225280762,-0.0022435388527810574,-0.08367078006267548,-0.06746262311935425,-0.0035040404181927443,-0.09303582459688187,0.12803436815738678,0.06743522733449936,0.04893123358488083,0.050935275852680206,0.19544042646884918,0.07541383802890778,0.03218769282102585,0.15456943213939667,-0.022057686001062393,-0.021746627986431122,0.0059333317913115025,-0.01443441491574049,0.05018626153469086,0.010741924867033958,0.03287121653556824,0.16888989508152008,0.260198175907135,0.17622998356819153,-0.07156695425510406,0.1486087590456009,0.09392323344945908,0.15862664580345154,0.09618833661079407,0.08350706100463867,0.05321936309337616,0.003274609101936221,0.029702015221118927,0.007057139649987221,-0.0002237640437670052,0.0999322235584259,0.20920772850513458,0.21540053188800812,0.16881120204925537,0.11107055842876434,0.22713366150856018,0.1650668978691101,0.026889963075518608,0.1393919736146927,0.0652264803647995,0.03433560952544212,0.05676506459712982,0.048245880752801895,0.07997892051935196,0.15832868218421936,0.19993290305137634,0.04247711971402168,0.2377631515264511,0.09965747594833374,0.20277906954288483,0.12670350074768066,0.08960742503404617,0.07155868411064148,0.011954693123698235,-0.09424738585948944,0.006503551732748747,0.051915813237428665,0.10815446078777313,0.12756229937076569,0.1465577334165573,0.10199844092130661,0.1087794303894043,0.20171120762825012,0.20890852808952332,0.14463499188423157,0.0650215744972229,0.10882318019866943,0.18036997318267822,0.07237837463617325,0.13493524491786957,0.09333302825689316,0.15099266171455383,0.12032831460237503,0.19941258430480957,0.07559487968683243,0.1481764316558838,0.05578058585524559,0.09361932426691055,0.20252646505832672,-0.04165693745017052,0.051335278898477554,0.11219867318868637,0.12834584712982178,0.16938091814517975,0.15435339510440826,0.11974938958883286,0.1610296666622162,0.09045267850160599,0.06322059780359268,0.09700052440166473,0.023205379024147987,-0.029049765318632126,0.22481754422187805,0.1567106395959854,0.05477241426706314,0.0581948459148407,0.16722549498081207,0.10705865919589996,0.050929583609104156,0.06030008941888809,0.10712510347366333,0.08414868265390396,0.037020981311798096,0.0024908927734941244,0.11825081706047058,0.01000076625496149,0.04448569938540459,0.08409550786018372,-0.0033057297114282846,-0.05347830057144165,0.062245093286037445,0.07200422137975693,-0.08562420308589935,0.11665424704551697,0.07905872166156769,0.018932169303297997,0.05785799399018288,0.053443074226379395,0.1664389967918396,0.0004907370894216001,-0.06293363869190216,-0.05056718364357948,-0.04944369196891785,0.054338064044713974,0.031091749668121338,0.10335559397935867,-0.010574855841696262,0.03982764109969139,0.039772484451532364,-0.008298107422888279,0.005360682029277086,-0.059202443808317184,0.018834738060832024,0.013035591691732407,0.0429648831486702,-0.08836061507463455,0.04949995502829552,0.03127415105700493,-0.0325763039290905,-0.06371749192476273,-0.19097256660461426,0.10665379464626312,0.09431447088718414,0.038758955895900726,0.02547534555196762,-0.05339736118912697,0.004904485307633877,-0.1041523888707161,-0.049558185040950775,-0.05146823078393936,0.026056045666337013,-0.1011614054441452,-0.12690967321395874,-0.07830245047807693,-0.08243127912282944,0.19103418290615082,0.10247959941625595,-0.05558522045612335,-0.09377845376729965,-0.05220827832818031,-0.09384256601333618,-0.0020258165895938873,-0.03820214793086052,-0.07661072164773941,-0.0036434270441532135,-0.15511637926101685,-0.0806671530008316,-0.07900930196046829,-0.008049245923757553,0.21306419372558594,0.10610807687044144,0.007993055507540703,-0.09912915527820587,-0.09205685555934906,-0.021867506206035614,0.006637657526880503,0.03213554620742798,0.05838382989168167,0.008541682735085487,-0.07019298523664474,0.025255246087908745,0.019679976627230644,0.13108676671981812,-0.03393205255270004,-0.09949369728565216,-0.03529748693108559,-0.14611271023750305,0.004510949365794659,0.11733004450798035,0.02872941642999649,0.11491793394088745,0.12383835017681122,0.012633497826755047,-0.0956697016954422,-0.06540235877037048,-0.052053991705179214,-0.05391741916537285,-0.04266993701457977,-0.042035140097141266,0.0005741509376093745,0.06081981956958771,0.12115709483623505,-0.07294826954603195,-0.13926543295383453,0.11152171343564987,0.03458911180496216,-0.05180152878165245,0.027238141745328903,0.03972882032394409,0.017300700768828392,-0.07799655199050903,0.11330168694257736,0.0835682600736618,0.0838642492890358,0.1433626115322113,-0.07947124540805817,-0.17410613596439362,-0.02672584168612957,0.14446040987968445,0.0326048880815506,-0.01752929575741291,0.027487119659781456,0.024954581633210182,-0.022676939144730568,-0.11027258634567261,0.008658175356686115,0.025479309260845184,0.12236178666353226,0.13570190966129303,-0.011766240932047367,0.12230245769023895,0.05476543679833412,0.23308303952217102,0.13448411226272583,-0.04269605129957199,-0.0673840120434761,-0.08760901540517807,-0.016711924225091934,-0.055995792150497437,0.12764839828014374,0.064878448843956,0.15680158138275146,0.013837284408509731,0.01244424656033516,0.15502223372459412,0.04459479823708534,0.13112331926822662,-0.11513743549585342,-0.07368814945220947,-0.054553162306547165,0.02836727723479271,0.03377610817551613,0.07149526476860046,0.0679050087928772,0.053063247352838516,0.09724847972393036,0.14978982508182526,0.055488914251327515,0.06453301757574081,0.03875594958662987,0.14323516190052032,-0.0873875841498375,-0.008096848614513874,-0.012251553125679493,0.05438276007771492,-0.022039204835891724,0.05385386943817139,0.010263331234455109,0.048645779490470886,0.03487556055188179,0.1769431233406067,0.09771464765071869,-0.027183515951037407,-0.019228942692279816,-0.013161547482013702,-0.12346933782100677,0.010462320409715176,0.06769215315580368,0.06868869066238403,0.07604615390300751,0.020020347088575363,0.00031422427855432034,0.007944521494209766,0.07354289293289185,0.049156609922647476,0.021949198096990585,0.056657154113054276,-0.03729772940278053,0.09072589874267578,0.0031733252108097076,0.04320478439331055,0.06525004655122757,0.12930931150913239,0.040872249752283096,0.06884992122650146,-0.07456076145172119,0.01999482326209545,0.08302781730890274,0.0383872427046299,0.068098284304142,0.14224593341350555,0.03969378396868706,0.08214706927537918,0.07575644552707672,0.06058704853057861,0.060242269188165665,0.10880204290151596,0.060470886528491974,0.035265322774648666,-0.004281407687813044,0.030255528166890144,0.05788780003786087,0.0832260251045227,0.12130985409021378,0.09348781406879425,0.08175432682037354,0.05901499092578888,0.1159462109208107,0.07076237350702286,0.0009506312781013548,0.12106310576200485,0.08762160688638687,0.03495156392455101,0.06611298024654388,0.06221425160765648,0.044553037732839584,0.01581668294966221,0.043219827115535736,0.06668996065855026,0.03628707677125931,0.13044922053813934,0.08283257484436035,0.09410171210765839,0.09868733584880829,0.03860348090529442,-0.023951126262545586,0.029254047200083733,0.013372530229389668,0.04665315896272659,-0.02700071781873703,0.07786611467599869,0.04604117199778557,0.13517434895038605,0.060385845601558685,0.08232038468122482,0.057041235268116,0.08531008660793304,0.14692868292331696,0.11040113121271133,0.025553906336426735,0.022141285240650177,0.023692019283771515,0.07898031175136566,0.044985201209783554,0.023978963494300842,0.036638833582401276,0.10960272699594498,0.07107215374708176,0.03985855355858803,0.059256743639707565,0.11489006876945496,0.09466519206762314,0.12379760295152664,0.021605173125863075,0.03664088621735573,0.036209285259246826,0.10617763549089432,0.1244678720831871,0.0969899445772171,0.06156833842396736,0.05149344354867935,0.01628817431628704,-0.022558921948075294,-0.024760348722338676,0.1073157861828804,0.1603044718503952,0.20823349058628082,0.16182880103588104,0.0610634870827198,-0.04948730766773224,0.08234108239412308,0.1005479246377945,0.08300187438726425,-0.033352285623550415,0.10098735988140106,0.028806298971176147,0.009874878451228142,-0.04013702645897865,0.04010021314024925,0.01838335022330284,0.05031803250312805,0.05564010888338089,-0.01675763726234436,-0.06724854558706284,-0.018457278609275818,-0.017602335661649704,0.03446478396654129,0.010047565214335918,-0.0800156369805336,-0.04291338101029396,-0.1060243621468544,-0.034426335245370865,0.040123093873262405,0.028286084532737732,-0.018363969400525093,0.04393523558974266,-0.014444323256611824,0.12413472682237625,0.08622974157333374,0.008848822675645351,0.08673801273107529,0.09406311810016632,0.046758078038692474,0.08221226185560226,0.05870762839913368,0.007021677680313587,-0.02405748888850212,-0.011773866601288319,0.0354543961584568,0.079014353454113,0.09301503002643585,0.0864652469754219,0.06915002316236496,0.05968218296766281,0.08842425048351288,0.05101384222507477,0.1002812460064888,0.20082703232765198,0.2033889889717102,0.14530904591083527,0.012682659551501274,-0.013399085029959679,-0.0015927214408293366,0.01814763993024826,0.06287182122468948,0.10092020034790039,0.1103140115737915,0.10374197363853455,0.08465974032878876,0.0814591646194458,0.19772812724113464,0.06580596417188644,0.16740599274635315,0.14240343868732452,0.0002926645101979375,0.08793497085571289,0.03159880265593529,0.10752195119857788,0.09495827555656433,0.06557372212409973,0.1322164535522461,0.13876807689666748,0.0727190226316452,0.0634022206068039,0.17246036231517792,0.06725385785102844,0.12240949273109436,0.08754025399684906,0.04746890068054199,0.017830519005656242,0.05882110446691513,0.09366568922996521,0.07270558178424835,0.12588568031787872,0.09020302444696426,0.013332678936421871,0.07112929224967957,0.09475380182266235,0.04175586998462677,-0.012259097769856453,0.005031961482018232,0.004272553138434887,0.0432596392929554,0.03728967532515526,-0.015003539621829987,0.12361320108175278,0.13948877155780792,0.003223752835765481,-0.002354742493480444,-0.006586858537048101,-0.05040193349123001,0.018235525116324425,-0.05079471692442894,-0.07933821529150009,-0.0535740964114666,-0.00904855690896511,-0.058200299739837646,-0.02177741564810276,-0.05771228298544884,0.029956411570310593,0.025516623631119728,-0.05490299314260483,-0.002763503696769476,-0.054495010524988174,0.019701378419995308,2.0533083443297073e-05,-0.0801699161529541,0.01592894457280636,0.012380135245621204,-0.08683079481124878,0.026574425399303436,-0.04255361109972,0.017547328025102615,-0.014067612588405609,0.02581958658993244,0.04614174738526344,0.004460498690605164,0.026574263349175453,0.024693146347999573,0.05298652872443199,-0.07635695487260818,-0.013581807725131512,0.05890444666147232,0.019162442535161972,0.03996129333972931,0.02704368531703949,-0.010761013254523277,0.06428226083517075,0.1295492798089981,0.006976265460252762,-0.011120648123323917,-0.043212223798036575,0.02236998826265335,0.08301463723182678,0.0675748884677887,0.03106313943862915,0.10149383544921875,0.15114666521549225,0.09551654011011124,0.08272618055343628,0.002774810418486595,-0.016468709334731102,0.08415964990854263,-0.040306828916072845,0.002059748861938715,0.03175059333443642,0.05708778649568558,0.011706620454788208,-0.045411061495542526,-0.09379244595766068,-0.10686030983924866,0.03997673839330673,0.017551502212882042,-0.034225862473249435,-0.0478416308760643,0.018727626651525497,0.047338105738162994,-0.037553902715444565,-0.03517775237560272,-0.02936505153775215,-0.07359006255865097,-0.14261400699615479,-0.15534429252147675,-0.22479531168937683,-0.1912175565958023,-0.14043022692203522,-0.08933082222938538,-0.1175304502248764,-0.050653036683797836,0.0002655696007423103,0.03280966356396675,0.06660879403352737,0.10937371850013733,0.06384143978357315,0.06238791346549988,-0.09322355687618256,-0.09934668242931366,-0.16298595070838928,-0.15826226770877838,-0.05531365051865578,-0.058178067207336426,-0.043677788227796555,-0.02757125534117222,-0.0038896696642041206,0.005508174188435078,0.04806787148118019,0.0825161561369896,0.12961708009243011,0.09443174302577972,0.1311851292848587,-0.003893061773851514,-0.09074082225561142,-0.14574545621871948,-0.20494242012500763,-0.03321627900004387,0.0701078400015831,0.12886597216129303,0.01773826964199543,0.08166053891181946,0.1334608495235443,0.14434950053691864,0.13258972764015198,0.0014500233810395002,-0.08118786662817001,-0.006722771096974611,-0.21129868924617767,-0.23101213574409485,-0.1537092626094818,-0.18059141933918,-0.005594457034021616,-0.0016196607612073421,0.05410988628864288,-0.04033916816115379,0.041253648698329926,-0.0670950636267662,-0.008596403524279594,0.0017757415771484375,-0.06945080310106277,0.11096911877393723,-0.061571404337882996,-0.14667785167694092,-0.13874894380569458,-0.005317382514476776,-0.06999871879816055,-0.0034004917833954096,-0.001668988261371851,0.014685397036373615,-0.10195962339639664,-0.11104674637317657,-0.11239270865917206,-0.17815528810024261,-0.014356546103954315,-0.0022363015450537205,-0.18887817859649658,-0.17330659925937653,-0.15406163036823273,-0.04286888614296913,0.055319882929325104,-0.03257004916667938,-0.06286948174238205,-0.03056088648736477,-0.01881515607237816,-0.08051270246505737,-0.13396857678890228,-0.005230321548879147,0.014680069871246815,-0.04612334445118904,-0.11848792433738708,-0.14438025653362274,-0.09947244822978973,-0.08305259048938751,-0.1385706663131714,-0.1269015073776245,0.03220910206437111,0.08053477853536606,-0.17165860533714294,-0.15070649981498718,-0.13853226602077484,-0.17990168929100037,-0.05913815647363663,-0.11157119274139404,-0.12879285216331482,-0.14271239936351776,-0.1807720810174942,-0.10697797685861588,-0.05649874731898308,-0.06347568333148956,-0.020057175308465958,-0.0932144969701767,-0.0588516928255558,-0.14106568694114685,-0.021430378779768944,-0.05909203365445137,-0.14124177396297455,-0.06164130941033363,-0.102424755692482,-0.07308102399110794,0.019607385620474815,-0.04829748719930649,-0.07342254370450974,-0.06719368696212769,0.011927181854844093,0.057549379765987396,-0.0881223976612091,-0.031701233237981796,-0.028060637414455414,-0.004029995296150446,-0.043034374713897705,-0.006266431882977486,-0.04128941148519516,-0.09654481709003448,-0.014408265240490437,0.011360011994838715,-0.10901353508234024,-0.12109805643558502,-0.06471320986747742,-0.010876938700675964,-0.15726348757743835,-0.06790288537740707,-0.12322982400655746,0.00936658401042223,-0.056237295269966125,-0.021407416090369225,-0.10879171639680862,0.027527596801519394,-0.028500720858573914,-0.05978545919060707,-0.055770065635442734,0.03505229577422142,0.06180298700928688,0.05316289886832237,-0.12054643034934998,-0.059741582721471786,-0.10507241636514664,-0.09074515849351883,-0.1027769073843956,-0.06339672952890396,-0.07597194612026215,0.03161012753844261,-0.02819027006626129,-0.06440882384777069,-0.09338405728340149,0.06464223563671112,0.05387626215815544,-0.052004892379045486]
        }
        
        
        if ( len(r_shapes[layer]) == 4 ):
            
            # Get the Renesas eAI Weights. This eAI weight follow the NCHW format
            #print(r_shapes[layer])
            #print(len(r_weights[layer]))
            np_layer_weights = np.reshape( r_weights[layer], r_shapes[layer])
            np_layer_weights = np.float32( np_layer_weights)
            #print(np_layer_weights[1,:,:,:])
        
            # convert format to Tensorflow NHWC
            tf_layer_weights = np.zeros((r_shapes[layer][2],
                                        r_shapes[layer][3],
                                        r_shapes[layer][1],
                                        r_shapes[layer][0]))
            tf_layer_weights = np.float32(tf_layer_weights)
            #print(tf_layer_weights.shape)

            f = c = h = w = 0
            for F in np_layer_weights:
                for C in F:
                    for H in C:
                        for W in H:
                            tf_layer_weights[h][w][c][f] = W
                            w += 1
                        h += 1
                        w = 0
                    h = 0
                    c += 1
                c = 0
                f +=1
            

        else :
            np_layer_weights = np.array( r_weights[layer])
            np_layer_weights = np_layer_weights.astype('float32')
            tf_layer_weights = np_layer_weights.reshape(r_shapes[layer])
            
        print(tf_layer_weights.shape)    
        return tf.get_variable(weights_name,
                                   initializer=tf_layer_weights)

In [ ]:
def Renesas_eAI_Biases ( layer, bias_name):
        
        r_biases = {
            0:[0.05],
            1:[0.17663578689098358,-0.03062542714178562,-0.005154006648808718,-0.039401449263095856,-0.139275461435318,-0.05490439385175705,0.16115404665470123,0.10268176347017288],
            2:[0.019345805048942566,-0.1250201314687729,0.033643681555986404,-0.003420395078137517,-0.06118509918451309,-0.07596641778945923,0.07035502046346664,-0.04821379855275154,-0.012458331882953644,-0.08418524265289307,-0.11561206728219986,-0.11016890406608582,0.03200647234916687,0.0021318173967301846,-0.009349653497338295,-0.06612539291381836],
            3:[0.019345805048942566,-0.1250201314687729,0.033643681555986404,-0.003420395078137517,-0.06118509918451309,-0.07596641778945923,0.07035502046346664,-0.04821379855275154,-0.012458331882953644,-0.08418524265289307,-0.11561206728219986,-0.11016890406608582,0.03200647234916687,0.0021318173967301846,-0.009349653497338295,-0.06612539291381836],
            4:[-0.025311341509222984,-0.025635316967964172,-0.05854886770248413,-0.10574378818273544,0.08432656526565552,0.07637543976306915,-0.17380240559577942,-0.15757164359092712,0.12110316753387451,0.12155985832214355,0.1214495524764061,-0.07581296563148499,-0.11006369441747665,-0.12113382667303085,-0.12685011327266693,0.027639448642730713],
            5:[0.2119542807340622,-0.02792523428797722,-0.2033587545156479],     
        }
        np_layer_biases = np.array(r_biases[layer])
        np_layer_biases = np_layer_biases.astype('float32')
        return tf.get_variable(bias_name,
                               initializer=np_layer_biases)

In [ ]:
eAI = eAISimple()
eAI.network()

In [ ]:
eAI2 = eAISimple(Renesas_eAI_Weights, Renesas_eAI_Biases)
eAI2.network()

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
def get_weights_variable(layer_name):
    # Retrieve an existing variable named 'weights' in the scope
    # with the given layer_name.
    # This is awkward because the TensorFlow function was
    # really intended for another purpose.

    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('weights')

    return variable

In [ ]:
saver = tf.train.Saver()
save_dir = 'checkpoints/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
save_path = os.path.join(save_dir, 'best_validation')

# Save all variables of the TensorFlow graph to file.
saver.save(sess=session, save_path=save_path)


In [ ]:
session.close()